In [1]:
import os
import pickle
import string

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1

REBUILDDICTIONARYFROMPICKLEFILE = 0

path = 'F:\LaTeX\LatexWork\Bible'
os.chdir(path)
print (os.getcwd())

In [2]:
# %load BibleWordList2.py
'''
Update 21 Deceber 2018, 0004
'''


import os
import pickle
import collections
import operator
import re

DEBUG = 0

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1
REBUILDDICTIONARYFROMPICKLEFILE = 0

punctuations = '''!()_;:,<>?@#$^.'''

exceptions = [
    'A', 'Accept', 'After', 'Aha', 'All','Alas','Also','Arise',
    'Amen', 'An', 'And', 'As', 'At', 'Awake','According',
    'Be', 'Because', 'Before', 'Behold', 'Blessed','Build','Better',
    'Blessings', 'Blind', 'Both', 'Bring', 'But','Beautiful',
    'By', 'Command', 'Concerning', 'Consider', 'Cut','Come',"Children's",
    'Deal', 'Deceit', 'Deep', 'Deliver', 'Depart','Did',
    'Doubtless', 'Each', 'Either', 'Even', 'Every','Excellent','Fear',
    'For', 'From', 'Get', 'Give', 'Go', 'Grace',
    'Great', 'Hatred', 'Having', 'He', 'Hear',
    'Hearing', 'Heaviness', 'Help', 'His', 'Hold',
    'Horror', 'How', 'If', 'In', 'Incline', 'Judge',
    'Leave', 'Let', 'Lo', 'Look', 'Lying', 'Lodge','Make','Must','Mark',
    'Many', 'Mine', 'My', 'Neither', 'Nevertheless','Nay',
    'Not', 'Now', 'Number', 'O', 'Of', 'On', 'One','Out',
    'Only', 'Open', 'Or', 'Order', 'Our', 'Pardon',
    'Plead', 'Princes', 'Quicken', 'Red', 'Remember',
    'Remove', 'Riches', 'Righteous', 'Rise', 'Rivers','Return',
    'Sacrifice', 'Salvation', 'Say', 'Seven', 'Shall','Spake',
    'So', 'Speak', 'Sprinkle', 'Stablish', 'Stand','Strong',
    'Surely', 'Take', 'Teach', 'That', 'The', 'Their',
    'Then', 'There', 'Therefore', 'These', 'They',
    'This', 'Those', 'Thou', 'Though', 'Through',
    'Thus', 'Thy', 'To', 'Treasures', 'Trouble','Told',
    'Turn', 'Two', 'Unless', 'Uphold', 'We',
    'What', 'Whatsoever', 'When', 'Where', 'Wherefore',
    'Wherewithal', 'Which', 'Who', 'Whom', 'Whoso',
    'Whosoever', 'Why', 'Wise', 'With', 'Withhold','Walk','Wisdom',
    'Would', 'Ye', 'Yea', 'Yet', 'Your']

properNameList = [
    'Aaron', 'Abelmizraim', 'Abiasaph', 'Abib', 'Abihu', 'Abraham', 'Abram',
    'Aceldama', 'Achaia', 'Achaicus', 'Achor', 'Achzib', 'Adam', 'Adramyttium',
    'Adria', 'Adullam', 'Agrippa', 'Ahab', 'Ahaz', 'Ahikam', 'Ahisamach',
    'Aholiab', 'Alexandria', 'Alexandrians', 'Alphaeus', 'Amalek', 'Amittai',
    'Ammi ', 'Amminadab', 'Amon', 'Amorite', 'Amorites', 'Amram', 'Anathoth',
    'Andrew', 'Angel', 'Antioch', 'Aphrah', 'Apollos', 'Apphia', 'Aquila',
    'Arabia ', 'Arabian', 'Archippus', 'Ard', 'Areli', 'Aretas', 'Aristarchus',
    'Arnon', 'Arodi', 'Aroer ', 'Asenath', 'Asenath', 'Ashbel', 'Asher',
    'Ashkelon', 'Asia', 'Assir', 'Assyria ', 'Assyrian', 'Atad', 'Augustus',
    'Azariah', 'Azur', 'BRANCH', 'Baal ', 'Baali ', 'Baalim', 'Baalzephon',
    'Bablyon', 'Balaam', 'Balak', 'Barbarian', 'Barjona', 'Barnabas',
    'Barsabas', 'Bartholomew', 'Baruch', 'Becher', 'Beeri', 'Beerlahairoi',
    'Beersheba', 'Belah', 'Belial', 'Belteshazzar', 'Benjamin', 'Beor',
    'Bered', 'Beriah', 'Bernice', 'Bethaven', 'Bethdiblathaim ', 'Bethel',
    'Bethezel', 'Bethgamul', 'Bethlehem', 'Bethmeon', 'Bethshemesh',
    'Bezaleel', 'Bilhah', 'Bozrah', 'Caesar ', 'Caesarea', 'Cainan', 'Cana',
    'Canaan', 'Canaanite', 'Canaanites', 'Canaanitish', 'Capernaum', 'Caphtor',
    'Carchemish', 'Carmi', 'Castor', 'Cephas', 'Chaldeans', 'Chaldeans',
    'Chemosh', 'Chittim', 'Chloe', 'Christ', 'Cilicia', 'Clauda', 'Cnidus',
    'Crete', 'Cretians', 'Crispus', 'Cyprus', 'Cyrenians', 'Cyrus',
    'Damascenes', 'Damascus', 'Dan', 'Daniel', 'Darius', 'David', 'Demas',
    'Diblaim', 'Dibon', 'Didymus', 'Dinah', 'Edom', 'Egypt', 'Egyptian',
    'Egyptians', 'Ehi', 'Elasah', 'Eleazar', 'Elias', 'Eliezer', 'Elim',
    'Elisheba', 'Elkanah', 'Elon', 'Elzaphan', 'Enoch', 'Enos', 'Epaphras',
    'Ephesus', 'Ephraim', 'Ephrath', 'Ephron', 'Er', 'Eri', 'Esaias', 'Esau',
    'Etham', 'Euphrates', 'Euroclydon', 'Ezbon', 'Father', 'Felix', 'Festus',
    'Fortunatus', 'Gad', 'Gadarenes', 'Gaius', 'Galatia', 'Galilee', 'Gath',
    'Gaza', 'Gedaliah', 'Gemariah', 'Gentiles', 'Gera', 'Gershom', 'Gershon',
    'Gibeah', 'Gibeon', 'Gilead', 'Gilgal', 'God', 'Gomer', 'Goshen',
    'Grecians', 'Greek', 'Guni', 'Hagar', 'Haggi', 'Ham', 'Hananiah', 'Hanoch',
    'Heber', 'Hebrew', 'Hebrews', "Hebrews'", 'Hebron', 'Heldai', 'Helem',
    'Hen', 'Heshbon', 'Hezekiah', 'Hezron', 'High', 'Hilkiah', 'Hinnom',
    'Hittite', 'Hittites', 'Hivite', 'Hivites', 'Horeb', 'Horonaim', 'Hosea',
    'Hoshaiah', 'Huppim', 'Hur', 'Hushim', 'I', 'Isaac', 'Ishi', 'Ishmael',
    'Ishmeelites', 'Ishuah', 'Israel', 'Issachar', 'Issachar', 'Isui', 'Italy',
    'Ithamar', 'Ithiel', 'Izhar', 'JEHOVAH', 'Jachin', 'Jacob', 'Jahleel',
    'Jahzeel', 'Jairus', 'Jakeh', 'James', 'Jamin', 'Japheth', 'Jareb:',
    'Jared', 'Jebusite', 'Jebusites', 'Jeconiah', 'Jedaiah', 'Jehoiada',
    'Jehoiakim', 'Jehoshapha', 'Jehovahnissi', 'Jehu', 'Jemuel', 'Jeremiah',
    'Jerusalem', 'Jesus', 'Jethro', 'Jews', 'Jezer', 'Jezreel', 'Jimnah',
    'Joash', 'Job', 'Jochebed', 'Joel', 'Johanan', 'John', 'Jonah', 'Jonas',
    'Joppa', 'Jordan', 'Josedech', 'Joseph', 'Josiah', 'Jotham', 'Judaea',
    'Judah', 'Judas', 'Julius', 'Just ', 'Justus ', 'Kadesh', 'Kareah',
    'Kedar', 'Kerioth', 'King', 'Kiriathaim ', 'Kohath', 'Korah', 'Korhites',
    'LORD', 'Laban', 'Laban', 'Lachis', 'Lamech', 'Laodicea', 'Lasea', 'Leah',
    'Lebanon', 'Legion', 'Levi', 'Libertines', 'Libni', 'Libyans', 'Loammi',
    'Lord', 'Lord Jesus Christ', 'Loruhamah', 'Lucas', 'Luhith', 'Luz', 'Lycia',
    'Lydians', 'Maaseiah', 'Macedonia', 'Macedonian', 'Machir', 'Machpelah',
    'Mahalaleel', 'Mahali', 'Malachi', 'Malchiel', 'Mamre', 'Manasseh',
    'Marcus', 'Mareshah', 'Maroth', 'Massah', 'Matthew', 'Matthias',
    'Melchizedek', 'Melita', 'Melzar', 'Merari', 'Meribah', 'Messias',
    'Methuselah', 'Micah', 'Midian', 'Migdol', 'Miriam', 'Misgab', 'Mishael',
    'Mizpah', 'Moab', 'Morasthite', 'Moreshethgath', 'Moses', 'Muppim', 'Mushi',
    'Myra', 'Naaman', 'Naashon', 'Nadab', 'Naphtali', 'Nathanael', 'Nazareth',
    'Nebo', 'Nebuchadnezzar', 'Nebuchadrezzar', 'Nebuzaradan', 'Nehelamite',
    'Nepheg', 'Neriah', 'Nicanor', 'Nicolas', 'Nicopolis', 'Nineveh', 'No ',
    'Noah', 'Noph', 'Ohad', 'Olivet', 'Omri', 'Onan', 'Onesimus', 'Padan',
    'Padanaram', 'Palestine', 'Pallu', 'Pamphylia', 'Parmenas', 'Pashur',
    'Pathros', 'Paul', 'Pentecost', 'Pergamos', 'Perizzite', 'Perizzites',
    'Peter', 'Pethuel', 'Phallu', 'Pharaoh', "Pharaoh's", 'Pharaohhophra ',
    'Pharaohnecho', 'Pharez', 'Pharisee ', 'Pharisees', 'Phenice',
    'Philadelphia', 'Philemon', 'Philip', 'Philippi', 'Philistines',
    'Phinehas', 'Phuvah', 'Pihahiroth', 'Pilate', 'Pithom', 'Pollux',
    'Potiphar', 'Potipherah', 'Prince', 'Priscilla', 'Prochorus', 'Puah',
    'Publius', 'Puteoli', 'Putiel', 'Raamses', 'Rachel', 'Ramah', 'Rameses',
    'Rephidim', 'Reuben', 'Reuel', 'Rhegium', 'Rock', 'Rome', 'Rosh', 'Ruhamah',
    'Sabeans', 'Sadducees', 'Salem', 'Salmone', 'Samaria', 'Samaritans',
    'Samuel', 'Saphir', 'Sara', 'Sarah', 'Sarai', 'Sardis', 'Satan', 'Saul',
    'Saviour', 'Scythian', 'Selah', 'Sepharad', 'Serah', 'Sered', 'Seth',
    'Shaphan', 'Shaul', 'Shelah', 'Shem', 'Shemaiah', 'Shillem', 'Shiloh',
    'Shimi', 'Shimron', 'Shiphrah', 'Shittim', 'Shuni', 'Sidon', 'Sihon',
    'Sihor', 'Simeon', 'Simon ', 'Simon Zelotes', 'Sinai', 'Sirion', 'Smyrna',
    'Solomon', 'Sosthenes', 'Stephanas', 'Stephen', 'Succoth', 'Sychar',
    'Syracuse', 'Tabor', 'Tahpanhes', 'Tarshish', 'Teman', 'Theophilus',
    'Thessalonica', 'Thomas', 'Thummim', 'Thyatira', 'Tiberias', 'Timon',
    'Timotheus', 'Timothy', 'Titus', 'Tobijah', 'Tola', 'Tophet', 'Tychicus',
    'Tyre', 'Tyrus', 'Ucal', 'Uri', 'Urim', 'Uzziah', 'Uzziel', 'Zaanan',
    'Zaphnathpaaneah', 'Zarephath', 'Zebedee', 'Zebulun', 'Zedekiah',
    'Zephaniah', 'Zerah', 'Zichri', 'Zidon', 'Zilpah', 'Zion', 'Ziphion',
    'Zipporah', 'Zithri', 'Zohar']

##########
def removePunc(input):
    no_punc = ''
    for char in input:
        if char not in punctuations:
            no_punc = no_punc + char
    return no_punc


##########
def generatedLatexFormattedText(dict):
    BUILDAWIP = 1
    BUILDNWIV = 1
    BUILDPNIP = 1
    BUILDNWIC = 1
    BUILDNLIW = 1
    BUILDBPIP = 1
    MINISTRY = 1

    topString = "\\footnote{\\textcolor[cmyk]{0.99998,1,0,0}{\\hyperlink{TOC}{Return to end " \
                "of Table of Contents.}}}"
    textColorString = "\\textcolor[cmyk]{0.99998,1,0,0}{"
    endString = "}"

    '''
    Print plain text with Latex color header:
        \textcolor[rgb]{0.00,0.00,1.00} == BLUE
        \textcolor[cmyk]{0.99998,1,0,0} == A DEEPER BLUE
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with Latex format\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')

    linesWith13Words = []
    linesWith18Words = []
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        newline = line[5]
        if versn2 == '1':
            print(topString + textColorString + newline + endString)
        else:
            print('[' + versn2 + '] ' + textColorString + newline + endString)
        words = newline.split()
        wordLength = len(words)
        if wordLength == 13:
            linesWith13Words.append(line)
        if wordLength == 18:
            linesWith18Words.append(line)
            
        
    '''
    NOW print the verses with LaTeX index items:
        AWIP - All Words in Passage
        NWIV - NUmber of Words in Verse
        PNIP - Proper noun in passage
        ... more later
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with indexed items\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        verseRef = "!" + booknm + "!" + chapnm + " " + chapno + ":" + versno 
        line = line[5]
        if versn2 == '1':
            verseString = topString + textColorString + line + endString
        else:
            verseString = '[' + versn2 + '] ' + textColorString + line + endString
        #print(booknm,chapnm,chapno,versno,versn2) 
        '''
        NWIV
        '''
        words = line.split()
        numWords = len(words)
        if DEBUG == 1:
            print(numWords,'Words in verse',versn2)
        extraString="\\index[NWIV]{" + str(numWords) + verseRef + "}"
        '''
        AWIP - here, Passage = verse
        '''
        verseDict = {}
        wordString = ''
        for word in words:
            word = removePunc(word)
            if word not in verseDict:
                verseDict[word] = 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef + "}"
            else:
                verseDict[word] = verseDict[word] + 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef 
                wordString = wordString + " (" + str(verseDict[word]) + ")}"
        '''
        PNIP - Proper names in passage
        '''
        propernameStrng = ''
        for item in verseDict:
            if item in properNameList:
                if item not in exceptions:
                    propernameStrng = propernameStrng + "\\index[PNIP]{" + item + verseRef + "}"

        builtVerseString = verseString + extraString + wordString  + propernameStrng
        print(builtVerseString)
    return linesWith18Words, linesWith13Words

##########
def showWordsByFrequency(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words by Frequency}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[1], reverse=True)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def showWordsAlphabetically(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words Alphabetically}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[0], reverse=False)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def generateWordStatisticsForChapter(label,verses):
    '''
    Need to simplify this code! ONLY count number of words, number of unique 
    words, number of itallic words, number of unique itallic words
    
    verseDictionary, rest for each verse
    chapterDictionary
    '''
    global properNameList
    wordCount = 0
    emphasizedwordCount = 0
    wordDict = {}
    emphasizedwordDict = {}
    verseString = ''
    A = re.findall(r'\d+', label)
    A = " ".join(map(str, A))
    digit = int(A)

    word1 = " ".join(re.findall("[a-zA-Z]+", label))
    
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('%%%%% Word Statistics\n%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    
    print('\\normalsize')

    verseStats = ["\\subsection{Chapter Word Statistics}",
                  "\n",
                  "%%%%%%%%%%",
                  "%%%%%%%%%%",
                  " ",
                  "\\begin{center}",
                  "\\begin{longtable}{l|c|c|c|c}",
                  "\\caption[Stats for "+word1+" "+str(digit) +"]{Stats for "+word1+" "+str(digit) +"} \\label{table:Stats for "+word1+" "+str(digit) +"} \\\ ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endfirsthead",
                  " ",
                  "\multicolumn{5}{c}",
                  "{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\  ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endhead",
                  " ",
                  "\\hline \\multicolumn{5}{|r|}{{Continued if needed}} \\\ \\hline",
                  "\\endfoot ",]

    '''
    This section prints JUST the Bible text in a LaTeX format with a textcolor encoding added
    Original Code
    '''
    chapterDict = {}
    chapwordWords = 0
    chapterUniqueWords = 0
    numberChapterUniqueItallics = 0
    numberChapterItallics = 0
    verseNo = 0
    for line in verses:
        verseNo += 1
        verseDict = {}
#        print(line)
        words = removePunc(line[5])
        wordcount = 0
        uniqueWords = 0        
        '''
        Process for verse
        '''
        for word in words.split():
            wordcount += 1
            if word not in verseDict:
                verseDict[word] = 1
                uniqueWords += 1
            else:
                verseDict[word] = verseDict[word] + 1
        numberUniqueItallics = 0
        numberItallics = 0
        for item in verseDict:
            if "\emph{" in item:
                numberUniqueItallics += 1
                numberItallics = numberItallics + verseDict[item]                
        '''
        Process for chapter
        '''                
        for word in words.split():
            chapwordWords += 1
            if word not in chapterDict:
                chapterDict[word] = 1
                chapterUniqueWords += 1
            else:
                chapterDict[word] = chapterDict[word] + 1
        sequence = str(verseNo),str(wordcount),str(uniqueWords),str(numberItallics),str(numberUniqueItallics)+'\\\ \hline'
        S = ' & '.join(sequence)
        verseStats.append(S)
    for item in chapterDict:
        if "\emph{" in item:
            numberChapterUniqueItallics +=1
            numberChapterItallics = numberChapterItallics + chapterDict[item]

    '''
    Finish Up!
    '''
#    print ("\n",wordCount,"words in passage, of which",len(wordDict),"are unique")
#    print (emphasizedwordCount,'words in ital, of which',len(emphasizedwordDict),'are unique')
    print ("\n\n")
    verseStats.append("\hline \hline")
    sequence = 'Total',str(chapwordWords),str(chapterUniqueWords),str(numberChapterItallics),str(numberChapterUniqueItallics)
    S = ' & '.join(sequence)
    verseStats.append(S)
    for item in verseStats:
        print (item)
    print ("\n\n")
    print ("\\end{longtable}")
    print ("\\end{center}\n")
#    print (chapwordWords,"words in passage, of which",chapterUniqueWords,"are unique")
#    print (numberChapterItallics,'words in ital, of which',numberChapterUniqueItallics,'are unique')
#    print ("\n\n")
#
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
    
    return chapterDict

##########
def getWordLengthsInChapter(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    wordLengthDictionary = {}
    for item in dictionary:
        if "\emph{" in item:
            tempitem = item.replace("\emph{",'')
            tempitem = tempitem.replace('}','')
            if len(tempitem) in wordLengthDictionary:
                wordLengthDictionary[len(tempitem)] = wordLengthDictionary[len(tempitem)] + ', ' + item
            else:
                    wordLengthDictionary[len(tempitem)] = item
        else:
            if len(item) in wordLengthDictionary:
                wordLengthDictionary[len(item)] = wordLengthDictionary[len(item)] + ', ' + item
            else:
                wordLengthDictionary[len(item)] = item
    wordLengthDictionary = sorted(wordLengthDictionary.items(), key=lambda x:x[0], reverse=False)

    print('\\subsection{Word Lengths in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"]{Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in wordLengthDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")

##########
def makeVersesWith13WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 13 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 13 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########
          
##########
def makeVersesWith18WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 18 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 18 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########

def countLettersInVerses(arg1,arg2):
    for item in arg2:
        letterDict = {}
        for item2 in item[5]:
            if "\emph" in item2:
                item2 = item2.replace('\emph','')
            if item2 != ' ':
                if item2 in letterDict:
                    letterDict[item2] = letterDict[item2] + 1
                else:
                    letterDict[item2] =  1
            print ("for verse",item[2],":",letterDict)

def ProcessVerses(arg1,arg2):
    ''' 
    arg1 is dictionary name and arg2 is the dictionary
    '''
#    saveToPickleFile(arg1,arg2)  
    linesWith18words, linesWith13words = generatedLatexFormattedText(arg2)         # format chapter in Latex
    #linesWith13words is for use later ... 
    chapterDict = generateWordStatisticsForChapter(arg1,arg2)   # do chapter word statistics
    showWordsByFrequency(chapterDict,arg1)                      # show words by frequency
    showWordsAlphabetically(chapterDict,arg1)                   # show words alphabetically
    getWordLengthsInChapter(chapterDict,arg1)                   # show word lengths in chapter
    
    if len(linesWith13words) > 0:
        #print(linesWith13words)
        makeVersesWith13WordsTable(linesWith13words,arg1)
        
    if len(linesWith18words) > 0:
        #print(linesWith13words)
        makeVersesWith18WordsTable(linesWith18words,arg1)
        
    ''' In Development
    countLettersInVerses(arg1,arg2)
    '''



In [3]:
path = 'F:\python'
os.chdir(path)
print (os.getcwd())

F:\python


In [4]:
# %load NEW05OT-MakeConcordance-Deuteronomy.py
"""
Created on Sun Jan 01 07:52:18 2017

@author: keith
"""

import os
import pickle
import string

from BibleWordList2 import *

BUILDAWIP = 1
BUILDNWIV = 1
BUILDPNIP = 1
BUILDNWIC = 1
BUILDNLIW = 1
BUILDBPIP = 1

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1

REBUILDDICTIONARYFROMPICKLEFILE = 0

Deuteronomy01 = [
    ['Deuteronomy','Deu','01','001','1',"These \emph{be} the words which Moses spake unto all Israel on this side Jordan in the wilderness, in the plain over against the Red \emph{sea}, between Paran, and Tophel, and Laban, and Hazeroth, and Dizahab."],
['Deuteronomy','Deu','01','002','2',"\emph{(There} \emph{are} eleven days' \emph{journey} from Horeb by the way of mount Seir unto Kadesh-barnea.)"],
['Deuteronomy','Deu','01','003','3',"And it came to pass in the fortieth year, in the eleventh month, on the first \emph{day} of the month, \emph{that} Moses spake unto the children of Israel, according unto all that the LORD had given him in commandment unto them;"],
['Deuteronomy','Deu','01','004','4',"After he had slain Sihon the king of the Amorites, which dwelt in Heshbon, and Og the king of Bashan, which dwelt at Astaroth in Edrei:"],
['Deuteronomy','Deu','01','005','5',"On this side Jordan, in the land of Moab, began Moses to declare this law, saying,"],
['Deuteronomy','Deu','01','006','6',"The LORD our God spake unto us in Horeb, saying, Ye have dwelt long enough in this mount:"],
['Deuteronomy','Deu','01','007','7',"Turn you, and take your journey, and go to the mount of the Amorites, and unto all \emph{the} \emph{places} nigh thereunto, in the plain, in the hills, and in the vale, and in the south, and by the sea side, to the land of the Canaanites, and unto Lebanon, unto the great river, the river Euphrates."],
['Deuteronomy','Deu','01','008','8',"Behold, I have set the land before you: go in and possess the land which the LORD sware unto your fathers, Abraham, Isaac, and Jacob, to give unto them and to their seed after them."],
['Deuteronomy','Deu','01','009','9',"And I spake unto you at that time, saying, I am not able to bear you myself alone:"],
['Deuteronomy','Deu','01','010','10',"The LORD your God hath multiplied you, and, behold, ye \emph{are} this day as the stars of heaven for multitude."],
['Deuteronomy','Deu','01','011','11',"(The LORD God of your fathers make you a thousand times so many more as ye \emph{are}, and bless you, as he hath promised you!)"],
['Deuteronomy','Deu','01','012','12',"How can I myself alone bear your cumbrance, and your burden, and your strife?"],
['Deuteronomy','Deu','01','013','13',"Take you wise men, and understanding, and known among your tribes, and I will make them rulers over you."],
['Deuteronomy','Deu','01','014','14',"And ye answered me, and said, The thing which thou hast spoken \emph{is} good \emph{for} \emph{us} to do."],
['Deuteronomy','Deu','01','015','15',"So I took the chief of your tribes, wise men, and known, and made them heads over you, captains over thousands, and captains over hundreds, and captains over fifties, and captains over tens, and officers among your tribes."],
['Deuteronomy','Deu','01','016','16',"And I charged your judges at that time, saying, Hear \emph{the} \emph{causes} between your brethren, and judge righteously between \emph{every} man and his brother, and the stranger \emph{that} \emph{is} with him."],
['Deuteronomy','Deu','01','017','17',"Ye shall not respect persons in judgment; \emph{but} ye shall hear the small as well as the great; ye shall not be afraid of the face of man; for the judgment \emph{is} God's: and the cause that is too hard for you, bring \emph{it} unto me, and I will hear it."],
['Deuteronomy','Deu','01','018','18',"And I commanded you at that time all the things which ye should do."],
['Deuteronomy','Deu','01','019','19',"And when we departed from Horeb, we went through all that great and terrible wilderness, which ye saw by the way of the mountain of the Amorites, as the LORD our God commanded us; and we came to Kadesh-barnea."],
['Deuteronomy','Deu','01','020','20',"And I said unto you, Ye are come unto the mountain of the Amorites, which the LORD our God doth give unto us."],
['Deuteronomy','Deu','01','021','21',"Behold, the LORD thy God hath set the land before thee: go up \emph{and} possess \emph{it}, as the LORD God of thy fathers hath said unto thee; fear not, neither be discouraged."],
['Deuteronomy','Deu','01','022','22',"And ye came near unto me every one of you, and said, We will send men before us, and they shall search us out the land, and bring us word again by what way we must go up, and into what cities we shall come."],
['Deuteronomy','Deu','01','023','23',"And the saying pleased me well: and I took twelve men of you, one of a tribe:"],
['Deuteronomy','Deu','01','024','24',"And they turned and went up into the mountain, and came unto the valley of Eshcol, and searched it out."],
['Deuteronomy','Deu','01','025','25',"And they took of the fruit of the land in their hands, and brought \emph{it} down unto us, and brought us word again, and said, \emph{It} \emph{is} a good land which the LORD our God doth give us."],
['Deuteronomy','Deu','01','026','26',"Notwithstanding ye would not go up, but rebelled against the commandment of the LORD your God:"],
['Deuteronomy','Deu','01','027','27',"And ye murmured in your tents, and said, Because the LORD hated us, he hath brought us forth out of the land of Egypt, to deliver us into the hand of the Amorites, to destroy us."],
['Deuteronomy','Deu','01','028','28',"Whither shall we go up? our brethren have discouraged our heart, saying, The people \emph{is} greater and taller than we; the cities \emph{are} great and walled up to heaven; and moreover we have seen the sons of the Anakims there."],
['Deuteronomy','Deu','01','029','29',"Then I said unto you, Dread not, neither be afraid of them."],
['Deuteronomy','Deu','01','030','30',"The LORD your God which goeth before you, he shall fight for you, according to all that he did for you in Egypt before your eyes;"],
['Deuteronomy','Deu','01','031','31',"And in the wilderness, where thou hast seen how that the LORD thy God bare thee, as a man doth bear his son, in all the way that ye went, until ye came into this place."],
['Deuteronomy','Deu','01','032','32',"Yet in this thing ye did not believe the LORD your God,"],
['Deuteronomy','Deu','01','033','33',"Who went in the way before you, to search you out a place to pitch your tents \emph{in}, in fire by night, to shew you by what way ye should go, and in a cloud by day."],
['Deuteronomy','Deu','01','034','34',"And the LORD heard the voice of your words, and was wroth, and sware, saying,"],
['Deuteronomy','Deu','01','035','35',"Surely there shall not one of these men of this evil generation see that good land, which I sware to give unto your fathers,"],
['Deuteronomy','Deu','01','036','36',"Save Caleb the son of Jephunneh; he shall see it, and to him will I give the land that he hath trodden upon, and to his children, because he hath wholly followed the LORD."],
['Deuteronomy','Deu','01','037','37',"Also the LORD was angry with me for your sakes, saying, Thou also shalt not go in thither."],
['Deuteronomy','Deu','01','038','38',"\emph{But} Joshua the son of Nun, which standeth before thee, he shall go in thither: encourage him: for he shall cause Israel to inherit it."],
['Deuteronomy','Deu','01','039','39',"Moreover your little ones, which ye said should be a prey, and your children, which in that day had no knowledge between good and evil, they shall go in thither, and unto them will I give it, and they shall possess it."],
['Deuteronomy','Deu','01','040','40',"But \emph{as} \emph{for} you, turn you, and take your journey into the wilderness by the way of the Red sea."],
['Deuteronomy','Deu','01','041','41',"Then ye answered and said unto me, We have sinned against the LORD, we will go up and fight, according to all that the LORD our God commanded us. And when ye had girded on every man his weapons of war, ye were ready to go up into the hill."],
['Deuteronomy','Deu','01','042','42',"And the LORD said unto me, Say unto them, Go not up, neither fight; for I \emph{am} not among you; lest ye be smitten before your enemies."],
['Deuteronomy','Deu','01','043','43',"So I spake unto you; and ye would not hear, but rebelled against the commandment of the LORD, and went presumptuously up into the hill."],
['Deuteronomy','Deu','01','044','44',"And the Amorites, which dwelt in that mountain, came out against you, and chased you, as bees do, and destroyed you in Seir, \emph{even} unto Hormah."],
['Deuteronomy','Deu','01','045','45',"And ye returned and wept before the LORD; but the LORD would not hearken to your voice, nor give ear unto you."],
['Deuteronomy','Deu','01','046','46',"So ye abode in Kadesh many days, according unto the days that ye abode \emph{there}."]
]

Deuteronomy02 = [
    ['Deuteronomy','Deu','02','001','1',"Then we turned, and took our journey into the wilderness by the way of the Red sea, as the LORD spake unto me: and we compassed mount Seir many days."],
['Deuteronomy','Deu','02','002','2',"And the LORD spake unto me, saying,"],
['Deuteronomy','Deu','02','003','3',"Ye have compassed this mountain long enough: turn you northward."],
['Deuteronomy','Deu','02','004','4',"And command thou the people, saying, Ye \emph{are} to pass through the coast of your brethren the children of Esau, which dwell in Seir; and they shall be afraid of you: take ye good heed unto yourselves therefore:"],
['Deuteronomy','Deu','02','005','5',"Meddle not with them; for I will not give you of their land, no, not so much as a foot breadth; because I have given mount Seir unto Esau \emph{for} a possession."],
['Deuteronomy','Deu','02','006','6',"Ye shall buy meat of them for money, that ye may eat; and ye shall also buy water of them for money, that ye may drink."],
['Deuteronomy','Deu','02','007','7',"For the LORD thy God hath blessed thee in all the works of thy hand: he knoweth thy walking through this great wilderness: these forty years the LORD thy God \emph{hath} \emph{been} with thee; thou hast lacked nothing."],
['Deuteronomy','Deu','02','008','8',"And when we passed by from our brethren the children of Esau, which dwelt in Seir, through the way of the plain from Elath, and from Ezion-gaber, we turned and passed by the way of the wilderness of Moab."],
['Deuteronomy','Deu','02','009','9',"And the LORD said unto me, Distress not the Moabites, neither contend with them in battle: for I will not give thee of their land \emph{for} a possession; because I have given Ar unto the children of Lot \emph{for} a possession."],
['Deuteronomy','Deu','02','010','10',"The Emims dwelt therein in times past, a people great, and many, and tall, as the Anakims;"],
['Deuteronomy','Deu','02','011','11',"Which also were accounted giants, as the Anakims; but the Moabites call them Emims."],
['Deuteronomy','Deu','02','012','12',"The Horims also dwelt in Seir beforetime; but the children of Esau succeeded them, when they had destroyed them from before them, and dwelt in their stead; as Israel did unto the land of his possession, which the LORD gave unto them."],
['Deuteronomy','Deu','02','013','13',"Now rise up, \emph{said} \emph{I}, and get you over the brook Zered. And we went over the brook Zered."],
['Deuteronomy','Deu','02','014','14',"And the space in which we came from Kadesh-barnea, until we were come over the brook Zered, \emph{was} thirty and eight years; until all the generation of the men of war were wasted out from among the host, as the LORD sware unto them."],
['Deuteronomy','Deu','02','015','15',"For indeed the hand of the LORD was against them, to destroy them from among the host, until they were consumed."],
['Deuteronomy','Deu','02','016','16',"So it came to pass, when all the men of war were consumed and dead from among the people,"],
['Deuteronomy','Deu','02','017','17',"That the LORD spake unto me, saying,"],
['Deuteronomy','Deu','02','018','18',"Thou art to pass over through Ar, the coast of Moab, this day:"],
['Deuteronomy','Deu','02','019','19',"And \emph{when} thou comest nigh over against the children of Ammon, distress them not, nor meddle with them: for I will not give thee of the land of the children of Ammon \emph{any} possession; because I have given it unto the children of Lot \emph{for} a possession."],
['Deuteronomy','Deu','02','020','20',"(That also was accounted a land of giants: giants dwelt therein in old time; and the Ammonites call them Zamzummims;"],
['Deuteronomy','Deu','02','021','21',"A people great, and many, and tall, as the Anakims; but the LORD destroyed them before them; and they succeeded them, and dwelt in their stead:"],
['Deuteronomy','Deu','02','022','22',"As he did to the children of Esau, which dwelt in Seir, when he destroyed the Horims from before them; and they succeeded them, and dwelt in their stead even unto this day:"],
['Deuteronomy','Deu','02','023','23',"And the Avims which dwelt in Hazerim, \emph{even} unto Azzah, the Caphtorims, which came forth out of Caphtor, destroyed them, and dwelt in their stead.)"],
['Deuteronomy','Deu','02','024','24',"Rise ye up, take your journey, and pass over the river Arnon: behold, I have given into thine hand Sihon the Amorite, king of Heshbon, and his land: begin to possess \emph{it}, and contend with him in battle."],
['Deuteronomy','Deu','02','025','25',"This day will I begin to put the dread of thee and the fear of thee upon the nations \emph{that} \emph{are} under the whole heaven, who shall hear report of thee, and shall tremble, and be in anguish because of thee."],
['Deuteronomy','Deu','02','026','26',"And I sent messengers out of the wilderness of Kedemoth unto Sihon king of Heshbon with words of peace, saying,"],
['Deuteronomy','Deu','02','027','27',"Let me pass through thy land: I will go along by the high way, I will neither turn unto the right hand nor to the left."],
['Deuteronomy','Deu','02','028','28',"Thou shalt sell me meat for money, that I may eat; and give me water for money, that I may drink: only I will pass through on my feet;"],
['Deuteronomy','Deu','02','029','29',"(As the children of Esau which dwell in Seir, and the Moabites which dwell in Ar, did unto me;) until I shall pass over Jordan into the land which the LORD our God giveth us."],
['Deuteronomy','Deu','02','030','30',"But Sihon king of Heshbon would not let us pass by him: for the LORD thy God hardened his spirit, and made his heart obstinate, that he might deliver him into thy hand, as \emph{appeareth} this day."],
['Deuteronomy','Deu','02','031','31',"And the LORD said unto me, Behold, I have begun to give Sihon and his land before thee: begin to possess, that thou mayest inherit his land."],
['Deuteronomy','Deu','02','032','32',"Then Sihon came out against us, he and all his people, to fight at Jahaz."],
['Deuteronomy','Deu','02','033','33',"And the LORD our God delivered him before us; and we smote him, and his sons, and all his people."],
['Deuteronomy','Deu','02','034','34',"And we took all his cities at that time, and utterly destroyed the men, and the women, and the little ones, of every city, we left none to remain:"],
['Deuteronomy','Deu','02','035','35',"Only the cattle we took for a prey unto ourselves, and the spoil of the cities which we took."],
['Deuteronomy','Deu','02','036','36',"From Aroer, which \emph{is} by the brink of the river of Arnon, and \emph{from} the city that \emph{is} by the river, even unto Gilead, there was not one city too strong for us: the LORD our God delivered all unto us:"],
['Deuteronomy','Deu','02','037','37',"Only unto the land of the children of Ammon thou camest not, \emph{nor} unto any place of the river Jabbok, nor unto the cities in the mountains, nor unto whatsoever the LORD our God forbad us."]
]

Deuteronomy03 = [
    ['Deuteronomy','Deu','03','001','1',"Then we turned, and went up the way to Bashan: and Og the king of Bashan came out against us, he and all his people, to battle at Edrei."],
['Deuteronomy','Deu','03','002','2',"And the LORD said unto me, Fear him not: for I will deliver him, and all his people, and his land, into thy hand; and thou shalt do unto him as thou didst unto Sihon king of the Amorites, which dwelt at Heshbon."],
['Deuteronomy','Deu','03','003','3',"So the LORD our God delivered into our hands Og also, the king of Bashan, and all his people: and we smote him until none was left to him remaining."],
['Deuteronomy','Deu','03','004','4',"And we took all his cities at that time, there was not a city which we took not from them, threescore cities, all the region of Argob, the kingdom of Og in Bashan."],
['Deuteronomy','Deu','03','005','5',"All these cities \emph{were} fenced with high walls, gates, and bars; beside unwalled towns a great many."],
['Deuteronomy','Deu','03','006','6',"And we utterly destroyed them, as we did unto Sihon king of Heshbon, utterly destroying the men, women, and children, of every city."],
['Deuteronomy','Deu','03','007','7',"But all the cattle, and the spoil of the cities, we took for a prey to ourselves."],
['Deuteronomy','Deu','03','008','8',"And we took at that time out of the hand of the two kings of the Amorites the land that \emph{was} on this side Jordan, from the river of Arnon unto mount Hermon;"],
['Deuteronomy','Deu','03','009','9',"\emph{(Which} Hermon the Sidonians call Sirion; and the Amorites call it Shenir;)"],
['Deuteronomy','Deu','03','010','10',"All the cities of the plain, and all Gilead, and all Bashan, unto Salchah and Edrei, cities of the kingdom of Og in Bashan."],
['Deuteronomy','Deu','03','011','11',"For only Og king of Bashan remained of the remnant of giants; behold, his bedstead \emph{was} a bedstead of iron; \emph{is} it not in Rabbath of the children of Ammon? nine cubits \emph{was} the length thereof, and four cubits the breadth of it, after the cubit of a man."],
['Deuteronomy','Deu','03','012','12',"And this land, \emph{which} we possessed at that time, from Aroer, which \emph{is} by the river Arnon, and half mount Gilead, and the cities thereof, gave I unto the Reubenites and to the Gadites."],
['Deuteronomy','Deu','03','013','13',"And the rest of Gilead, and all Bashan, \emph{being} the kingdom of Og, gave I unto the half tribe of Manasseh; all the region of Argob, with all Bashan, which was called the land of giants."],
['Deuteronomy','Deu','03','014','14',"Jair the son of Manasseh took all the country of Argob unto the coasts of Geshuri and Maachathi; and called them after his own name, Bashan-havoth-jair, unto this day."],
['Deuteronomy','Deu','03','015','15',"And I gave Gilead unto Machir."],
['Deuteronomy','Deu','03','016','16',"And unto the Reubenites and unto the Gadites I gave from Gilead even unto the river Arnon half the valley, and the border even unto the river Jabbok, \emph{which} \emph{is} the border of the children of Ammon;"],
['Deuteronomy','Deu','03','017','17',"The plain also, and Jordan, and the coast \emph{thereof}, from Chinnereth even unto the sea of the plain, \emph{even} the salt sea, under Ashdoth-pisgah eastward."],
['Deuteronomy','Deu','03','018','18',"And I commanded you at that time, saying, The LORD your God hath given you this land to possess it: ye shall pass over armed before your brethren the children of Israel, all \emph{that} \emph{are} meet for the war."],
['Deuteronomy','Deu','03','019','19',"But your wives, and your little ones, and your cattle, \emph{(for} I know that ye have much cattle,) shall abide in your cities which I have given you;"],
['Deuteronomy','Deu','03','020','20',"Until the LORD have given rest unto your brethren, as well as unto you, and \emph{until} they also possess the land which the LORD your God hath given them beyond Jordan: and \emph{then} shall ye return every man unto his possession, which I have given you."],
['Deuteronomy','Deu','03','021','21',"And I commanded Joshua at that time, saying, Thine eyes have seen all that the LORD your God hath done unto these two kings: so shall the LORD do unto all the kingdoms whither thou passest."],
['Deuteronomy','Deu','03','022','22',"Ye shall not fear them: for the LORD your God he shall fight for you."],
['Deuteronomy','Deu','03','023','23',"And I besought the LORD at that time, saying,"],
['Deuteronomy','Deu','03','024','24',"O Lord GOD, thou hast begun to shew thy servant thy greatness, and thy mighty hand: for what God \emph{is} \emph{there} in heaven or in earth, that can do according to thy works, and according to thy might?"],
['Deuteronomy','Deu','03','025','25',"I pray thee, let me go over, and see the good land that \emph{is} beyond Jordan, that goodly mountain, and Lebanon."],
['Deuteronomy','Deu','03','026','26',"But the LORD was wroth with me for your sakes, and would not hear me: and the LORD said unto me, Let it suffice thee; speak no more unto me of this matter."],
['Deuteronomy','Deu','03','027','27',"Get thee up into the top of Pisgah, and lift up thine eyes westward, and northward, and southward, and eastward, and behold \emph{it} with thine eyes: for thou shalt not go over this Jordan."],
['Deuteronomy','Deu','03','028','28',"But charge Joshua, and encourage him, and strengthen him: for he shall go over before this people, and he shall cause them to inherit the land which thou shalt see."],
['Deuteronomy','Deu','03','029','29',"So we abode in the valley over against Beth-peor."]
]

Deuteronomy04 = [
    ['Deuteronomy','Deu','04','001','1',"Now therefore hearken, O Israel, unto the statutes and unto the judgments, which I teach you, for to do \emph{them}, that ye may live, and go in and possess the land which the LORD God of your fathers giveth you."],
['Deuteronomy','Deu','04','002','2',"Ye shall not add unto the word which I command you, neither shall ye diminish \emph{ought} from it, that ye may keep the commandments of the LORD your God which I command you."],
['Deuteronomy','Deu','04','003','3',"Your eyes have seen what the LORD did because of Baal-peor: for all the men that followed Baal-peor, the LORD thy God hath destroyed them from among you."],
['Deuteronomy','Deu','04','004','4',"But ye that did cleave unto the LORD your God \emph{are} alive every one of you this day."],
['Deuteronomy','Deu','04','005','5',"Behold, I have taught you statutes and judgments, even as the LORD my God commanded me, that ye should do so in the land whither ye go to possess it."],
['Deuteronomy','Deu','04','006','6',"Keep therefore and do \emph{them}; for this \emph{is} your wisdom and your understanding in the sight of the nations, which shall hear all these statutes, and say, Surely this great nation \emph{is} a wise and understanding people."],
['Deuteronomy','Deu','04','007','7',"For what nation \emph{is} \emph{there} \emph{so} great, who \emph{hath} God \emph{so} nigh unto them, as the LORD our God \emph{is} in all \emph{things} \emph{that} we call upon him \emph{for}?"],
['Deuteronomy','Deu','04','008','8',"And what nation \emph{is} \emph{there} \emph{so} great, that hath statutes and judgments \emph{so} righteous as all this law, which I set before you this day?"],
['Deuteronomy','Deu','04','009','9',"Only take heed to thyself, and keep thy soul diligently, lest thou forget the things which thine eyes have seen, and lest they depart from thy heart all the days of thy life: but teach them thy sons, and thy sons' sons;"],
['Deuteronomy','Deu','04','010','10',"\emph{Specially} the day that thou stoodest before the LORD thy God in Horeb, when the LORD said unto me, Gather me the people together, and I will make them hear my words, that they may learn to fear me all the days that they shall live upon the earth, and \emph{that} they may teach their children."],
['Deuteronomy','Deu','04','011','11',"And ye came near and stood under the mountain; and the mountain burned with fire unto the midst of heaven, with darkness, clouds, and thick darkness."],
['Deuteronomy','Deu','04','012','12',"And the LORD spake unto you out of the midst of the fire: ye heard the voice of the words, but saw no similitude; only \emph{ye} \emph{heard} a voice."],
['Deuteronomy','Deu','04','013','13',"And he declared unto you his covenant, which he commanded you to perform, \emph{even} ten commandments; and he wrote them upon two tables of stone."],
['Deuteronomy','Deu','04','014','14',"And the LORD commanded me at that time to teach you statutes and judgments, that ye might do them in the land whither ye go over to possess it."],
['Deuteronomy','Deu','04','015','15',"Take ye therefore good heed unto yourselves; for ye saw no manner of similitude on the day \emph{that} the LORD spake unto you in Horeb out of the midst of the fire:"],
['Deuteronomy','Deu','04','016','16',"Lest ye corrupt \emph{yourselves}, and make you a graven image, the similitude of any figure, the likeness of male or female,"],
['Deuteronomy','Deu','04','017','17',"The likeness of any beast that \emph{is} on the earth, the likeness of any winged fowl that flieth in the air,"],
['Deuteronomy','Deu','04','018','18',"The likeness of any thing that creepeth on the ground, the likeness of any fish that \emph{is} in the waters beneath the earth:"],
['Deuteronomy','Deu','04','019','19',"And lest thou lift up thine eyes unto heaven, and when thou seest the sun, and the moon, and the stars, \emph{even} all the host of heaven, shouldest be driven to worship them, and serve them, which the LORD thy God hath divided unto all nations under the whole heaven."],
['Deuteronomy','Deu','04','020','20',"But the LORD hath taken you, and brought you forth out of the iron furnace, \emph{even} out of Egypt, to be unto him a people of inheritance, as \emph{ye} \emph{are} this day."],
['Deuteronomy','Deu','04','021','21',"Furthermore the LORD was angry with me for your sakes, and sware that I should not go over Jordan, and that I should not go in unto that good land, which the LORD thy God giveth thee \emph{for} an inheritance:"],
['Deuteronomy','Deu','04','022','22',"But I must die in this land, I must not go over Jordan: but ye shall go over, and possess that good land."],
['Deuteronomy','Deu','04','023','23',"Take heed unto yourselves, lest ye forget the covenant of the LORD your God, which he made with you, and make you a graven image, \emph{or} the likeness of any \emph{thing}, which the LORD thy God hath forbidden thee."],
['Deuteronomy','Deu','04','024','24',"For the LORD thy God \emph{is} a consuming fire, \emph{even} a jealous God."],
['Deuteronomy','Deu','04','025','25',"When thou shalt beget children, and children's children, and ye shall have remained long in the land, and shall corrupt \emph{yourselves}, and make a graven image, \emph{or} the likeness of any \emph{thing}, and shall do evil in the sight of the LORD thy God, to provoke him to anger:"],
['Deuteronomy','Deu','04','026','26',"I call heaven and earth to witness against you this day, that ye shall soon utterly perish from off the land whereunto ye go over Jordan to possess it; ye shall not prolong \emph{your} days upon it, but shall utterly be destroyed."],
['Deuteronomy','Deu','04','027','27',"And the LORD shall scatter you among the nations, and ye shall be left few in number among the heathen, whither the LORD shall lead you."],
['Deuteronomy','Deu','04','028','28',"And there ye shall serve gods, the work of men's hands, wood and stone, which neither see, nor hear, nor eat, nor smell."],
['Deuteronomy','Deu','04','029','29',"But if from thence thou shalt seek the LORD thy God, thou shalt find \emph{him}, if thou seek him with all thy heart and with all thy soul."],
['Deuteronomy','Deu','04','030','30',"When thou art in tribulation, and all these things are come upon thee, \emph{even} in the latter days, if thou turn to the LORD thy God, and shalt be obedient unto his voice;"],
['Deuteronomy','Deu','04','031','31',"(For the LORD thy God \emph{is} a merciful God;) he will not forsake thee, neither destroy thee, nor forget the covenant of thy fathers which he sware unto them."],
['Deuteronomy','Deu','04','032','32',"For ask now of the days that are past, which were before thee, since the day that God created man upon the earth, and \emph{ask} from the one side of heaven unto the other, whether there hath been \emph{any} \emph{such} \emph{thing} as this great thing \emph{is}, or hath been heard like it?"],
['Deuteronomy','Deu','04','033','33',"Did \emph{ever} people hear the voice of God speaking out of the midst of the fire, as thou hast heard, and live?"],
['Deuteronomy','Deu','04','034','34',"Or hath God assayed to go \emph{and} take him a nation from the midst of \emph{another} nation, by temptations, by signs, and by wonders, and by war, and by a mighty hand, and by a stretched out arm, and by great terrors, according to all that the LORD your God did for you in Egypt before your eyes?"],
['Deuteronomy','Deu','04','035','35',"Unto thee it was shewed, that thou mightest know that the LORD he \emph{is} God; \emph{there} \emph{is} none else beside him."],
['Deuteronomy','Deu','04','036','36',"Out of heaven he made thee to hear his voice, that he might instruct thee: and upon earth he shewed thee his great fire; and thou heardest his words out of the midst of the fire."],
['Deuteronomy','Deu','04','037','37',"And because he loved thy fathers, therefore he chose their seed after them, and brought thee out in his sight with his mighty power out of Egypt;"],
['Deuteronomy','Deu','04','038','38',"To drive out nations from before thee greater and mightier than thou \emph{art}, to bring thee in, to give thee their land \emph{for} an inheritance, as \emph{it} \emph{is} this day."],
['Deuteronomy','Deu','04','039','39',"Know therefore this day, and consider \emph{it} in thine heart, that the LORD he \emph{is} God in heaven above, and upon the earth beneath: \emph{there} \emph{is} none else."],
['Deuteronomy','Deu','04','040','40',"Thou shalt keep therefore his statutes, and his commandments, which I command thee this day, that it may go well with thee, and with thy children after thee, and that thou mayest prolong \emph{thy} days upon the earth, which the LORD thy God giveth thee, for ever."],
['Deuteronomy','Deu','04','041','41',"Then Moses severed three cities on this side Jordan toward the sunrising;"],
['Deuteronomy','Deu','04','042','42',"That the slayer might flee thither, which should kill his neighbour unawares, and hated him not in times past; and that fleeing unto one of these cities he might live:"],
['Deuteronomy','Deu','04','043','43',"\emph{Namely}, Bezer in the wilderness, in the plain country, of the Reubenites; and Ramoth in Gilead, of the Gadites; and Golan in Bashan, of the Manassites."],
['Deuteronomy','Deu','04','044','44',"And this \emph{is} the law which Moses set before the children of Israel:"],
['Deuteronomy','Deu','04','045','45',"These \emph{are} the testimonies, and the statutes, and the judgments, which Moses spake unto the children of Israel, after they came forth out of Egypt,"],
['Deuteronomy','Deu','04','046','46',"On this side Jordan, in the valley over against Beth-peor, in the land of Sihon king of the Amorites, who dwelt at Heshbon, whom Moses and the children of Israel smote, after they were come forth out of Egypt:"],
['Deuteronomy','Deu','04','047','47',"And they possessed his land, and the land of Og king of Bashan, two kings of the Amorites, which \emph{were} on this side Jordan toward the sunrising;"],
['Deuteronomy','Deu','04','048','48',"From Aroer, which \emph{is} by the bank of the river Arnon, even unto mount Sion, which \emph{is} Hermon,"],
['Deuteronomy','Deu','04','049','49',"And all the plain on this side Jordan eastward, even unto the sea of the plain, under the springs of Pisgah."]
]

Deuteronomy05 = [
    ['Deuteronomy','Deu','05','001','1',"And Moses called all Israel, and said unto them, Hear, O Israel, the statutes and judgments which I speak in your ears this day, that ye may learn them, and keep, and do them."],
['Deuteronomy','Deu','05','002','2',"The LORD our God made a covenant with us in Horeb."],
['Deuteronomy','Deu','05','003','3',"The LORD made not this covenant with our fathers, but with us, \emph{even} us, who \emph{are} all of us here alive this day."],
['Deuteronomy','Deu','05','004','4',"The LORD talked with you face to face in the mount out of the midst of the fire,"],
['Deuteronomy','Deu','05','005','5',"(I stood between the LORD and you at that time, to shew you the word of the LORD: for ye were afraid by reason of the fire, and went not up into the mount;) saying,"],
['Deuteronomy','Deu','05','006','6',"I \emph{am} the LORD thy God, which brought thee out of the land of Egypt, from the house of bondage."],
['Deuteronomy','Deu','05','007','7',"Thou shalt have none other gods before me."],
['Deuteronomy','Deu','05','008','8',"Thou shalt not make thee \emph{any} graven image, \emph{or} any likeness \emph{of} \emph{any} \emph{thing} that \emph{is} in heaven above, or that \emph{is} in the earth beneath, or that \emph{is} in the waters beneath the earth:"],
['Deuteronomy','Deu','05','009','9',"Thou shalt not bow down thyself unto them, nor serve them: for I the LORD thy God \emph{am} a jealous God, visiting the iniquity of the fathers upon the children unto the third and fourth \emph{generation} of them that hate me,"],
['Deuteronomy','Deu','05','010','10',"And shewing mercy unto thousands of them that love me and keep my commandments."],
['Deuteronomy','Deu','05','011','11',"Thou shalt not take the name of the LORD thy God in vain: for the LORD will not hold \emph{him} guiltless that taketh his name in vain."],
['Deuteronomy','Deu','05','012','12',"Keep the sabbath day to sanctify it, as the LORD thy God hath commanded thee."],
['Deuteronomy','Deu','05','013','13',"Six days thou shalt labour, and do all thy work:"],
['Deuteronomy','Deu','05','014','14',"But the seventh day \emph{is} the sabbath of the LORD thy God: \emph{in} \emph{it} thou shalt not do any work, thou, nor thy son, nor thy daughter, nor thy manservant, nor thy maidservant, nor thine ox, nor thine ass, nor any of thy cattle, nor thy stranger that \emph{is} within thy gates; that thy manservant and thy maidservant may rest as well as thou."],
['Deuteronomy','Deu','05','015','15',"And remember that thou wast a servant in the land of Egypt, and \emph{that} the LORD thy God brought thee out thence through a mighty hand and by a stretched out arm: therefore the LORD thy God commanded thee to keep the sabbath day."],
['Deuteronomy','Deu','05','016','16',"Honour thy father and thy mother, as the LORD thy God hath commanded thee; that thy days may be prolonged, and that it may go well with thee, in the land which the LORD thy God giveth thee."],
['Deuteronomy','Deu','05','017','17',"Thou shalt not kill."],
['Deuteronomy','Deu','05','018','18',"Neither shalt thou commit adultery."],
['Deuteronomy','Deu','05','019','19',"Neither shalt thou steal."],
['Deuteronomy','Deu','05','020','20',"Neither shalt thou bear false witness against thy neighbour."],
['Deuteronomy','Deu','05','021','21',"Neither shalt thou desire thy neighbour's wife, neither shalt thou covet thy neighbour's house, his field, or his manservant, or his maidservant, his ox, or his ass, or any \emph{thing} that \emph{is} thy neighbour's."],
['Deuteronomy','Deu','05','022','22',"These words the LORD spake unto all your assembly in the mount out of the midst of the fire, of the cloud, and of the thick darkness, with a great voice: and he added no more. And he wrote them in two tables of stone, and delivered them unto me."],
['Deuteronomy','Deu','05','023','23',"And it came to pass, when ye heard the voice out of the midst of the darkness, (for the mountain did burn with fire,) that ye came near unto me, \emph{even} all the heads of your tribes, and your elders;"],
['Deuteronomy','Deu','05','024','24',"And ye said, Behold, the LORD our God hath shewed us his glory and his greatness, and we have heard his voice out of the midst of the fire: we have seen this day that God doth talk with man, and he liveth."],
['Deuteronomy','Deu','05','025','25',"Now therefore why should we die? for this great fire will consume us: if we hear the voice of the LORD our God any more, then we shall die."],
['Deuteronomy','Deu','05','026','26',"For who \emph{is} \emph{there} \emph{of} all flesh, that hath heard the voice of the living God speaking out of the midst of the fire, as we \emph{have}, and lived?"],
['Deuteronomy','Deu','05','027','27',"Go thou near, and hear all that the LORD our God shall say: and speak thou unto us all that the LORD our God shall speak unto thee; and we will hear \emph{it}, and do \emph{it}."],
['Deuteronomy','Deu','05','028','28',"And the LORD heard the voice of your words, when ye spake unto me; and the LORD said unto me, I have heard the voice of the words of this people, which they have spoken unto thee: they have well said all that they have spoken."],
['Deuteronomy','Deu','05','029','29',"O that there were such an heart in them, that they would fear me, and keep all my commandments always, that it might be well with them, and with their children for ever!"],
['Deuteronomy','Deu','05','030','30',"Go say to them, Get you into your tents again."],
['Deuteronomy','Deu','05','031','31',"But as for thee, stand thou here by me, and I will speak unto thee all the commandments, and the statutes, and the judgments, which thou shalt teach them, that they may do \emph{them} in the land which I give them to possess it."],
['Deuteronomy','Deu','05','032','32',"Ye shall observe to do therefore as the LORD your God hath commanded you: ye shall not turn aside to the right hand or to the left."],
['Deuteronomy','Deu','05','033','33',"Ye shall walk in all the ways which the LORD your God hath commanded you, that ye may live, and \emph{that} \emph{it} \emph{may} \emph{be} well with you, and \emph{that} ye may prolong \emph{your} days in the land which ye shall possess."]
]

Deuteronomy06 = [
    ['Deuteronomy','Deu','06','001','1',"Now these \emph{are} the commandments, the statutes, and the judgments, which the LORD your God commanded to teach you, that ye might do \emph{them} in the land whither ye go to possess it:"],
['Deuteronomy','Deu','06','002','2',"That thou mightest fear the LORD thy God, to keep all his statutes and his commandments, which I command thee, thou, and thy son, and thy son's son, all the days of thy life; and that thy days may be prolonged."],
['Deuteronomy','Deu','06','003','3',"Hear therefore, O Israel, and observe to do \emph{it}; that it may be well with thee, and that ye may increase mightily, as the LORD God of thy fathers hath promised thee, in the land that floweth with milk and honey."],
['Deuteronomy','Deu','06','004','4',"Hear, O Israel: The LORD our God \emph{is} one LORD:"],
['Deuteronomy','Deu','06','005','5',"And thou shalt love the LORD thy God with all thine heart, and with all thy soul, and with all thy might."],
['Deuteronomy','Deu','06','006','6',"And these words, which I command thee this day, shall be in thine heart:"],
['Deuteronomy','Deu','06','007','7',"And thou shalt teach them diligently unto thy children, and shalt talk of them when thou sittest in thine house, and when thou walkest by the way, and when thou liest down, and when thou risest up."],
['Deuteronomy','Deu','06','008','8',"And thou shalt bind them for a sign upon thine hand, and they shall be as frontlets between thine eyes."],
['Deuteronomy','Deu','06','009','9',"And thou shalt write them upon the posts of thy house, and on thy gates."],
['Deuteronomy','Deu','06','010','10',"And it shall be, when the LORD thy God shall have brought thee into the land which he sware unto thy fathers, to Abraham, to Isaac, and to Jacob, to give thee great and goodly cities, which thou buildedst not,"],
['Deuteronomy','Deu','06','011','11',"And houses full of all good \emph{things}, which thou filledst not, and wells digged, which thou diggedst not, vineyards and olive trees, which thou plantedst not; when thou shalt have eaten and be full;"],
['Deuteronomy','Deu','06','012','12',"\emph{Then} beware lest thou forget the LORD, which brought thee forth out of the land of Egypt, from the house of bondage."],
['Deuteronomy','Deu','06','013','13',"Thou shalt fear the LORD thy God, and serve him, and shalt swear by his name."],
['Deuteronomy','Deu','06','014','14',"Ye shall not go after other gods, of the gods of the people which \emph{are} round about you;"],
['Deuteronomy','Deu','06','015','15',"(For the LORD thy God \emph{is} a jealous God among you) lest the anger of the LORD thy God be kindled against thee, and destroy thee from off the face of the earth."],
['Deuteronomy','Deu','06','016','16',"Ye shall not tempt the LORD your God, as ye tempted \emph{him} in Massah."],
['Deuteronomy','Deu','06','017','17',"Ye shall diligently keep the commandments of the LORD your God, and his testimonies, and his statutes, which he hath commanded thee."],
['Deuteronomy','Deu','06','018','18',"And thou shalt do \emph{that} \emph{which} \emph{is} right and good in the sight of the LORD: that it may be well with thee, and that thou mayest go in and possess the good land which the LORD sware unto thy fathers,"],
['Deuteronomy','Deu','06','019','19',"To cast out all thine enemies from before thee, as the LORD hath spoken."],
['Deuteronomy','Deu','06','020','20',"\emph{And} when thy son asketh thee in time to come, saying, What \emph{mean} the testimonies, and the statutes, and the judgments, which the LORD our God hath commanded you?"],
['Deuteronomy','Deu','06','021','21',"Then thou shalt say unto thy son, We were Pharaoh's bondmen in Egypt; and the LORD brought us out of Egypt with a mighty hand:"],
['Deuteronomy','Deu','06','022','22',"And the LORD shewed signs and wonders, great and sore, upon Egypt, upon Pharaoh, and upon all his household, before our eyes:"],
['Deuteronomy','Deu','06','023','23',"And he brought us out from thence, that he might bring us in, to give us the land which he sware unto our fathers."],
['Deuteronomy','Deu','06','024','24',"And the LORD commanded us to do all these statutes, to fear the LORD our God, for our good always, that he might preserve us alive, as \emph{it} \emph{is} at this day."],
['Deuteronomy','Deu','06','025','25',"And it shall be our righteousness, if we observe to do all these commandments before the LORD our God, as he hath commanded us."]
]


Deuteronomy07 = [
    ['Deuteronomy','Deu','07','001','1',"When the LORD thy God shall bring thee into the land whither thou goest to possess it, and hath cast out many nations before thee, the Hittites, and the Girgashites, and the Amorites, and the Canaanites, and the Perizzites, and the Hivites, and the Jebusites, seven nations greater and mightier than thou;"],
    ['Deuteronomy','Deu','07','002','2',"And when the LORD thy God shall deliver them before thee; thou shalt smite them, \emph{and} utterly destroy them; thou shalt make no covenant with them, nor shew mercy unto them:"],
    ['Deuteronomy','Deu','07','003','3',"Neither shalt thou make marriages with them; thy daughter thou shalt not give unto his son, nor his daughter shalt thou take unto thy son."],
    ['Deuteronomy','Deu','07','004','4',"For they will turn away thy son from following me, that they may serve other gods: so will the anger of the LORD be kindled against you, and destroy thee suddenly."],
    ['Deuteronomy','Deu','07','005','5',"But thus shall ye deal with them; ye shall destroy their altars, and break down their images, and cut down their groves, and burn their graven images with fire."],
    ['Deuteronomy','Deu','07','006','6',"For thou \emph{art} an holy people unto the LORD thy God: the LORD thy God hath chosen thee to be a special people unto himself, above all people that \emph{are} upon the face of the earth."],
    ['Deuteronomy','Deu','07','007','7',"The LORD did not set his love upon you, nor choose you, because ye were more in number than any people; for ye \emph{were} the fewest of all people:"],
    ['Deuteronomy','Deu','07','008','8',"But because the LORD loved you, and because he would keep the oath which he had sworn unto your fathers, hath the LORD brought you out with a mighty hand, and redeemed you out of the house of bondmen, from the hand of Pharaoh king of Egypt."],
    ['Deuteronomy','Deu','07','009','9',"Know therefore that the LORD thy God, he \emph{is} God, the faithful God, which keepeth covenant and mercy with them that love him and keep his commandments to a thousand generations;"],
    ['Deuteronomy','Deu','07','010','10',"And repayeth them that hate him to their face, to destroy them: he will not be slack to him that hateth him, he will repay him to his face."],
    ['Deuteronomy','Deu','07','011','11',"Thou shalt therefore keep the commandments, and the statutes, and the judgments, which I command thee this day, to do them."],
    ['Deuteronomy','Deu','07','012','12',"Wherefore it shall come to pass, if ye hearken to these judgments, and keep, and do them, that the LORD thy God shall keep unto thee the covenant and the mercy which he sware unto thy fathers:"],
    ['Deuteronomy','Deu','07','013','13',"And he will love thee, and bless thee, and multiply thee: he will also bless the fruit of thy womb, and the fruit of thy land, thy corn, and thy wine, and thine oil, the increase of thy kine, and the flocks of thy sheep, in the land which he sware unto thy fathers to give thee."],
    ['Deuteronomy','Deu','07','014','14',"Thou shalt be blessed above all people: there shall not be male or female barren among you, or among your cattle."],
    ['Deuteronomy','Deu','07','015','15',"And the LORD will take away from thee all sickness, and will put none of the evil diseases of Egypt, which thou knowest, upon thee; but will lay them upon all \emph{them} that hate thee."],
    ['Deuteronomy','Deu','07','016','16',"And thou shalt consume all the people which the LORD thy God shall deliver thee; thine eye shall have no pity upon them: neither shalt thou serve their gods; for that \emph{will} \emph{be} a snare unto thee."],
    ['Deuteronomy','Deu','07','017','17',"If thou shalt say in thine heart, These nations \emph{are} more than I; how can I dispossess them?"],
    ['Deuteronomy','Deu','07','018','18',"Thou shalt not be afraid of them: \emph{but} shalt well remember what the LORD thy God did unto Pharaoh, and unto all Egypt;"],
    ['Deuteronomy','Deu','07','019','19',"The great temptations which thine eyes saw, and the signs, and the wonders, and the mighty hand, and the stretched out arm, whereby the LORD thy God brought thee out: so shall the LORD thy God do unto all the people of whom thou art afraid."],
    ['Deuteronomy','Deu','07','020','20',"Moreover the LORD thy God will send the hornet among them, until they that are left, and hide themselves from thee, be destroyed."],
    ['Deuteronomy','Deu','07','021','21',"Thou shalt not be affrighted at them: for the LORD thy God \emph{is} among you, a mighty God and terrible."],
    ['Deuteronomy','Deu','07','022','22',"And the LORD thy God will put out those nations before thee by little and little: thou mayest not consume them at once, lest the beasts of the field increase upon thee."],
    ['Deuteronomy','Deu','07','023','23',"But the LORD thy God shall deliver them unto thee, and shall destroy them with a mighty destruction, until they be destroyed."],
    ['Deuteronomy','Deu','07','024','24',"And he shall deliver their kings into thine hand, and thou shalt destroy their name from under heaven: there shall no man be able to stand before thee, until thou have destroyed them."],
    ['Deuteronomy','Deu','07','025','25',"The graven images of their gods shall ye burn with fire: thou shalt not desire the silver or gold \emph{that} \emph{is} on them, nor take \emph{it} unto thee, lest thou be snared therein: for it \emph{is} an abomination to the LORD thy God."],
    ['Deuteronomy','Deu','07','026','26',"Neither shalt thou bring an abomination into thine house, lest thou be a cursed thing like it: \emph{but} thou shalt utterly detest it, and thou shalt utterly abhor it; for it \emph{is} a cursed thing."]
]

Deuteronomy08 = [
    ['Deuteronomy','Deu','08','001','1',"All the commandments which I command thee this day shall ye observe to do, that ye may live, and multiply, and go in and possess the land which the LORD sware unto your fathers."],
['Deuteronomy','Deu','08','002','2',"And thou shalt remember all the way which the LORD thy God led thee these forty years in the wilderness, to humble thee, \emph{and} to prove thee, to know what \emph{was} in thine heart, whether thou wouldest keep his commandments, or no."],
['Deuteronomy','Deu','08','003','3',"And he humbled thee, and suffered thee to hunger, and fed thee with manna, which thou knewest not, neither did thy fathers know; that he might make thee know that man doth not live by bread only, but by every \emph{word} that proceedeth out of the mouth of the LORD doth man live."],
['Deuteronomy','Deu','08','004','4',"Thy raiment waxed not old upon thee, neither did thy foot swell, these forty years."],
['Deuteronomy','Deu','08','005','5',"Thou shalt also consider in thine heart, that, as a man chasteneth his son, \emph{so} the LORD thy God chasteneth thee."],
['Deuteronomy','Deu','08','006','6',"Therefore thou shalt keep the commandments of the LORD thy God, to walk in his ways, and to fear him."],
['Deuteronomy','Deu','08','007','7',"For the LORD thy God bringeth thee into a good land, a land of brooks of water, of fountains and depths that spring out of valleys and hills;"],
['Deuteronomy','Deu','08','008','8',"A land of wheat, and barley, and vines, and fig trees, and pomegranates; a land of oil olive, and honey;"],
['Deuteronomy','Deu','08','009','9',"A land wherein thou shalt eat bread without scarceness, thou shalt not lack any \emph{thing} in it; a land whose stones \emph{are} iron, and out of whose hills thou mayest dig brass."],
['Deuteronomy','Deu','08','010','10',"When thou hast eaten and art full, then thou shalt bless the LORD thy God for the good land which he hath given thee."],
['Deuteronomy','Deu','08','011','11',"Beware that thou forget not the LORD thy God, in not keeping his commandments, and his judgments, and his statutes, which I command thee this day:"],
['Deuteronomy','Deu','08','012','12',"Lest \emph{when} thou hast eaten and art full, and hast built goodly houses, and dwelt \emph{therein};"],
['Deuteronomy','Deu','08','013','13',"And \emph{when} thy herds and thy flocks multiply, and thy silver and thy gold is multiplied, and all that thou hast is multiplied;"],
['Deuteronomy','Deu','08','014','14',"Then thine heart be lifted up, and thou forget the LORD thy God, which brought thee forth out of the land of Egypt, from the house of bondage;"],
['Deuteronomy','Deu','08','015','15',"Who led thee through that great and terrible wilderness, \emph{wherein} \emph{were} fiery serpents, and scorpions, and drought, where \emph{there} \emph{was} no water; who brought thee forth water out of the rock of flint;"],
['Deuteronomy','Deu','08','016','16',"Who fed thee in the wilderness with manna, which thy fathers knew not, that he might humble thee, and that he might prove thee, to do thee good at thy latter end;"],
['Deuteronomy','Deu','08','017','17',"And thou say in thine heart, My power and the might of \emph{mine} hand hath gotten me this wealth."],
['Deuteronomy','Deu','08','018','18',"But thou shalt remember the LORD thy God: for \emph{it} \emph{is} he that giveth thee power to get wealth, that he may establish his covenant which he sware unto thy fathers, as \emph{it} \emph{is} this day."],
['Deuteronomy','Deu','08','019','19',"And it shall be, if thou do at all forget the LORD thy God, and walk after other gods, and serve them, and worship them, I testify against you this day that ye shall surely perish."],
['Deuteronomy','Deu','08','020','20',"As the nations which the LORD destroyeth before your face, so shall ye perish; because ye would not be obedient unto the voice of the LORD your God."]
]

Deuteronomy09 = [
    ['Deuteronomy','Deu','09','001','1',"Hear, O Israel: Thou \emph{art} to pass over Jordan this day, to go in to possess nations greater and mightier than thyself, cities great and fenced up to heaven,"],
['Deuteronomy','Deu','09','002','2',"A people great and tall, the children of the Anakims, whom thou knowest, and \emph{of} \emph{whom} thou hast heard \emph{say}, Who can stand before the children of Anak!"],
['Deuteronomy','Deu','09','003','3',"Understand therefore this day, that the LORD thy God \emph{is} he which goeth over before thee; \emph{as} a consuming fire he shall destroy them, and he shall bring them down before thy face: so shalt thou drive them out, and destroy them quickly, as the LORD hath said unto thee."],
['Deuteronomy','Deu','09','004','4',"Speak not thou in thine heart, after that the LORD thy God hath cast them out from before thee, saying, For my righteousness the LORD hath brought me in to possess this land: but for the wickedness of these nations the LORD doth drive them out from before thee."],
['Deuteronomy','Deu','09','005','5',"Not for thy righteousness, or for the uprightness of thine heart, dost thou go to possess their land: but for the wickedness of these nations the LORD thy God doth drive them out from before thee, and that he may perform the word which the LORD sware unto thy fathers, Abraham, Isaac, and Jacob."],
['Deuteronomy','Deu','09','006','6',"Understand therefore, that the LORD thy God giveth thee not this good land to possess it for thy righteousness; for thou \emph{art} a stiffnecked people."],
['Deuteronomy','Deu','09','007','7',"Remember, \emph{and} forget not, how thou provokedst the LORD thy God to wrath in the wilderness: from the day that thou didst depart out of the land of Egypt, until ye came unto this place, ye have been rebellious against the LORD."],
['Deuteronomy','Deu','09','008','8',"Also in Horeb ye provoked the LORD to wrath, so that the LORD was angry with you to have destroyed you."],
['Deuteronomy','Deu','09','009','9',"When I was gone up into the mount to receive the tables of stone, \emph{even} the tables of the covenant which the LORD made with you, then I abode in the mount forty days and forty nights, I neither did eat bread nor drink water:"],
['Deuteronomy','Deu','09','010','10',"And the LORD delivered unto me two tables of stone written with the finger of God; and on them \emph{was} \emph{written} according to all the words, which the LORD spake with you in the mount out of the midst of the fire in the day of the assembly."],
['Deuteronomy','Deu','09','011','11',"And it came to pass at the end of forty days and forty nights, \emph{that} the LORD gave me the two tables of stone, \emph{even} the tables of the covenant."],
['Deuteronomy','Deu','09','012','12',"And the LORD said unto me, Arise, get thee down quickly from hence; for thy people which thou hast brought forth out of Egypt have corrupted \emph{themselves}; they are quickly turned aside out of the way which I commanded them; they have made them a molten image."],
['Deuteronomy','Deu','09','013','13',"Furthermore the LORD spake unto me, saying, I have seen this people, and, behold, it \emph{is} a stiffnecked people:"],
['Deuteronomy','Deu','09','014','14',"Let me alone, that I may destroy them, and blot out their name from under heaven: and I will make of thee a nation mightier and greater than they."],
['Deuteronomy','Deu','09','015','15',"So I turned and came down from the mount, and the mount burned with fire: and the two tables of the covenant \emph{were} in my two hands."],
['Deuteronomy','Deu','09','016','16',"And I looked, and, behold, ye had sinned against the LORD your God, \emph{and} had made you a molten calf: ye had turned aside quickly out of the way which the LORD had commanded you."],
['Deuteronomy','Deu','09','017','17',"And I took the two tables, and cast them out of my two hands, and brake them before your eyes."],
['Deuteronomy','Deu','09','018','18',"And I fell down before the LORD, as at the first, forty days and forty nights: I did neither eat bread, nor drink water, because of all your sins which ye sinned, in doing wickedly in the sight of the LORD, to provoke him to anger."],
['Deuteronomy','Deu','09','019','19',"For I was afraid of the anger and hot displeasure, wherewith the LORD was wroth against you to destroy you. But the LORD hearkened unto me at that time also."],
['Deuteronomy','Deu','09','020','20',"And the LORD was very angry with Aaron to have destroyed him: and I prayed for Aaron also the same time."],
['Deuteronomy','Deu','09','021','21',"And I took your sin, the calf which ye had made, and burnt it with fire, and stamped it, \emph{and} ground \emph{it} very small, \emph{even} until it was as small as dust: and I cast the dust thereof into the brook that descended out of the mount."],
['Deuteronomy','Deu','09','022','22',"And at Taberah, and at Massah, and at Kibroth-hattaavah, ye provoked the LORD to wrath."],
['Deuteronomy','Deu','09','023','23',"Likewise when the LORD sent you from Kadesh-barnea, saying, Go up and possess the land which I have given you; then ye rebelled against the commandment of the LORD your God, and ye believed him not, nor hearkened to his voice."],
['Deuteronomy','Deu','09','024','24',"Ye have been rebellious against the LORD from the day that I knew you."],
['Deuteronomy','Deu','09','025','25',"Thus I fell down before the LORD forty days and forty nights, as I fell down \emph{at} \emph{the} \emph{first}; because the LORD had said he would destroy you."],
['Deuteronomy','Deu','09','026','26',"I prayed therefore unto the LORD, and said, O Lord GOD, destroy not thy people and thine inheritance, which thou hast redeemed through thy greatness, which thou hast brought forth out of Egypt with a mighty hand."],
['Deuteronomy','Deu','09','027','27',"Remember thy servants, Abraham, Isaac, and Jacob; look not unto the stubbornness of this people, nor to their wickedness, nor to their sin:"],
['Deuteronomy','Deu','09','028','28',"Lest the land whence thou broughtest us out say, Because the LORD was not able to bring them into the land which he promised them, and because he hated them, he hath brought them out to slay them in the wilderness."],
['Deuteronomy','Deu','09','029','29',"Yet they \emph{are} thy people and thine inheritance, which thou broughtest out by thy mighty power and by thy stretched out arm."]
]

Deuteronomy10 = [
    ['Deuteronomy','Deu','10','001','1',"At that time the LORD said unto me, Hew thee two tables of stone like unto the first, and come up unto me into the mount, and make thee an ark of wood."],
['Deuteronomy','Deu','10','002','2',"And I will write on the tables the words that were in the first tables which thou brakest, and thou shalt put them in the ark."],
['Deuteronomy','Deu','10','003','3',"And I made an ark \emph{of} shittim wood, and hewed two tables of stone like unto the first, and went up into the mount, having the two tables in mine hand."],
['Deuteronomy','Deu','10','004','4',"And he wrote on the tables, according to the first writing, the ten commandments, which the LORD spake unto you in the mount out of the midst of the fire in the day of the assembly: and the LORD gave them unto me."],
['Deuteronomy','Deu','10','005','5',"And I turned myself and came down from the mount, and put the tables in the ark which I had made; and there they be, as the LORD commanded me."],
['Deuteronomy','Deu','10','006','6',"And the children of Israel took their journey from Beeroth of the children of Jaakan to Mosera: there Aaron died, and there he was buried; and Eleazar his son ministered in the priest's office in his stead."],
['Deuteronomy','Deu','10','007','7',"From thence they journeyed unto Gudgodah; and from Gudgodah to Jotbath, a land of rivers of waters."],
['Deuteronomy','Deu','10','008','8',"At that time the LORD separated the tribe of Levi, to bear the ark of the covenant of the LORD, to stand before the LORD to minister unto him, and to bless in his name, unto this day."],
['Deuteronomy','Deu','10','009','9',"Wherefore Levi hath no part nor inheritance with his brethren; the LORD \emph{is} his inheritance, according as the LORD thy God promised him."],
['Deuteronomy','Deu','10','010','10',"And I stayed in the mount, according to the first time, forty days and forty nights; and the LORD hearkened unto me at that time also, \emph{and} the LORD would not destroy thee."],
['Deuteronomy','Deu','10','011','11',"And the LORD said unto me, Arise, take \emph{thy} journey before the people, that they may go in and possess the land, which I sware unto their fathers to give unto them."],
['Deuteronomy','Deu','10','012','12',"And now, Israel, what doth the LORD thy God require of thee, but to fear the LORD thy God, to walk in all his ways, and to love him, and to serve the LORD thy God with all thy heart and with all thy soul,"],
['Deuteronomy','Deu','10','013','13',"To keep the commandments of the LORD, and his statutes, which I command thee this day for thy good?"],
['Deuteronomy','Deu','10','014','14',"Behold, the heaven and the heaven of heavens \emph{is} the LORD'S thy God, the earth \emph{also}, with all that therein \emph{is}."],
['Deuteronomy','Deu','10','015','15',"Only the LORD had a delight in thy fathers to love them, and he chose their seed after them, \emph{even} you above all people, as \emph{it} \emph{is} this day."],
['Deuteronomy','Deu','10','016','16',"Circumcise therefore the foreskin of your heart, and be no more stiffnecked."],
['Deuteronomy','Deu','10','017','17',"For the LORD your God \emph{is} God of gods, and Lord of lords, a great God, a mighty, and a terrible, which regardeth not persons, nor taketh reward:"],
['Deuteronomy','Deu','10','018','18',"He doth execute the judgment of the fatherless and widow, and loveth the stranger, in giving him food and raiment."],
['Deuteronomy','Deu','10','019','19',"Love ye therefore the stranger: for ye were strangers in the land of Egypt."],
['Deuteronomy','Deu','10','020','20',"Thou shalt fear the LORD thy God; him shalt thou serve, and to him shalt thou cleave, and swear by his name."],
['Deuteronomy','Deu','10','021','21',"He \emph{is} thy praise, and he \emph{is} thy God, that hath done for thee these great and terrible things, which thine eyes have seen."],
['Deuteronomy','Deu','10','022','22',"Thy fathers went down into Egypt with threescore and ten persons; and now the LORD thy God hath made thee as the stars of heaven for multitude."]
]

Deuteronomy11 = [
    ['Deuteronomy','Deu','11','001','1',"Therefore thou shalt love the LORD thy God, and keep his charge, and his statutes, and his judgments, and his commandments, alway."],
['Deuteronomy','Deu','11','002','2',"And know ye this day: for \emph{I} \emph{speak} not with your children which have not known, and which have not seen the chastisement of the LORD your God, his greatness, his mighty hand, and his stretched out arm,"],
['Deuteronomy','Deu','11','003','3',"And his miracles, and his acts, which he did in the midst of Egypt unto Pharaoh the king of Egypt, and unto all his land;"],
['Deuteronomy','Deu','11','004','4',"And what he did unto the army of Egypt, unto their horses, and to their chariots; how he made the water of the Red sea to overflow them as they pursued after you, and \emph{how} the LORD hath destroyed them unto this day;"],
['Deuteronomy','Deu','11','005','5',"And what he did unto you in the wilderness, until ye came into this place;"],
['Deuteronomy','Deu','11','006','6',"And what he did unto Dathan and Abiram, the sons of Eliab, the son of Reuben: how the earth opened her mouth, and swallowed them up, and their households, and their tents, and all the substance that \emph{was} in their possession, in the midst of all Israel:"],
['Deuteronomy','Deu','11','007','7',"But your eyes have seen all the great acts of the LORD which he did."],
['Deuteronomy','Deu','11','008','8',"Therefore shall ye keep all the commandments which I command you this day, that ye may be strong, and go in and possess the land, whither ye go to possess it;"],
['Deuteronomy','Deu','11','009','9',"And that ye may prolong \emph{your} days in the land, which the LORD sware unto your fathers to give unto them and to their seed, a land that floweth with milk and honey."],
['Deuteronomy','Deu','11','010','10',"For the land, whither thou goest in to possess it, \emph{is} not as the land of Egypt, from whence ye came out, where thou sowedst thy seed, and wateredst \emph{it} with thy foot, as a garden of herbs:"],
['Deuteronomy','Deu','11','011','11',"But the land, whither ye go to possess it, \emph{is} a land of hills and valleys, \emph{and} drinketh water of the rain of heaven:"],
['Deuteronomy','Deu','11','012','12',"A land which the LORD thy God careth for: the eyes of the LORD thy God \emph{are} always upon it, from the beginning of the year even unto the end of the year."],
['Deuteronomy','Deu','11','013','13',"And it shall come to pass, if ye shall hearken diligently unto my commandments which I command you this day, to love the LORD your God, and to serve him with all your heart and with all your soul,"],
['Deuteronomy','Deu','11','014','14',"That I will give \emph{you} the rain of your land in his due season, the first rain and the latter rain, that thou mayest gather in thy corn, and thy wine, and thine oil."],
['Deuteronomy','Deu','11','015','15',"And I will send grass in thy fields for thy cattle, that thou mayest eat and be full."],
['Deuteronomy','Deu','11','016','16',"Take heed to yourselves, that your heart be not deceived, and ye turn aside, and serve other gods, and worship them;"],
['Deuteronomy','Deu','11','017','17',"And \emph{then} the LORD'S wrath be kindled against you, and he shut up the heaven, that there be no rain, and that the land yield not her fruit; and \emph{lest} ye perish quickly from off the good land which the LORD giveth you."],
['Deuteronomy','Deu','11','018','18',"Therefore shall ye lay up these my words in your heart and in your soul, and bind them for a sign upon your hand, that they may be as frontlets between your eyes."],
['Deuteronomy','Deu','11','019','19',"And ye shall teach them your children, speaking of them when thou sittest in thine house, and when thou walkest by the way, when thou liest down, and when thou risest up."],
['Deuteronomy','Deu','11','020','20',"And thou shalt write them upon the door posts of thine house, and upon thy gates:"],
['Deuteronomy','Deu','11','021','21',"That your days may be multiplied, and the days of your children, in the land which the LORD sware unto your fathers to give them, as the days of heaven upon the earth."],
['Deuteronomy','Deu','11','022','22',"For if ye shall diligently keep all these commandments which I command you, to do them, to love the LORD your God, to walk in all his ways, and to cleave unto him;"],
['Deuteronomy','Deu','11','023','23',"Then will the LORD drive out all these nations from before you, and ye shall possess greater nations and mightier than yourselves."],
['Deuteronomy','Deu','11','024','24',"Every place whereon the soles of your feet shall tread shall be your's: from the wilderness and Lebanon, from the river, the river Euphrates, even unto the uttermost sea shall your coast be."],
['Deuteronomy','Deu','11','025','25',"There shall no man be able to stand before you: \emph{for} the LORD your God shall lay the fear of you and the dread of you upon all the land that ye shall tread upon, as he hath said unto you."],
['Deuteronomy','Deu','11','026','26',"Behold, I set before you this day a blessing and a curse;"],
['Deuteronomy','Deu','11','027','27',"A blessing, if ye obey the commandments of the LORD your God, which I command you this day:"],
['Deuteronomy','Deu','11','028','28',"And a curse, if ye will not obey the commandments of the LORD your God, but turn aside out of the way which I command you this day, to go after other gods, which ye have not known."],
['Deuteronomy','Deu','11','029','29',"And it shall come to pass, when the LORD thy God hath brought thee in unto the land whither thou goest to possess it, that thou shalt put the blessing upon mount Gerizim, and the curse upon mount Ebal."],
['Deuteronomy','Deu','11','030','30',"\emph{Are} they not on the other side Jordan, by the way where the sun goeth down, in the land of the Canaanites, which dwell in the champaign over against Gilgal, beside the plains of Moreh?"],
['Deuteronomy','Deu','11','031','31',"For ye shall pass over Jordan to go in to possess the land which the LORD your God giveth you, and ye shall possess it, and dwell therein."],
['Deuteronomy','Deu','11','032','32',"And ye shall observe to do all the statutes and judgments which I set before you this day."]
]

Deuteronomy12 = [
    ['Deuteronomy','Deu','12','001','1',"These \emph{are} the statutes and judgments, which ye shall observe to do in the land, which the LORD God of thy fathers giveth thee to possess it, all the days that ye live upon the earth."],
['Deuteronomy','Deu','12','002','2',"Ye shall utterly destroy all the places, wherein the nations which ye shall possess served their gods, upon the high mountains, and upon the hills, and under every green tree:"],
['Deuteronomy','Deu','12','003','3',"And ye shall overthrow their altars, and break their pillars, and burn their groves with fire; and ye shall hew down the graven images of their gods, and destroy the names of them out of that place."],
['Deuteronomy','Deu','12','004','4',"Ye shall not do so unto the LORD your God."],
['Deuteronomy','Deu','12','005','5',"But unto the place which the LORD your God shall choose out of all your tribes to put his name there, \emph{even} unto his habitation shall ye seek, and thither thou shalt come:"],
['Deuteronomy','Deu','12','006','6',"And thither ye shall bring your burnt offerings, and your sacrifices, and your tithes, and heave offerings of your hand, and your vows, and your freewill offerings, and the firstlings of your herds and of your flocks:"],
['Deuteronomy','Deu','12','007','7',"And there ye shall eat before the LORD your God, and ye shall rejoice in all that ye put your hand unto, ye and your households, wherein the LORD thy God hath blessed thee."],
['Deuteronomy','Deu','12','008','8',"Ye shall not do after all \emph{the} \emph{things} that we do here this day, every man whatsoever \emph{is} right in his own eyes."],
['Deuteronomy','Deu','12','009','9',"For ye are not as yet come to the rest and to the inheritance, which the LORD your God giveth you."],
['Deuteronomy','Deu','12','010','10',"But \emph{when} ye go over Jordan, and dwell in the land which the LORD your God giveth you to inherit, and \emph{when} he giveth you rest from all your enemies round about, so that ye dwell in safety;"],
['Deuteronomy','Deu','12','011','11',"Then there shall be a place which the LORD your God shall choose to cause his name to dwell there; thither shall ye bring all that I command you; your burnt offerings, and your sacrifices, your tithes, and the heave offering of your hand, and all your choice vows which ye vow unto the LORD:"],
['Deuteronomy','Deu','12','012','12',"And ye shall rejoice before the LORD your God, ye, and your sons, and your daughters, and your menservants, and your maidservants, and the Levite that \emph{is} within your gates; forasmuch as he hath no part nor inheritance with you."],
['Deuteronomy','Deu','12','013','13',"Take heed to thyself that thou offer not thy burnt offerings in every place that thou seest:"],
['Deuteronomy','Deu','12','014','14',"But in the place which the LORD shall choose in one of thy tribes, there thou shalt offer thy burnt offerings, and there thou shalt do all that I command thee."],
['Deuteronomy','Deu','12','015','15',"Notwithstanding thou mayest kill and eat flesh in all thy gates, whatsoever thy soul lusteth after, according to the blessing of the LORD thy God which he hath given thee: the unclean and the clean may eat thereof, as of the roebuck, and as of the hart."],
['Deuteronomy','Deu','12','016','16',"Only ye shall not eat the blood; ye shall pour it upon the earth as water."],
['Deuteronomy','Deu','12','017','17',"Thou mayest not eat within thy gates the tithe of thy corn, or of thy wine, or of thy oil, or the firstlings of thy herds or of thy flock, nor any of thy vows which thou vowest, nor thy freewill offerings, or heave offering of thine hand:"],
['Deuteronomy','Deu','12','018','18',"But thou must eat them before the LORD thy God in the place which the LORD thy God shall choose, thou, and thy son, and thy daughter, and thy manservant, and thy maidservant, and the Levite that \emph{is} within thy gates: and thou shalt rejoice before the LORD thy God in all that thou puttest thine hands unto."],
['Deuteronomy','Deu','12','019','19',"Take heed to thyself that thou forsake not the Levite as long as thou livest upon the earth."],
['Deuteronomy','Deu','12','020','20',"When the LORD thy God shall enlarge thy border, as he hath promised thee, and thou shalt say, I will eat flesh, because thy soul longeth to eat flesh; thou mayest eat flesh, whatsoever thy soul lusteth after."],
['Deuteronomy','Deu','12','021','21',"If the place which the LORD thy God hath chosen to put his name there be too far from thee, then thou shalt kill of thy herd and of thy flock, which the LORD hath given thee, as I have commanded thee, and thou shalt eat in thy gates whatsoever thy soul lusteth after."],
['Deuteronomy','Deu','12','022','22',"Even as the roebuck and the hart is eaten, so thou shalt eat them: the unclean and the clean shall eat \emph{of} them alike."],
['Deuteronomy','Deu','12','023','23',"Only be sure that thou eat not the blood: for the blood \emph{is} the life; and thou mayest not eat the life with the flesh."],
['Deuteronomy','Deu','12','024','24',"Thou shalt not eat it; thou shalt pour it upon the earth as water."],
['Deuteronomy','Deu','12','025','25',"Thou shalt not eat it; that it may go well with thee, and with thy children after thee, when thou shalt do \emph{that} \emph{which} \emph{is} right in the sight of the LORD."],
['Deuteronomy','Deu','12','026','26',"Only thy holy things which thou hast, and thy vows, thou shalt take, and go unto the place which the LORD shall choose:"],
['Deuteronomy','Deu','12','027','27',"And thou shalt offer thy burnt offerings, the flesh and the blood, upon the altar of the LORD thy God: and the blood of thy sacrifices shall be poured out upon the altar of the LORD thy God, and thou shalt eat the flesh."],
['Deuteronomy','Deu','12','028','28',"Observe and hear all these words which I command thee, that it may go well with thee, and with thy children after thee for ever, when thou doest \emph{that} \emph{which} \emph{is} good and right in the sight of the LORD thy God."],
['Deuteronomy','Deu','12','029','29',"When the LORD thy God shall cut off the nations from before thee, whither thou goest to possess them, and thou succeedest them, and dwellest in their land;"],
['Deuteronomy','Deu','12','030','30',"Take heed to thyself that thou be not snared by following them, after that they be destroyed from before thee; and that thou enquire not after their gods, saying, How did these nations serve their gods? even so will I do likewise."],
['Deuteronomy','Deu','12','031','31',"Thou shalt not do so unto the LORD thy God: for every abomination to the LORD, which he hateth, have they done unto their gods; for even their sons and their daughters they have burnt in the fire to their gods."],
['Deuteronomy','Deu','12','032','32',"What thing soever I command you, observe to do it: thou shalt not add thereto, nor diminish from it."]
]

Deuteronomy13 = [
    ['Deuteronomy','Deu','13','001','1',"If there arise among you a prophet, or a dreamer of dreams, and giveth thee a sign or a wonder,"],
['Deuteronomy','Deu','13','002','2',"And the sign or the wonder come to pass, whereof he spake unto thee, saying, Let us go after other gods, which thou hast not known, and let us serve them;"],
['Deuteronomy','Deu','13','003','3',"Thou shalt not hearken unto the words of that prophet, or that dreamer of dreams: for the LORD your God proveth you, to know whether ye love the LORD your God with all your heart and with all your soul."],
['Deuteronomy','Deu','13','004','4',"Ye shall walk after the LORD your God, and fear him, and keep his commandments, and obey his voice, and ye shall serve him, and cleave unto him."],
['Deuteronomy','Deu','13','005','5',"And that prophet, or that dreamer of dreams, shall be put to death; because he hath spoken to turn \emph{you} away from the LORD your God, which brought you out of the land of Egypt, and redeemed you out of the house of bondage, to thrust thee out of the way which the LORD thy God commanded thee to walk in. So shalt thou put the evil away from the midst of thee."],
['Deuteronomy','Deu','13','006','6',"If thy brother, the son of thy mother, or thy son, or thy daughter, or the wife of thy bosom, or thy friend, which \emph{is} as thine own soul, entice thee secretly, saying, Let us go and serve other gods, which thou hast not known, thou, nor thy fathers;"],
['Deuteronomy','Deu','13','007','7',"\emph{Namely}, of the gods of the people which \emph{are} round about you, nigh unto thee, or far off from thee, from the \emph{one} end of the earth even unto the \emph{other} end of the earth;"],
['Deuteronomy','Deu','13','008','8',"Thou shalt not consent unto him, nor hearken unto him; neither shall thine eye pity him, neither shalt thou spare, neither shalt thou conceal him:"],
['Deuteronomy','Deu','13','009','9',"But thou shalt surely kill him; thine hand shall be first upon him to put him to death, and afterwards the hand of all the people."],
['Deuteronomy','Deu','13','010','10',"And thou shalt stone him with stones, that he die; because he hath sought to thrust thee away from the LORD thy God, which brought thee out of the land of Egypt, from the house of bondage."],
['Deuteronomy','Deu','13','011','11',"And all Israel shall hear, and fear, and shall do no more any such wickedness as this is among you."],
['Deuteronomy','Deu','13','012','12',"If thou shalt hear \emph{say} in one of thy cities, which the LORD thy God hath given thee to dwell there, saying,"],
['Deuteronomy','Deu','13','013','13',"\emph{Certain} men, the children of Belial, are gone out from among you, and have withdrawn the inhabitants of their city, saying, Let us go and serve other gods, which ye have not known;"],
['Deuteronomy','Deu','13','014','14',"Then shalt thou enquire, and make search, and ask diligently; and, behold, \emph{if} \emph{it} \emph{be} truth, \emph{and} the thing certain, \emph{that} such abomination is wrought among you;"],
['Deuteronomy','Deu','13','015','15',"Thou shalt surely smite the inhabitants of that city with the edge of the sword, destroying it utterly, and all that \emph{is} therein, and the cattle thereof, with the edge of the sword."],
['Deuteronomy','Deu','13','016','16',"And thou shalt gather all the spoil of it into the midst of the street thereof, and shalt burn with fire the city, and all the spoil thereof every whit, for the LORD thy God: and it shall be an heap for ever; it shall not be built again."],
['Deuteronomy','Deu','13','017','17',"And there shall cleave nought of the cursed thing to thine hand: that the LORD may turn from the fierceness of his anger, and shew thee mercy, and have compassion upon thee, and multiply thee, as he hath sworn unto thy fathers;"],
['Deuteronomy','Deu','13','018','18',"When thou shalt hearken to the voice of the LORD thy God, to keep all his commandments which I command thee this day, to do \emph{that} \emph{which} \emph{is} right in the eyes of the LORD thy God."]
]

Deuteronomy14 = [
    ['Deuteronomy','Deu','14','001','1',"Ye \emph{are} the children of the LORD your God: ye shall not cut yourselves, nor make any baldness between your eyes for the dead."],
    ['Deuteronomy','Deu','14','002','2',"For thou \emph{art} an holy people unto the LORD thy God, and the LORD hath chosen thee to be a peculiar people unto himself, above all the nations that \emph{are} upon the earth."],
    ['Deuteronomy','Deu','14','003','3',"Thou shalt not eat any abominable thing."],
    ['Deuteronomy','Deu','14','004','4',"These \emph{are} the beasts which ye shall eat: the ox, the sheep, and the goat,"],
    ['Deuteronomy','Deu','14','005','5',"The hart, and the roebuck, and the fallow deer, and the wild goat, and the pygarg, and the wild ox, and the chamois."],
['Deuteronomy','Deu','14','006','6',"And every beast that parteth the hoof, and cleaveth the cleft into two claws, \emph{and} cheweth the cud among the beasts, that ye shall eat."],
['Deuteronomy','Deu','14','007','7',"Nevertheless these ye shall not eat of them that chew the cud, or of them that divide the cloven hoof; \emph{as} the camel, and the hare, and the coney: for they chew the cud, but divide not the hoof; \emph{therefore} they \emph{are} unclean unto you."],
['Deuteronomy','Deu','14','008','8',"And the swine, because it divideth the hoof, yet cheweth not the cud, it \emph{is} unclean unto you: ye shall not eat of their flesh, nor touch their dead carcase."],
['Deuteronomy','Deu','14','009','9',"These ye shall eat of all that \emph{are} in the waters: all that have fins and scales shall ye eat:"],
['Deuteronomy','Deu','14','010','10',"And whatsoever hath not fins and scales ye may not eat; it \emph{is} unclean unto you."],
['Deuteronomy','Deu','14','011','11',"\emph{Of} all clean birds ye shall eat."],
['Deuteronomy','Deu','14','012','12',"But these \emph{are} \emph{they} of which ye shall not eat: the eagle, and the ossifrage, and the ospray,"],
['Deuteronomy','Deu','14','013','13',"And the glede, and the kite, and the vulture after his kind,"],
['Deuteronomy','Deu','14','014','14',"And every raven after his kind,"],
['Deuteronomy','Deu','14','015','15',"And the owl, and the night hawk, and the cuckow, and the hawk after his kind,"],
['Deuteronomy','Deu','14','016','16',"The little owl, and the great owl, and the swan,"],
['Deuteronomy','Deu','14','017','17',"And the pelican, and the gier eagle, and the cormorant,"],
['Deuteronomy','Deu','14','018','18',"And the stork, and the heron after her kind, and the lapwing, and the bat."],
['Deuteronomy','Deu','14','019','19',"And every creeping thing that flieth \emph{is} unclean unto you: they shall not be eaten."],
['Deuteronomy','Deu','14','020','20',"\emph{But} \emph{of} all clean fowls ye may eat."],
['Deuteronomy','Deu','14','021','21',"Ye shall not eat \emph{of} any thing that dieth of itself: thou shalt give it unto the stranger that \emph{is} in thy gates, that he may eat it; or thou mayest sell it unto an alien: for thou \emph{art} an holy people unto the LORD thy God. Thou shalt not seethe a kid in his mother's milk."],
['Deuteronomy','Deu','14','022','22',"Thou shalt truly tithe all the increase of thy seed, that the field bringeth forth year by year."],
['Deuteronomy','Deu','14','023','23',"And thou shalt eat before the LORD thy God, in the place which he shall choose to place his name there, the tithe of thy corn, of thy wine, and of thine oil, and the firstlings of thy herds and of thy flocks; that thou mayest learn to fear the LORD thy God always."],
['Deuteronomy','Deu','14','024','24',"And if the way be too long for thee, so that thou art not able to carry it; \emph{or} if the place be too far from thee, which the LORD thy God shall choose to set his name there, when the LORD thy God hath blessed thee:"],
['Deuteronomy','Deu','14','025','25',"Then shalt thou turn \emph{it} into money, and bind up the money in thine hand, and shalt go unto the place which the LORD thy God shall choose:"],
['Deuteronomy','Deu','14','026','26',"And thou shalt bestow that money for whatsoever thy soul lusteth after, for oxen, or for sheep, or for wine, or for strong drink, or for whatsoever thy soul desireth: and thou shalt eat there before the LORD thy God, and thou shalt rejoice, thou, and thine household,"],
['Deuteronomy','Deu','14','027','27',"And the Levite that \emph{is} within thy gates; thou shalt not forsake him; for he hath no part nor inheritance with thee."],
['Deuteronomy','Deu','14','028','28',"At the end of three years thou shalt bring forth all the tithe of thine increase the same year, and shalt lay \emph{it} up within thy gates:"],
['Deuteronomy','Deu','14','029','29',"And the Levite, (because he hath no part nor inheritance with thee,) and the stranger, and the fatherless, and the widow, which \emph{are} within thy gates, shall come, and shall eat and be satisfied; that the LORD thy God may bless thee in all the work of thine hand which thou doest."]
]

Deuteronomy15 = [
['Deuteronomy','Deu','15','001','1',"At the end of \emph{every} seven years thou shalt make a release."],
['Deuteronomy','Deu','15','002','2',"And this \emph{is} the manner of the release: Every creditor that lendeth \emph{ought} unto his neighbour shall release \emph{it}; he shall not exact \emph{it} of his neighbour, or of his brother; because it is called the LORD'S release."],
['Deuteronomy','Deu','15','003','3',"Of a foreigner thou mayest exact \emph{it} \emph{again}: but \emph{that} which is thine with thy brother thine hand shall release;"],
['Deuteronomy','Deu','15','004','4',"Save when there shall be no poor among you; for the LORD shall greatly bless thee in the land which the LORD thy God giveth thee \emph{for} an inheritance to possess it:"],
['Deuteronomy','Deu','15','005','5',"Only if thou carefully hearken unto the voice of the LORD thy God, to observe to do all these commandments which I command thee this day."],
['Deuteronomy','Deu','15','006','6',"For the LORD thy God blesseth thee, as he promised thee: and thou shalt lend unto many nations, but thou shalt not borrow; and thou shalt reign over many nations, but they shall not reign over thee."],
['Deuteronomy','Deu','15','007','7',"If there be among you a poor man of one of thy brethren within any of thy gates in thy land which the LORD thy God giveth thee, thou shalt not harden thine heart, nor shut thine hand from thy poor brother:"],
['Deuteronomy','Deu','15','008','8',"But thou shalt open thine hand wide unto him, and shalt surely lend him sufficient for his need, \emph{in} \emph{that} which he wanteth."],
['Deuteronomy','Deu','15','009','9',"Beware that there be not a thought in thy wicked heart, saying, The seventh year, the year of release, is at hand; and thine eye be evil against thy poor brother, and thou givest him nought; and he cry unto the LORD against thee, and it be sin unto thee."],
['Deuteronomy','Deu','15','010','10',"Thou shalt surely give him, and thine heart shall not be grieved when thou givest unto him: because that for this thing the LORD thy God shall bless thee in all thy works, and in all that thou puttest thine hand unto."],
['Deuteronomy','Deu','15','011','11',"For the poor shall never cease out of the land: therefore I command thee, saying, Thou shalt open thine hand wide unto thy brother, to thy poor, and to thy needy, in thy land."],
['Deuteronomy','Deu','15','012','12',"\emph{And} if thy brother, an Hebrew man, or an Hebrew woman, be sold unto thee, and serve thee six years; then in the seventh year thou shalt let him go free from thee."],
['Deuteronomy','Deu','15','013','13',"And when thou sendest him out free from thee, thou shalt not let him go away empty:"],
['Deuteronomy','Deu','15','014','14',"Thou shalt furnish him liberally out of thy flock, and out of thy floor, and out of thy winepress: \emph{of} \emph{that} wherewith the LORD thy God hath blessed thee thou shalt give unto him."],
['Deuteronomy','Deu','15','015','15',"And thou shalt remember that thou wast a bondman in the land of Egypt, and the LORD thy God redeemed thee: therefore I command thee this thing to day."],
['Deuteronomy','Deu','15','016','16',"And it shall be, if he say unto thee, I will not go away from thee; because he loveth thee and thine house, because he is well with thee;"],
['Deuteronomy','Deu','15','017','17',"Then thou shalt take an aul, and thrust \emph{it} through his ear unto the door, and he shall be thy servant for ever. And also unto thy maidservant thou shalt do likewise."],
['Deuteronomy','Deu','15','018','18',"It shall not seem hard unto thee, when thou sendest him away free from thee; for he hath been worth a double hired servant \emph{to} \emph{thee}, in serving thee six years: and the LORD thy God shall bless thee in all that thou doest."],
['Deuteronomy','Deu','15','019','19',"All the firstling males that come of thy herd and of thy flock thou shalt sanctify unto the LORD thy God: thou shalt do no work with the firstling of thy bullock, nor shear the firstling of thy sheep."],
['Deuteronomy','Deu','15','020','20',"Thou shalt eat \emph{it} before the LORD thy God year by year in the place which the LORD shall choose, thou and thy household."],
['Deuteronomy','Deu','15','021','21',"And if there be \emph{any} blemish therein, \emph{as} \emph{if} \emph{it} \emph{be} lame, or blind, \emph{or} \emph{have} any ill blemish, thou shalt not sacrifice it unto the LORD thy God."],
['Deuteronomy','Deu','15','022','22',"Thou shalt eat it within thy gates: the unclean and the clean \emph{person} \emph{shall} \emph{eat} \emph{it} alike, as the roebuck, and as the hart."],
['Deuteronomy','Deu','15','023','23',"Only thou shalt not eat the blood thereof; thou shalt pour it upon the ground as water."]
]

Deuteronomy16 = [
    ['Deuteronomy','Deu','16','001','1',"Observe the month of Abib, and keep the passover unto the LORD thy God: for in the month of Abib the LORD thy God brought thee forth out of Egypt by night."],
['Deuteronomy','Deu','16','002','2',"Thou shalt therefore sacrifice the passover unto the LORD thy God, of the flock and the herd, in the place which the LORD shall choose to place his name there."],
['Deuteronomy','Deu','16','003','3',"Thou shalt eat no leavened bread with it; seven days shalt thou eat unleavened bread therewith, \emph{even} the bread of affliction; for thou camest forth out of the land of Egypt in haste: that thou mayest remember the day when thou camest forth out of the land of Egypt all the days of thy life."],
['Deuteronomy','Deu','16','004','4',"And there shall be no leavened bread seen with thee in all thy coast seven days; neither shall there \emph{any} \emph{thing} of the flesh, which thou sacrificedst the first day at even, remain all night until the morning."],
['Deuteronomy','Deu','16','005','5',"Thou mayest not sacrifice the passover within any of thy gates, which the LORD thy God giveth thee:"],
['Deuteronomy','Deu','16','006','6',"But at the place which the LORD thy God shall choose to place his name in, there thou shalt sacrifice the passover at even, at the going down of the sun, at the season that thou camest forth out of Egypt."],
['Deuteronomy','Deu','16','007','7',"And thou shalt roast and eat \emph{it} in the place which the LORD thy God shall choose: and thou shalt turn in the morning, and go unto thy tents."],
['Deuteronomy','Deu','16','008','8',"Six days thou shalt eat unleavened bread: and on the seventh day \emph{shall} \emph{be} a solemn assembly to the LORD thy God: thou shalt do no work \emph{therein}."],
['Deuteronomy','Deu','16','009','9',"Seven weeks shalt thou number unto thee: begin to number the seven weeks from \emph{such} \emph{time} \emph{as} thou beginnest \emph{to} \emph{put} the sickle to the corn."],
['Deuteronomy','Deu','16','010','10',"And thou shalt keep the feast of weeks unto the LORD thy God with a tribute of a freewill offering of thine hand, which thou shalt give \emph{unto} \emph{the} \emph{LORD} \emph{thy} \emph{God}, according as the LORD thy God hath blessed thee:"],
['Deuteronomy','Deu','16','011','11',"And thou shalt rejoice before the LORD thy God, thou, and thy son, and thy daughter, and thy manservant, and thy maidservant, and the Levite that \emph{is} within thy gates, and the stranger, and the fatherless, and the widow, that \emph{are} among you, in the place which the LORD thy God hath chosen to place his name there."],
['Deuteronomy','Deu','16','012','12',"And thou shalt remember that thou wast a bondman in Egypt: and thou shalt observe and do these statutes."],
['Deuteronomy','Deu','16','013','13',"Thou shalt observe the feast of tabernacles seven days, after that thou hast gathered in thy corn and thy wine:"],
['Deuteronomy','Deu','16','014','14',"And thou shalt rejoice in thy feast, thou, and thy son, and thy daughter, and thy manservant, and thy maidservant, and the Levite, the stranger, and the fatherless, and the widow, that \emph{are} within thy gates."],
['Deuteronomy','Deu','16','015','15',"Seven days shalt thou keep a solemn feast unto the LORD thy God in the place which the LORD shall choose: because the LORD thy God shall bless thee in all thine increase, and in all the works of thine hands, therefore thou shalt surely rejoice."],
['Deuteronomy','Deu','16','016','16',"Three times in a year shall all thy males appear before the LORD thy God in the place which he shall choose; in the feast of unleavened bread, and in the feast of weeks, and in the feast of tabernacles: and they shall not appear before the LORD empty:"],
['Deuteronomy','Deu','16','017','17',"Every man \emph{shall} \emph{give} as he is able, according to the blessing of the LORD thy God which he hath given thee."],
['Deuteronomy','Deu','16','018','18',"Judges and officers shalt thou make thee in all thy gates, which the LORD thy God giveth thee, throughout thy tribes: and they shall judge the people with just judgment."],
['Deuteronomy','Deu','16','019','19',"Thou shalt not wrest judgment; thou shalt not respect persons, neither take a gift: for a gift doth blind the eyes of the wise, and pervert the words of the righteous."],
['Deuteronomy','Deu','16','020','20',"That which is altogether just shalt thou follow, that thou mayest live, and inherit the land which the LORD thy God giveth thee."],
['Deuteronomy','Deu','16','021','21',"Thou shalt not plant thee a grove of any trees near unto the altar of the LORD thy God, which thou shalt make thee."],
['Deuteronomy','Deu','16','022','22',"Neither shalt thou set thee up \emph{any} image; which the LORD thy God hateth."]
]

Deuteronomy17 = [
    ['Deuteronomy','Deu','17','001','1',"Thou shalt not sacrifice unto the LORD thy God \emph{any} bullock, or sheep, wherein is blemish, \emph{or} any evilfavouredness: for that \emph{is} an abomination unto the LORD thy God."],
['Deuteronomy','Deu','17','002','2',"If there be found among you, within any of thy gates which the LORD thy God giveth thee, man or woman, that hath wrought wickedness in the sight of the LORD thy God, in transgressing his covenant,"],
['Deuteronomy','Deu','17','003','3',"And hath gone and served other gods, and worshipped them, either the sun, or moon, or any of the host of heaven, which I have not commanded;"],
['Deuteronomy','Deu','17','004','4',"And it be told thee, and thou hast heard \emph{of} \emph{it}, and enquired diligently, and, behold, \emph{it} \emph{be} true, \emph{and} the thing certain, \emph{that} such abomination is wrought in Israel:"],
['Deuteronomy','Deu','17','005','5',"Then shalt thou bring forth that man or that woman, which have committed that wicked thing, unto thy gates, \emph{even} that man or that woman, and shalt stone them with stones, till they die."],
['Deuteronomy','Deu','17','006','6',"At the mouth of two witnesses, or three witnesses, shall he that is worthy of death be put to death; \emph{but} at the mouth of one witness he shall not be put to death."],
['Deuteronomy','Deu','17','007','7',"The hands of the witnesses shall be first upon him to put him to death, and afterward the hands of all the people. So thou shalt put the evil away from among you."],
['Deuteronomy','Deu','17','008','8',"If there arise a matter too hard for thee in judgment, between blood and blood, between plea and plea, and between stroke and stroke, \emph{being} matters of controversy within thy gates: then shalt thou arise, and get thee up into the place which the LORD thy God shall choose;"],
['Deuteronomy','Deu','17','009','9',"And thou shalt come unto the priests the Levites, and unto the judge that shall be in those days, and enquire; and they shall shew thee the sentence of judgment:"],
['Deuteronomy','Deu','17','010','10',"And thou shalt do according to the sentence, which they of that place which the LORD shall choose shall shew thee; and thou shalt observe to do according to all that they inform thee:"],
['Deuteronomy','Deu','17','011','11',"According to the sentence of the law which they shall teach thee, and according to the judgment which they shall tell thee, thou shalt do: thou shalt not decline from the sentence which they shall shew thee, \emph{to} the right hand, nor \emph{to} the left."],
['Deuteronomy','Deu','17','012','12',"And the man that will do presumptuously, and will not hearken unto the priest that standeth to minister there before the LORD thy God, or unto the judge, even that man shall die: and thou shalt put away the evil from Israel."],
['Deuteronomy','Deu','17','013','13',"And all the people shall hear, and fear, and do no more presumptuously."],
['Deuteronomy','Deu','17','014','14',"When thou art come unto the land which the LORD thy God giveth thee, and shalt possess it, and shalt dwell therein, and shalt say, I will set a king over me, like as all the nations that \emph{are} about me;"],
['Deuteronomy','Deu','17','015','15',"Thou shalt in any wise set \emph{him} king over thee, whom the LORD thy God shall choose: \emph{one} from among thy brethren shalt thou set king over thee: thou mayest not set a stranger over thee, which \emph{is} not thy brother."],
['Deuteronomy','Deu','17','016','16',"But he shall not multiply horses to himself, nor cause the people to return to Egypt, to the end that he should multiply horses: forasmuch as the LORD hath said unto you, Ye shall henceforth return no more that way."],
['Deuteronomy','Deu','17','017','17',"Neither shall he multiply wives to himself, that his heart turn not away: neither shall he greatly multiply to himself silver and gold."],
['Deuteronomy','Deu','17','018','18',"And it shall be, when he sitteth upon the throne of his kingdom, that he shall write him a copy of this law in a book out of \emph{that} \emph{which} \emph{is} before the priests the Levites:"],
['Deuteronomy','Deu','17','019','19',"And it shall be with him, and he shall read therein all the days of his life: that he may learn to fear the LORD his God, to keep all the words of this law and these statutes, to do them:"],
['Deuteronomy','Deu','17','020','20',"That his heart be not lifted up above his brethren, and that he turn not aside from the commandment, \emph{to} the right hand, or \emph{to} the left: to the end that he may prolong \emph{his} days in his kingdom, he, and his children, in the midst of Israel."]
]

Deuteronomy18 = [
    ['Deuteronomy','Deu','18','001','1',"The priests the Levites, \emph{and} all the tribe of Levi, shall have no part nor inheritance with Israel: they shall eat the offerings of the LORD made by fire, and his inheritance."],
['Deuteronomy','Deu','18','002','2',"Therefore shall they have no inheritance among their brethren: the LORD \emph{is} their inheritance, as he hath said unto them."],
['Deuteronomy','Deu','18','003','3',"And this shall be the priest's due from the people, from them that offer a sacrifice, whether \emph{it} \emph{be} ox or sheep; and they shall give unto the priest the shoulder, and the two cheeks, and the maw."],
['Deuteronomy','Deu','18','004','4',"The firstfruit \emph{also} of thy corn, of thy wine, and of thine oil, and the first of the fleece of thy sheep, shalt thou give him."],
['Deuteronomy','Deu','18','005','5',"For the LORD thy God hath chosen him out of all thy tribes, to stand to minister in the name of the LORD, him and his sons for ever."],
['Deuteronomy','Deu','18','006','6',"And if a Levite come from any of thy gates out of all Israel, where he sojourned, and come with all the desire of his mind unto the place which the LORD shall choose;"],
['Deuteronomy','Deu','18','007','7',"Then he shall minister in the name of the LORD his God, as all his brethren the Levites \emph{do}, which stand there before the LORD."],
['Deuteronomy','Deu','18','008','8',"They shall have like portions to eat, beside that which cometh of the sale of his patrimony."],
['Deuteronomy','Deu','18','009','9',"When thou art come into the land which the LORD thy God giveth thee, thou shalt not learn to do after the abominations of those nations."],
['Deuteronomy','Deu','18','010','10',"There shall not be found among you \emph{any} \emph{one} that maketh his son or his daughter to pass through the fire, \emph{or} that useth divination, \emph{or} an observer of times, or an enchanter, or a witch,"],
['Deuteronomy','Deu','18','011','11',"Or a charmer, or a consulter with familiar spirits, or a wizard, or a necromancer."],
['Deuteronomy','Deu','18','012','12',"For all that do these things \emph{are} an abomination unto the LORD: and because of these abominations the LORD thy God doth drive them out from before thee."],
['Deuteronomy','Deu','18','013','13',"Thou shalt be perfect with the LORD thy God."],
['Deuteronomy','Deu','18','014','14',"For these nations, which thou shalt possess, hearkened unto observers of times, and unto diviners: but as for thee, the LORD thy God hath not suffered thee so \emph{to} \emph{do}."],
['Deuteronomy','Deu','18','015','15',"The LORD thy God will raise up unto thee a Prophet from the midst of thee, of thy brethren, like unto me; unto him ye shall hearken;"],
['Deuteronomy','Deu','18','016','16',"According to all that thou desiredst of the LORD thy God in Horeb in the day of the assembly, saying, Let me not hear again the voice of the LORD my God, neither let me see this great fire any more, that I die not."],
['Deuteronomy','Deu','18','017','17',"And the LORD said unto me, They have well \emph{spoken} \emph{that} which they have spoken."],
['Deuteronomy','Deu','18','018','18',"I will raise them up a Prophet from among their brethren, like unto thee, and will put my words in his mouth; and he shall speak unto them all that I shall command him."],
['Deuteronomy','Deu','18','019','19',"And it shall come to pass, \emph{that} whosoever will not hearken unto my words which he shall speak in my name, I will require \emph{it} of him."],
['Deuteronomy','Deu','18','020','20',"But the prophet, which shall presume to speak a word in my name, which I have not commanded him to speak, or that shall speak in the name of other gods, even that prophet shall die."],
['Deuteronomy','Deu','18','021','21',"And if thou say in thine heart, How shall we know the word which the LORD hath not spoken?"],
['Deuteronomy','Deu','18','022','22',"When a prophet speaketh in the name of the LORD, if the thing follow not, nor come to pass, that \emph{is} the thing which the LORD hath not spoken, \emph{but} the prophet hath spoken it presumptuously: thou shalt not be afraid of him."],
]

Deuteronomy19 = [
    ['Deuteronomy','Deu','19','001','1',"When the LORD thy God hath cut off the nations, whose land the LORD thy God giveth thee, and thou succeedest them, and dwellest in their cities, and in their houses;"],
['Deuteronomy','Deu','19','002','2',"Thou shalt separate three cities for thee in the midst of thy land, which the LORD thy God giveth thee to possess it."],
['Deuteronomy','Deu','19','003','3',"Thou shalt prepare thee a way, and divide the coasts of thy land, which the LORD thy God giveth thee to inherit, into three parts, that every slayer may flee thither."],
['Deuteronomy','Deu','19','004','4',"And this \emph{is} the case of the slayer, which shall flee thither, that he may live: Whoso killeth his neighbour ignorantly, whom he hated not in time past;"],
['Deuteronomy','Deu','19','005','5',"As when a man goeth into the wood with his neighbour to hew wood, and his hand fetcheth a stroke with the axe to cut down the tree, and the head slippeth from the helve, and lighteth upon his neighbour, that he die; he shall flee unto one of those cities, and live:"],
['Deuteronomy','Deu','19','006','6',"Lest the avenger of the blood pursue the slayer, while his heart is hot, and overtake him, because the way is long, and slay him; whereas he \emph{was} not worthy of death, inasmuch as he hated him not in time past."],
['Deuteronomy','Deu','19','007','7',"Wherefore I command thee, saying, Thou shalt separate three cities for thee."],
['Deuteronomy','Deu','19','008','8',"And if the LORD thy God enlarge thy coast, as he hath sworn unto thy fathers, and give thee all the land which he promised to give unto thy fathers;"],
['Deuteronomy','Deu','19','009','9',"If thou shalt keep all these commandments to do them, which I command thee this day, to love the LORD thy God, and to walk ever in his ways; then shalt thou add three cities more for thee, beside these three:"],
['Deuteronomy','Deu','19','010','10',"That innocent blood be not shed in thy land, which the LORD thy God giveth thee \emph{for} an inheritance, and \emph{so} blood be upon thee."],
['Deuteronomy','Deu','19','011','11',"But if any man hate his neighbour, and lie in wait for him, and rise up against him, and smite him mortally that he die, and fleeth into one of these cities:"],
['Deuteronomy','Deu','19','012','12',"Then the elders of his city shall send and fetch him thence, and deliver him into the hand of the avenger of blood, that he may die."],
['Deuteronomy','Deu','19','013','13',"Thine eye shall not pity him, but thou shalt put away \emph{the} \emph{guilt} \emph{of} innocent blood from Israel, that it may go well with thee."],
['Deuteronomy','Deu','19','014','14',"Thou shalt not remove thy neighbour's landmark, which they of old time have set in thine inheritance, which thou shalt inherit in the land that the LORD thy God giveth thee to possess it."],
['Deuteronomy','Deu','19','015','15',"One witness shall not rise up against a man for any iniquity, or for any sin, in any sin that he sinneth: at the mouth of two witnesses, or at the mouth of three witnesses, shall the matter be established."],
['Deuteronomy','Deu','19','016','16',"If a false witness rise up against any man to testify against him \emph{that} \emph{which} \emph{is} wrong;"],
['Deuteronomy','Deu','19','017','17',"Then both the men, between whom the controversy \emph{is}, shall stand before the LORD, before the priests and the judges, which shall be in those days;"],
['Deuteronomy','Deu','19','018','18',"And the judges shall make diligent inquisition: and, behold, \emph{if} the witness \emph{be} a false witness, \emph{and} hath testified falsely against his brother;"],
['Deuteronomy','Deu','19','019','19',"Then shall ye do unto him, as he had thought to have done unto his brother: so shalt thou put the evil away from among you."],
['Deuteronomy','Deu','19','020','20',"And those which remain shall hear, and fear, and shall henceforth commit no more any such evil among you."],
['Deuteronomy','Deu','19','021','21',"And thine eye shall not pity; \emph{but} life \emph{shall} \emph{go} for life, eye for eye, tooth for tooth, hand for hand, foot for foot."]
]

Deuteronomy20 = [
   ['Deuteronomy','Deu','20','001','1',"When thou goest out to battle against thine enemies, and seest horses, and chariots, \emph{and} a people more than thou, be not afraid of them: for the LORD thy God \emph{is} with thee, which brought thee up out of the land of Egypt."],
['Deuteronomy','Deu','20','002','2',"And it shall be, when ye are come nigh unto the battle, that the priest shall approach and speak unto the people,"],
['Deuteronomy','Deu','20','003','3',"And shall say unto them, Hear, O Israel, ye approach this day unto battle against your enemies: let not your hearts faint, fear not, and do not tremble, neither be ye terrified because of them;"],
['Deuteronomy','Deu','20','004','4',"For the LORD your God \emph{is} he that goeth with you, to fight for you against your enemies, to save you."],
['Deuteronomy','Deu','20','005','5',"And the officers shall speak unto the people, saying, What man \emph{is} \emph{there} that hath built a new house, and hath not dedicated it? let him go and return to his house, lest he die in the battle, and another man dedicate it."],
['Deuteronomy','Deu','20','006','6',"And what man \emph{is} \emph{he} that hath planted a vineyard, and hath not \emph{yet} eaten of it? let him \emph{also} go and return unto his house, lest he die in the battle, and another man eat of it."],
['Deuteronomy','Deu','20','007','7',"And what man \emph{is} \emph{there} that hath betrothed a wife, and hath not taken her? let him go and return unto his house, lest he die in the battle, and another man take her."],
['Deuteronomy','Deu','20','008','8',"And the officers shall speak further unto the people, and they shall say, What man \emph{is} \emph{there} \emph{that} \emph{is} fearful and fainthearted? let him go and return unto his house, lest his brethren's heart faint as well as his heart."],
['Deuteronomy','Deu','20','009','9',"And it shall be, when the officers have made an end of speaking unto the people, that they shall make captains of the armies to lead the people."],
['Deuteronomy','Deu','20','010','10',"When thou comest nigh unto a city to fight against it, then proclaim peace unto it."],
['Deuteronomy','Deu','20','011','11',"And it shall be, if it make thee answer of peace, and open unto thee, then it shall be, \emph{that} all the people \emph{that} \emph{is} found therein shall be tributaries unto thee, and they shall serve thee."],
['Deuteronomy','Deu','20','012','12',"And if it will make no peace with thee, but will make war against thee, then thou shalt besiege it:"],
['Deuteronomy','Deu','20','013','13',"And when the LORD thy God hath delivered it into thine hands, thou shalt smite every male thereof with the edge of the sword:"],
['Deuteronomy','Deu','20','014','14',"But the women, and the little ones, and the cattle, and all that is in the city, \emph{even} all the spoil thereof, shalt thou take unto thyself; and thou shalt eat the spoil of thine enemies, which the LORD thy God hath given thee."],
['Deuteronomy','Deu','20','015','15',"Thus shalt thou do unto all the cities \emph{which} \emph{are} very far off from thee, which \emph{are} not of the cities of these nations."],
['Deuteronomy','Deu','20','016','16',"But of the cities of these people, which the LORD thy God doth give thee \emph{for} an inheritance, thou shalt save alive nothing that breatheth:"],
['Deuteronomy','Deu','20','017','17',"But thou shalt utterly destroy them; \emph{namely}, the Hittites, and the Amorites, the Canaanites, and the Perizzites, the Hivites, and the Jebusites; as the LORD thy God hath commanded thee:"],
['Deuteronomy','Deu','20','018','18',"That they teach you not to do after all their abominations, which they have done unto their gods; so should ye sin against the LORD your God."],
['Deuteronomy','Deu','20','019','19',"When thou shalt besiege a city a long time, in making war against it to take it, thou shalt not destroy the trees thereof by forcing an axe against them: for thou mayest eat of them, and thou shalt not cut them down (for the tree of the field \emph{is} man's \emph{life}) to employ \emph{them} in the siege:"],
['Deuteronomy','Deu','20','020','20',"Only the trees which thou knowest that they \emph{be} not trees for meat, thou shalt destroy and cut them down; and thou shalt build bulwarks against the city that maketh war with thee, until it be subdued."] 
]

Deuteronomy21 = [
    ['Deuteronomy','Deu','21','001','1',"If \emph{one} be found slain in the land which the LORD thy God giveth thee to possess it, lying in the field, \emph{and} it be not known who hath slain him:"],
['Deuteronomy','Deu','21','002','2',"Then thy elders and thy judges shall come forth, and they shall measure unto the cities which \emph{are} round about him that is slain:"],
['Deuteronomy','Deu','21','003','3',"And it shall be, \emph{that} the city \emph{which} \emph{is} next unto the slain man, even the elders of that city shall take an heifer, which hath not been wrought with, \emph{and} which hath not drawn in the yoke;"],
['Deuteronomy','Deu','21','004','4',"And the elders of that city shall bring down the heifer unto a rough valley, which is neither eared nor sown, and shall strike off the heifer's neck there in the valley:"],
['Deuteronomy','Deu','21','005','5',"And the priests the sons of Levi shall come near; for them the LORD thy God hath chosen to minister unto him, and to bless in the name of the LORD; and by their word shall every controversy and every stroke be \emph{tried}:"],
['Deuteronomy','Deu','21','006','6',"And all the elders of that city, \emph{that} \emph{are} next unto the slain \emph{man}, shall wash their hands over the heifer that is beheaded in the valley:"],
['Deuteronomy','Deu','21','007','7',"And they shall answer and say, Our hands have not shed this blood, neither have our eyes seen \emph{it}."],
['Deuteronomy','Deu','21','008','8',"Be merciful, O LORD, unto thy people Israel, whom thou hast redeemed, and lay not innocent blood unto thy people of Israel's charge. And the blood shall be forgiven them."],
['Deuteronomy','Deu','21','009','9',"So shalt thou put away the \emph{guilt} \emph{of} innocent blood from among you, when thou shalt do \emph{that} \emph{which} \emph{is} right in the sight of the LORD."],
['Deuteronomy','Deu','21','010','10',"When thou goest forth to war against thine enemies, and the LORD thy God hath delivered them into thine hands, and thou hast taken them captive,"],
['Deuteronomy','Deu','21','011','11',"And seest among the captives a beautiful woman, and hast a desire unto her, that thou wouldest have her to thy wife;"],
['Deuteronomy','Deu','21','012','12',"Then thou shalt bring her home to thine house; and she shall shave her head, and pare her nails;"],
['Deuteronomy','Deu','21','013','13',"And she shall put the raiment of her captivity from off her, and shall remain in thine house, and bewail her father and her mother a full month: and after that thou shalt go in unto her, and be her husband, and she shall be thy wife."],
['Deuteronomy','Deu','21','014','14',"And it shall be, if thou have no delight in her, then thou shalt let her go whither she will; but thou shalt not sell her at all for money, thou shalt not make merchandise of her, because thou hast humbled her."],
['Deuteronomy','Deu','21','015','15',"If a man have two wives, one beloved, and another hated, and they have born him children, \emph{both} the beloved and the hated; and \emph{if} the firstborn son be hers that was hated:"],
['Deuteronomy','Deu','21','016','16',"Then it shall be, when he maketh his sons to inherit \emph{that} which he hath, \emph{that} he may not make the son of the beloved firstborn before the son of the hated, \emph{which} \emph{is} \emph{indeed} the firstborn:"],
['Deuteronomy','Deu','21','017','17',"But he shall acknowledge the son of the hated \emph{for} the firstborn, by giving him a double portion of all that he hath: for he \emph{is} the beginning of his strength; the right of the firstborn \emph{is} his."],
['Deuteronomy','Deu','21','018','18',"If a man have a stubborn and rebellious son, which will not obey the voice of his father, or the voice of his mother, and \emph{that}, when they have chastened him, will not hearken unto them:"],
['Deuteronomy','Deu','21','019','19',"Then shall his father and his mother lay hold on him, and bring him out unto the elders of his city, and unto the gate of his place;"],
['Deuteronomy','Deu','21','020','20',"And they shall say unto the elders of his city, This our son \emph{is} stubborn and rebellious, he will not obey our voice; \emph{he} \emph{is} a glutton, and a drunkard."],
['Deuteronomy','Deu','21','021','21',"And all the men of his city shall stone him with stones, that he die: so shalt thou put evil away from among you; and all Israel shall hear, and fear."],
['Deuteronomy','Deu','21','022','22',"And if a man have committed a sin worthy of death, and he be to be put to death, and thou hang him on a tree:"],
['Deuteronomy','Deu','21','023','23',"His body shall not remain all night upon the tree, but thou shalt in any wise bury him that day; (for he that is hanged \emph{is} accursed of God;) that thy land be not defiled, which the LORD thy God giveth thee \emph{for} an inheritance."]
]

Deuteronomy22 = [
    ['Deuteronomy','Deu','22','001','1',"Thou shalt not see thy brother's ox or his sheep go astray, and hide thyself from them: thou shalt in any case bring them again unto thy brother."],
['Deuteronomy','Deu','22','002','2',"And if thy brother \emph{be} not nigh unto thee, or if thou know him not, then thou shalt bring it unto thine own house, and it shall be with thee until thy brother seek after it, and thou shalt restore it to him again."],
['Deuteronomy','Deu','22','003','3',"In like manner shalt thou do with his ass; and so shalt thou do with his raiment; and with all lost thing of thy brother's, which he hath lost, and thou hast found, shalt thou do likewise: thou mayest not hide thyself."],
['Deuteronomy','Deu','22','004','4',"Thou shalt not see thy brother's ass or his ox fall down by the way, and hide thyself from them: thou shalt surely help him to lift \emph{them} up again."],
['Deuteronomy','Deu','22','005','5',"The woman shall not wear that which pertaineth unto a man, neither shall a man put on a woman's garment: for all that do so \emph{are} abomination unto the LORD thy God."],
['Deuteronomy','Deu','22','006','6',"If a bird's nest chance to be before thee in the way in any tree, or on the ground, \emph{whether} \emph{they} \emph{be} young ones, or eggs, and the dam sitting upon the young, or upon the eggs, thou shalt not take the dam with the young:"],
['Deuteronomy','Deu','22','007','7',"\emph{But} thou shalt in any wise let the dam go, and take the young to thee; that it may be well with thee, and \emph{that} thou mayest prolong \emph{thy} days."],
['Deuteronomy','Deu','22','008','8',"When thou buildest a new house, then thou shalt make a battlement for thy roof, that thou bring not blood upon thine house, if any man fall from thence."],
['Deuteronomy','Deu','22','009','9',"Thou shalt not sow thy vineyard with divers seeds: lest the fruit of thy seed which thou hast sown, and the fruit of thy vineyard, be defiled."],
['Deuteronomy','Deu','22','010','10',"Thou shalt not plow with an ox and an ass together."],
['Deuteronomy','Deu','22','011','11',"Thou shalt not wear a garment of divers sorts, \emph{as} of woollen and linen together."],
['Deuteronomy','Deu','22','012','12',"Thou shalt make thee fringes upon the four quarters of thy vesture, wherewith thou coverest \emph{thyself}."],
['Deuteronomy','Deu','22','013','13',"If any man take a wife, and go in unto her, and hate her,"],
['Deuteronomy','Deu','22','014','14',"And give occasions of speech against her, and bring up an evil name upon her, and say, I took this woman, and when I came to her, I found her not a maid:"],
['Deuteronomy','Deu','22','015','15',"Then shall the father of the damsel, and her mother, take and bring forth \emph{the} \emph{tokens} \emph{of} the damsel's virginity unto the elders of the city in the gate:"],
['Deuteronomy','Deu','22','016','16',"And the damsel's father shall say unto the elders, I gave my daughter unto this man to wife, and he hateth her;"],
['Deuteronomy','Deu','22','017','17',"And, lo, he hath given occasions of speech \emph{against} \emph{her}, saying, I found not thy daughter a maid; and yet these \emph{are} \emph{the} \emph{tokens} \emph{of} my daughter's virginity. And they shall spread the cloth before the elders of the city."],
['Deuteronomy','Deu','22','018','18',"And the elders of that city shall take that man and chastise him;"],
['Deuteronomy','Deu','22','019','19',"And they shall amerce him in an hundred \emph{shekels} of silver, and give \emph{them} unto the father of the damsel, because he hath brought up an evil name upon a virgin of Israel: and she shall be his wife; he may not put her away all his days."],
['Deuteronomy','Deu','22','020','20',"But if this thing be true, \emph{and} \emph{the} \emph{tokens} \emph{of} virginity be not found for the damsel:"],
['Deuteronomy','Deu','22','021','21',"Then they shall bring out the damsel to the door of her father's house, and the men of her city shall stone her with stones that she die: because she hath wrought folly in Israel, to play the whore in her father's house: so shalt thou put evil away from among you."],
['Deuteronomy','Deu','22','022','22',"If a man be found lying with a woman married to an husband, then they shall both of them die, \emph{both} the man that lay with the woman, and the woman: so shalt thou put away evil from Israel."],
['Deuteronomy','Deu','22','023','23',"If a damsel \emph{that} \emph{is} a virgin be betrothed unto an husband, and a man find her in the city, and lie with her;"],
['Deuteronomy','Deu','22','024','24',"Then ye shall bring them both out unto the gate of that city, and ye shall stone them with stones that they die; the damsel, because she cried not, \emph{being} in the city; and the man, because he hath humbled his neighbour's wife: so thou shalt put away evil from among you."],
['Deuteronomy','Deu','22','025','25',"But if a man find a betrothed damsel in the field, and the man force her, and lie with her: then the man only that lay with her shall die:"],
['Deuteronomy','Deu','22','026','26',"But unto the damsel thou shalt do nothing; \emph{there} \emph{is} in the damsel no sin \emph{worthy} of death: for as when a man riseth against his neighbour, and slayeth him, even so \emph{is} this matter:"],
['Deuteronomy','Deu','22','027','27',"For he found her in the field, \emph{and} the betrothed damsel cried, and \emph{there} \emph{was} none to save her."],
['Deuteronomy','Deu','22','028','28',"If a man find a damsel \emph{that} \emph{is} a virgin, which is not betrothed, and lay hold on her, and lie with her, and they be found;"],
['Deuteronomy','Deu','22','029','29',"Then the man that lay with her shall give unto the damsel's father fifty \emph{shekels} of silver, and she shall be his wife; because he hath humbled her, he may not put her away all his days."],
['Deuteronomy','Deu','22','030','30',"A man shall not take his father's wife, nor discover his father's skirt."]
]

Deuteronomy23 = [
    ['Deuteronomy','Deu','23','001','1',"He that is wounded in the stones, or hath his privy member cut off, shall not enter into the congregation of the LORD."],
['Deuteronomy','Deu','23','002','2',"A bastard shall not enter into the congregation of the LORD; even to his tenth generation shall he not enter into the congregation of the LORD."],
['Deuteronomy','Deu','23','003','3',"An Ammonite or Moabite shall not enter into the congregation of the LORD; even to their tenth generation shall they not enter into the congregation of the LORD for ever:"],
['Deuteronomy','Deu','23','004','4',"Because they met you not with bread and with water in the way, when ye came forth out of Egypt; and because they hired against thee Balaam the son of Beor of Pethor of Mesopotamia, to curse thee."],
['Deuteronomy','Deu','23','005','5',"Nevertheless the LORD thy God would not hearken unto Balaam; but the LORD thy God turned the curse into a blessing unto thee, because the LORD thy God loved thee."],
['Deuteronomy','Deu','23','006','6',"Thou shalt not seek their peace nor their prosperity all thy days for ever."],
['Deuteronomy','Deu','23','007','7',"Thou shalt not abhor an Edomite; for he \emph{is} thy brother: thou shalt not abhor an Egyptian; because thou wast a stranger in his land."],
['Deuteronomy','Deu','23','008','8',"The children that are begotten of them shall enter into the congregation of the LORD in their third generation."],
['Deuteronomy','Deu','23','009','9',"When the host goeth forth against thine enemies, then keep thee from every wicked thing."],
['Deuteronomy','Deu','23','010','10',"If there be among you any man, that is not clean by reason of uncleanness that chanceth him by night, then shall he go abroad out of the camp, he shall not come within the camp:"],
['Deuteronomy','Deu','23','011','11',"But it shall be, when evening cometh on, he shall wash \emph{himself} with water: and when the sun is down, he shall come into the camp \emph{again}."],
['Deuteronomy','Deu','23','012','12',"Thou shalt have a place also without the camp, whither thou shalt go forth abroad:"],
['Deuteronomy','Deu','23','013','13',"And thou shalt have a paddle upon thy weapon; and it shall be, when thou wilt ease thyself abroad, thou shalt dig therewith, and shalt turn back and cover that which cometh from thee:"],
['Deuteronomy','Deu','23','014','14',"For the LORD thy God walketh in the midst of thy camp, to deliver thee, and to give up thine enemies before thee; therefore shall thy camp be holy: that he see no unclean thing in thee, and turn away from thee."],
['Deuteronomy','Deu','23','015','15',"Thou shalt not deliver unto his master the servant which is escaped from his master unto thee:"],
['Deuteronomy','Deu','23','016','16',"He shall dwell with thee, \emph{even} among you, in that place which he shall choose in one of thy gates, where it liketh him best: thou shalt not oppress him."],
['Deuteronomy','Deu','23','017','17',"There shall be no whore of the daughters of Israel, nor a sodomite of the sons of Israel."],
['Deuteronomy','Deu','23','018','18',"Thou shalt not bring the hire of a whore, or the price of a dog, into the house of the LORD thy God for any vow: for even both these \emph{are} abomination unto the LORD thy God."],
['Deuteronomy','Deu','23','019','19',"Thou shalt not lend upon usury to thy brother; usury of money, usury of victuals, usury of any thing that is lent upon usury:"],
['Deuteronomy','Deu','23','020','20',"Unto a stranger thou mayest lend upon usury; but unto thy brother thou shalt not lend upon usury: that the LORD thy God may bless thee in all that thou settest thine hand to in the land whither thou goest to possess it."],
['Deuteronomy','Deu','23','021','21',"When thou shalt vow a vow unto the LORD thy God, thou shalt not slack to pay it: for the LORD thy God will surely require it of thee; and it would be sin in thee."],
['Deuteronomy','Deu','23','022','22',"But if thou shalt forbear to vow, it shall be no sin in thee."],
['Deuteronomy','Deu','23','023','23',"That which is gone out of thy lips thou shalt keep and perform; \emph{even} a freewill offering, according as thou hast vowed unto the LORD thy God, which thou hast promised with thy mouth."],
['Deuteronomy','Deu','23','024','24',"When thou comest into thy neighbour's vineyard, then thou mayest eat grapes thy fill at thine own pleasure; but thou shalt not put \emph{any} in thy vessel."],
['Deuteronomy','Deu','23','025','25',"When thou comest into the standing corn of thy neighbour, then thou mayest pluck the ears with thine hand; but thou shalt not move a sickle unto thy neighbour's standing corn."]
]

Deuteronomy24 = [
    ['Deuteronomy','Deu','24','001','1',"When a man hath taken a wife, and married her, and it come to pass that she find no favour in his eyes, because he hath found some uncleanness in her: then let him write her a bill of divorcement, and give \emph{it} in her hand, and send her out of his house."],
['Deuteronomy','Deu','24','002','2',"And when she is departed out of his house, she may go and be another man's \emph{wife}."],
['Deuteronomy','Deu','24','003','3',"And \emph{if} the latter husband hate her, and write her a bill of divorcement, and giveth \emph{it} in her hand, and sendeth her out of his house; or if the latter husband die, which took her \emph{to} \emph{be} his wife;"],
['Deuteronomy','Deu','24','004','4',"Her former husband, which sent her away, may not take her again to be his wife, after that she is defiled; for that \emph{is} abomination before the LORD: and thou shalt not cause the land to sin, which the LORD thy God giveth thee \emph{for} an inheritance."],
['Deuteronomy','Deu','24','005','5',"When a man hath taken a new wife, he shall not go out to war, neither shall he be charged with any business: \emph{but} he shall be free at home one year, and shall cheer up his wife which he hath taken."],
['Deuteronomy','Deu','24','006','6',"No man shall take the nether or the upper millstone to pledge: for he taketh \emph{a} \emph{man's} life to pledge."],
['Deuteronomy','Deu','24','007','7',"If a man be found stealing any of his brethren of the children of Israel, and maketh merchandise of him, or selleth him; then that thief shall die; and thou shalt put evil away from among you."],
['Deuteronomy','Deu','24','008','8',"Take heed in the plague of leprosy, that thou observe diligently, and do according to all that the priests the Levites shall teach you: as I commanded them, \emph{so} ye shall observe to do."],
['Deuteronomy','Deu','24','009','9',"Remember what the LORD thy God did unto Miriam by the way, after that ye were come forth out of Egypt."],
['Deuteronomy','Deu','24','010','10',"When thou dost lend thy brother any thing, thou shalt not go into his house to fetch his pledge."],
['Deuteronomy','Deu','24','011','11',"Thou shalt stand abroad, and the man to whom thou dost lend shall bring out the pledge abroad unto thee."],
['Deuteronomy','Deu','24','012','12',"And if the man \emph{be} poor, thou shalt not sleep with his pledge:"],
['Deuteronomy','Deu','24','013','13',"In any case thou shalt deliver him the pledge again when the sun goeth down, that he may sleep in his own raiment, and bless thee: and it shall be righteousness unto thee before the LORD thy God."],
['Deuteronomy','Deu','24','014','14',"Thou shalt not oppress an hired servant \emph{that} \emph{is} poor and needy, \emph{whether} \emph{he} \emph{be} of thy brethren, or of thy strangers that \emph{are} in thy land within thy gates:"],
['Deuteronomy','Deu','24','015','15',"At his day thou shalt give \emph{him} his hire, neither shall the sun go down upon it; for he \emph{is} poor, and setteth his heart upon it: lest he cry against thee unto the LORD, and it be sin unto thee."],
['Deuteronomy','Deu','24','016','16',"The fathers shall not be put to death for the children, neither shall the children be put to death for the fathers: every man shall be put to death for his own sin."],
['Deuteronomy','Deu','24','017','17',"Thou shalt not pervert the judgment of the stranger, \emph{nor} of the fatherless; nor take a widow's raiment to pledge:"],
['Deuteronomy','Deu','24','018','18',"But thou shalt remember that thou wast a bondman in Egypt, and the LORD thy God redeemed thee thence: therefore I command thee to do this thing."],
['Deuteronomy','Deu','24','019','19',"When thou cuttest down thine harvest in thy field, and hast forgot a sheaf in the field, thou shalt not go again to fetch it: it shall be for the stranger, for the fatherless, and for the widow: that the LORD thy God may bless thee in all the work of thine hands."],
['Deuteronomy','Deu','24','020','20',"When thou beatest thine olive tree, thou shalt not go over the boughs again: it shall be for the stranger, for the fatherless, and for the widow."],
['Deuteronomy','Deu','24','021','21',"When thou gatherest the grapes of thy vineyard, thou shalt not glean \emph{it} afterward: it shall be for the stranger, for the fatherless, and for the widow."],
['Deuteronomy','Deu','24','022','22',"And thou shalt remember that thou wast a bondman in the land of Egypt: therefore I command thee to do this thing."]
]

Deuteronomy25 = [
    ['Deuteronomy','Deu','25','001','1',"If there be a controversy between men, and they come unto judgment, that \emph{the} \emph{judges} may judge them; then they shall justify the righteous, and condemn the wicked."],
['Deuteronomy','Deu','25','002','2',"And it shall be, if the wicked man \emph{be} worthy to be beaten, that the judge shall cause him to lie down, and to be beaten before his face, according to his fault, by a certain number."],
['Deuteronomy','Deu','25','003','3',"Forty stripes he may give him, \emph{and} not exceed: lest, \emph{if} he should exceed, and beat him above these with many stripes, then thy brother should seem vile unto thee."],
['Deuteronomy','Deu','25','004','4',"Thou shalt not muzzle the ox when he treadeth out \emph{the} \emph{corn}."],
['Deuteronomy','Deu','25','005','5',"If brethren dwell together, and one of them die, and have no child, the wife of the dead shall not marry without unto a stranger: her husband's brother shall go in unto her, and take her to him to wife, and perform the duty of an husband's brother unto her."],
['Deuteronomy','Deu','25','006','6',"And it shall be, \emph{that} the firstborn which she beareth shall succeed in the name of his brother \emph{which} \emph{is} dead, that his name be not put out of Israel."],
['Deuteronomy','Deu','25','007','7',"And if the man like not to take his brother's wife, then let his brother's wife go up to the gate unto the elders, and say, My husband's brother refuseth to raise up unto his brother a name in Israel, he will not perform the duty of my husband's brother."],
['Deuteronomy','Deu','25','008','8',"Then the elders of his city shall call him, and speak unto him: and \emph{if} he stand \emph{to} \emph{it}, and say, I like not to take her;"],
['Deuteronomy','Deu','25','009','9',"Then shall his brother's wife come unto him in the presence of the elders, and loose his shoe from off his foot, and spit in his face, and shall answer and say, So shall it be done unto that man that will not build up his brother's house."],
['Deuteronomy','Deu','25','010','10',"And his name shall be called in Israel, The house of him that hath his shoe loosed."],
['Deuteronomy','Deu','25','011','11',"When men strive together one with another, and the wife of the one draweth near for to deliver her husband out of the hand of him that smiteth him, and putteth forth her hand, and taketh him by the secrets:"],
['Deuteronomy','Deu','25','012','12',"Then thou shalt cut off her hand, thine eye shall not pity \emph{her}."],
['Deuteronomy','Deu','25','013','13',"Thou shalt not have in thy bag divers weights, a great and a small."],
['Deuteronomy','Deu','25','014','14',"Thou shalt not have in thine house divers measures, a great and a small."],
['Deuteronomy','Deu','25','015','15',"\emph{But} thou shalt have a perfect and just weight, a perfect and just measure shalt thou have: that thy days may be lengthened in the land which the LORD thy God giveth thee."],
['Deuteronomy','Deu','25','016','16',"For all that do such things, \emph{and} all that do unrighteously, \emph{are} an abomination unto the LORD thy God."],
['Deuteronomy','Deu','25','017','17',"Remember what Amalek did unto thee by the way, when ye were come forth out of Egypt;"],
['Deuteronomy','Deu','25','018','18',"How he met thee by the way, and smote the hindmost of thee, \emph{even} all \emph{that} \emph{were} feeble behind thee, when thou \emph{wast} faint and weary; and he feared not God."],
['Deuteronomy','Deu','25','019','19',"Therefore it shall be, when the LORD thy God hath given thee rest from all thine enemies round about, in the land which the LORD thy God giveth thee \emph{for} an inheritance to possess it, \emph{that} thou shalt blot out the remembrance of Amalek from under heaven; thou shalt not forget \emph{it}."]
]

Deuteronomy26 = [
    ['Deuteronomy','Deu','26','001','1',"And it shall be, when thou \emph{art} come in unto the land which the LORD thy God giveth thee \emph{for} an inheritance, and possessest it, and dwellest therein;"],
['Deuteronomy','Deu','26','002','2',"That thou shalt take of the first of all the fruit of the earth, which thou shalt bring of thy land that the LORD thy God giveth thee, and shalt put \emph{it} in a basket, and shalt go unto the place which the LORD thy God shall choose to place his name there."],
['Deuteronomy','Deu','26','003','3',"And thou shalt go unto the priest that shall be in those days, and say unto him, I profess this day unto the LORD thy God, that I am come unto the country which the LORD sware unto our fathers for to give us."],
['Deuteronomy','Deu','26','004','4',"And the priest shall take the basket out of thine hand, and set it down before the altar of the LORD thy God."],
['Deuteronomy','Deu','26','005','5',"And thou shalt speak and say before the LORD thy God, A Syrian ready to perish \emph{was} my father, and he went down into Egypt, and sojourned there with a few, and became there a nation, great, mighty, and populous:"],
['Deuteronomy','Deu','26','006','6',"And the Egyptians evil entreated us, and afflicted us, and laid upon us hard bondage:"],
['Deuteronomy','Deu','26','007','7',"And when we cried unto the LORD God of our fathers, the LORD heard our voice, and looked on our affliction, and our labour, and our oppression:"],
['Deuteronomy','Deu','26','008','8',"And the LORD brought us forth out of Egypt with a mighty hand, and with an outstretched arm, and with great terribleness, and with signs, and with wonders:"],
['Deuteronomy','Deu','26','009','9',"And he hath brought us into this place, and hath given us this land, \emph{even} a land that floweth with milk and honey."],
['Deuteronomy','Deu','26','010','10',"And now, behold, I have brought the firstfruits of the land, which thou, O LORD, hast given me. And thou shalt set it before the LORD thy God, and worship before the LORD thy God:"],
['Deuteronomy','Deu','26','011','11',"And thou shalt rejoice in every good \emph{thing} which the LORD thy God hath given unto thee, and unto thine house, thou, and the Levite, and the stranger that \emph{is} among you."],
['Deuteronomy','Deu','26','012','12',"When thou hast made an end of tithing all the tithes of thine increase the third year, \emph{which} \emph{is} the year of tithing, and hast given \emph{it} unto the Levite, the stranger, the fatherless, and the widow, that they may eat within thy gates, and be filled;"],
['Deuteronomy','Deu','26','013','13',"Then thou shalt say before the LORD thy God, I have brought away the hallowed things out of \emph{mine} house, and also have given them unto the Levite, and unto the stranger, to the fatherless, and to the widow, according to all thy commandments which thou hast commanded me: I have not transgressed thy commandments, neither have I forgotten \emph{them}:"],
['Deuteronomy','Deu','26','014','14',"I have not eaten thereof in my mourning, neither have I taken away \emph{ought} thereof for \emph{any} unclean \emph{use}, nor given \emph{ought} thereof for the dead: \emph{but} I have hearkened to the voice of the LORD my God, \emph{and} have done according to all that thou hast commanded me."],
['Deuteronomy','Deu','26','015','15',"Look down from thy holy habitation, from heaven, and bless thy people Israel, and the land which thou hast given us, as thou swarest unto our fathers, a land that floweth with milk and honey."],
['Deuteronomy','Deu','26','016','16',"This day the LORD thy God hath commanded thee to do these statutes and judgments: thou shalt therefore keep and do them with all thine heart, and with all thy soul."],
['Deuteronomy','Deu','26','017','17',"Thou hast avouched the LORD this day to be thy God, and to walk in his ways, and to keep his statutes, and his commandments, and his judgments, and to hearken unto his voice:"],
['Deuteronomy','Deu','26','018','18',"And the LORD hath avouched thee this day to be his peculiar people, as he hath promised thee, and that \emph{thou} shouldest keep all his commandments;"],
['Deuteronomy','Deu','26','019','19',"And to make thee high above all nations which he hath made, in praise, and in name, and in honour; and that thou mayest be an holy people unto the LORD thy God, as he hath spoken."]
]

Deuteronomy27 = [
    ['Deuteronomy','Deu','27','001','1',"And Moses with the elders of Israel commanded the people, saying, Keep all the commandments which I command you this day."],
['Deuteronomy','Deu','27','002','2',"And it shall be on the day when ye shall pass over Jordan unto the land which the LORD thy God giveth thee, that thou shalt set thee up great stones, and plaister them with plaister:"],
['Deuteronomy','Deu','27','003','3',"And thou shalt write upon them all the words of this law, when thou art passed over, that thou mayest go in unto the land which the LORD thy God giveth thee, a land that floweth with milk and honey; as the LORD God of thy fathers hath promised thee."],
['Deuteronomy','Deu','27','004','4',"Therefore it shall be when ye be gone over Jordan, \emph{that} ye shall set up these stones, which I command you this day, in mount Ebal, and thou shalt plaister them with plaister."],
['Deuteronomy','Deu','27','005','5',"And there shalt thou build an altar unto the LORD thy God, an altar of stones: thou shalt not lift up \emph{any} iron \emph{tool} upon them."],
['Deuteronomy','Deu','27','006','6',"Thou shalt build the altar of the LORD thy God of whole stones: and thou shalt offer burnt offerings thereon unto the LORD thy God:"],
['Deuteronomy','Deu','27','007','7',"And thou shalt offer peace offerings, and shalt eat there, and rejoice before the LORD thy God."],
['Deuteronomy','Deu','27','008','8',"And thou shalt write upon the stones all the words of this law very plainly."],
['Deuteronomy','Deu','27','009','9',"And Moses and the priests the Levites spake unto all Israel, saying, Take heed, and hearken, O Israel; this day thou art become the people of the LORD thy God."],
['Deuteronomy','Deu','27','010','10',"Thou shalt therefore obey the voice of the LORD thy God, and do his commandments and his statutes, which I command thee this day."],
['Deuteronomy','Deu','27','011','11',"And Moses charged the people the same day, saying,"],
['Deuteronomy','Deu','27','012','12',"These shall stand upon mount Gerizim to bless the people, when ye are come over Jordan; Simeon, and Levi, and Judah, and Issachar, and Joseph, and Benjamin:"],
['Deuteronomy','Deu','27','013','13',"And these shall stand upon mount Ebal to curse; Reuben, Gad, and Asher, and Zebulun, Dan, and Naphtali."],
['Deuteronomy','Deu','27','014','14',"And the Levites shall speak, and say unto all the men of Israel with a loud voice,"],
['Deuteronomy','Deu','27','015','15',"Cursed \emph{be} the man that maketh \emph{any} graven or molten image, an abomination unto the LORD, the work of the hands of the craftsman, and putteth \emph{it} in \emph{a} secret \emph{place}. And all the people shall answer and say, Amen."],
['Deuteronomy','Deu','27','016','16',"Cursed \emph{be} he that setteth light by his father or his mother. And all the people shall say, Amen."],
['Deuteronomy','Deu','27','017','17',"Cursed \emph{be} he that removeth his neighbour's landmark. And all the people shall say, Amen."],
['Deuteronomy','Deu','27','018','18',"Cursed \emph{be} he that maketh the blind to wander out of the way. And all the people shall say, Amen."],
['Deuteronomy','Deu','27','019','19',"Cursed \emph{be} he that perverteth the judgment of the stranger, fatherless, and widow. And all the people shall say, Amen."],
['Deuteronomy','Deu','27','020','20',"Cursed \emph{be} he that lieth with his father's wife; because he uncovereth his father's skirt. And all the people shall say, Amen."],
['Deuteronomy','Deu','27','021','21',"Cursed \emph{be} he that lieth with any manner of beast. And all the people shall say, Amen."],
['Deuteronomy','Deu','27','022','22',"Cursed \emph{be} he that lieth with his sister, the daughter of his father, or the daughter of his mother. And all the people shall say, Amen."],
['Deuteronomy','Deu','27','023','23',"Cursed \emph{be} he that lieth with his mother in law. And all the people shall say, Amen."],
['Deuteronomy','Deu','27','024','24',"Cursed \emph{be} he that smiteth his neighbour secretly. And all the people shall say, Amen."],
['Deuteronomy','Deu','27','025','25',"Cursed \emph{be} he that taketh reward to slay an innocent person. And all the people shall say, Amen."],
['Deuteronomy','Deu','27','026','26',"Cursed \emph{be} he that confirmeth not \emph{all} the words of this law to do them. And all the people shall say, Amen."]
]

Deuteronomy28 = [
    ['Deuteronomy','Due','28','001','1',"And it shall come to pass, if thou shalt hearken diligently unto the voice of the LORD thy God, to observe \emph{and} to do all his commandments which I command thee this day, that the LORD thy God will set thee on high above all nations of the earth:"],
['Deuteronomy','Due','28','002','2',"And all these blessings shall come on thee, and overtake thee, if thou shalt hearken unto the voice of the LORD thy God."],
['Deuteronomy','Due','28','003','3',"Blessed \emph{shalt} thou \emph{be} in the city, and blessed \emph{shalt} thou \emph{be} in the field."],
['Deuteronomy','Due','28','004','4',"Blessed \emph{shall} \emph{be} the fruit of thy body, and the fruit of thy ground, and the fruit of thy cattle, the increase of thy kine, and the flocks of thy sheep."],
['Deuteronomy','Due','28','005','5',"Blessed \emph{shall} \emph{be} thy basket and thy store."],
['Deuteronomy','Due','28','006','6',"Blessed \emph{shalt} thou \emph{be} when thou comest in, and blessed \emph{shalt} thou \emph{be} when thou goest out."],
['Deuteronomy','Due','28','007','7',"The LORD shall cause thine enemies that rise up against thee to be smitten before thy face: they shall come out against thee one way, and flee before thee seven ways."],
['Deuteronomy','Due','28','008','8',"The LORD shall command the blessing upon thee in thy storehouses, and in all that thou settest thine hand unto; and he shall bless thee in the land which the LORD thy God giveth thee."],
['Deuteronomy','Due','28','009','9',"The LORD shall establish thee an holy people unto himself, as he hath sworn unto thee, if thou shalt keep the commandments of the LORD thy God, and walk in his ways."],
['Deuteronomy','Due','28','010','10',"And all people of the earth shall see that thou art called by the name of the LORD; and they shall be afraid of thee."],
['Deuteronomy','Due','28','011','11',"And the LORD shall make thee plenteous in goods, in the fruit of thy body, and in the fruit of thy cattle, and in the fruit of thy ground, in the land which the LORD sware unto thy fathers to give thee."],
['Deuteronomy','Due','28','012','12',"The LORD shall open unto thee his good treasure, the heaven to give the rain unto thy land in his season, and to bless all the work of thine hand: and thou shalt lend unto many nations, and thou shalt not borrow."],
['Deuteronomy','Due','28','013','13',"And the LORD shall make thee the head, and not the tail; and thou shalt be above only, and thou shalt not be beneath; if that thou hearken unto the commandments of the LORD thy God, which I command thee this day, to observe and to do \emph{them}:"],
['Deuteronomy','Due','28','014','14',"And thou shalt not go aside from any of the words which I command thee this day, \emph{to} the right hand, or \emph{to} the left, to go after other gods to serve them."],
['Deuteronomy','Due','28','015','15',"But it shall come to pass, if thou wilt not hearken unto the voice of the LORD thy God, to observe to do all his commandments and his statutes which I command thee this day; that all these curses shall come upon thee, and overtake thee:"],
['Deuteronomy','Due','28','016','16',"Cursed \emph{shalt} thou \emph{be} in the city, and cursed \emph{shalt} thou \emph{be} in the field."],
['Deuteronomy','Due','28','017','17',"Cursed \emph{shall} \emph{be} thy basket and thy store."],
['Deuteronomy','Due','28','018','18',"Cursed \emph{shall} \emph{be} the fruit of thy body, and the fruit of thy land, the increase of thy kine, and the flocks of thy sheep."],
['Deuteronomy','Due','28','019','19',"Cursed \emph{shalt} thou \emph{be} when thou comest in, and cursed \emph{shalt} thou \emph{be} when thou goest out."],
['Deuteronomy','Due','28','020','20',"The LORD shall send upon thee cursing, vexation, and rebuke, in all that thou settest thine hand unto for to do, until thou be destroyed, and until thou perish quickly; because of the wickedness of thy doings, whereby thou hast forsaken me."],
['Deuteronomy','Due','28','021','21',"The LORD shall make the pestilence cleave unto thee, until he have consumed thee from off the land, whither thou goest to possess it."],
['Deuteronomy','Due','28','022','22',"The LORD shall smite thee with a consumption, and with a fever, and with an inflammation, and with an extreme burning, and with the sword, and with blasting, and with mildew; and they shall pursue thee until thou perish."],
['Deuteronomy','Due','28','023','23',"And thy heaven that \emph{is} over thy head shall be brass, and the earth that is under thee \emph{shall} \emph{be} iron."],
['Deuteronomy','Due','28','024','24',"The LORD shall make the rain of thy land powder and dust: from heaven shall it come down upon thee, until thou be destroyed."],
['Deuteronomy','Due','28','025','25',"The LORD shall cause thee to be smitten before thine enemies: thou shalt go out one way against them, and flee seven ways before them: and shalt be removed into all the kingdoms of the earth."],
['Deuteronomy','Due','28','026','26',"And thy carcase shall be meat unto all fowls of the air, and unto the beasts of the earth, and no man shall fray \emph{them} away."],
['Deuteronomy','Due','28','027','27',"The LORD will smite thee with the botch of Egypt, and with the emerods, and with the scab, and with the itch, whereof thou canst not be healed."],
['Deuteronomy','Due','28','028','28',"The LORD shall smite thee with madness, and blindness, and astonishment of heart:"],
['Deuteronomy','Due','28','029','29',"And thou shalt grope at noonday, as the blind gropeth in darkness, and thou shalt not prosper in thy ways: and thou shalt be only oppressed and spoiled evermore, and no man shall save \emph{thee}."],
['Deuteronomy','Due','28','030','30',"Thou shalt betroth a wife, and another man shall lie with her: thou shalt build an house, and thou shalt not dwell therein: thou shalt plant a vineyard, and shalt not gather the grapes thereof."],
['Deuteronomy','Due','28','031','31',"Thine ox \emph{shall} \emph{be} slain before thine eyes, and thou shalt not eat thereof: thine ass \emph{shall} \emph{be} violently taken away from before thy face, and shall not be restored to thee: thy sheep \emph{shall} \emph{be} given unto thine enemies, and thou shalt have none to rescue \emph{them}."],
['Deuteronomy','Due','28','032','32',"Thy sons and thy daughters \emph{shall} \emph{be} given unto another people, and thine eyes shall look, and fail \emph{with} \emph{longing} for them all the day long: and \emph{there} \emph{shall} \emph{be} no might in thine hand."],
['Deuteronomy','Due','28','033','33',"The fruit of thy land, and all thy labours, shall a nation which thou knowest not eat up; and thou shalt be only oppressed and crushed alway:"],
['Deuteronomy','Due','28','034','34',"So that thou shalt be mad for the sight of thine eyes which thou shalt see."],
['Deuteronomy','Due','28','035','35',"The LORD shall smite thee in the knees, and in the legs, with a sore botch that cannot be healed, from the sole of thy foot unto the top of thy head."],
['Deuteronomy','Due','28','036','36',"The LORD shall bring thee, and thy king which thou shalt set over thee, unto a nation which neither thou nor thy fathers have known; and there shalt thou serve other gods, wood and stone."],
['Deuteronomy','Due','28','037','37',"And thou shalt become an astonishment, a proverb, and a byword, among all nations whither the LORD shall lead thee."],
['Deuteronomy','Due','28','038','38',"Thou shalt carry much seed out into the field, and shalt gather \emph{but} little in; for the locust shall consume it."],
['Deuteronomy','Due','28','039','39',"Thou shalt plant vineyards, and dress \emph{them}, but shalt neither drink \emph{of} the wine, nor gather \emph{the} \emph{grapes}; for the worms shall eat them."],
['Deuteronomy','Due','28','040','40',"Thou shalt have olive trees throughout all thy coasts, but thou shalt not anoint \emph{thyself} with the oil; for thine olive shall cast \emph{his} \emph{fruit}."],
['Deuteronomy','Due','28','041','41',"Thou shalt beget sons and daughters, but thou shalt not enjoy them; for they shall go into captivity."],
['Deuteronomy','Due','28','042','42',"All thy trees and fruit of thy land shall the locust consume."],
['Deuteronomy','Due','28','043','43',"The stranger that \emph{is} within thee shall get up above thee very high; and thou shalt come down very low."],
['Deuteronomy','Due','28','044','44',"He shall lend to thee, and thou shalt not lend to him: he shall be the head, and thou shalt be the tail."],
['Deuteronomy','Due','28','045','45',"Moreover all these curses shall come upon thee, and shall pursue thee, and overtake thee, till thou be destroyed; because thou hearkenedst not unto the voice of the LORD thy God, to keep his commandments and his statutes which he commanded thee:"],
['Deuteronomy','Due','28','046','46',"And they shall be upon thee for a sign and for a wonder, and upon thy seed for ever."],
['Deuteronomy','Due','28','047','47',"Because thou servedst not the LORD thy God with joyfulness, and with gladness of heart, for the abundance of all \emph{things};"],
['Deuteronomy','Due','28','048','48',"Therefore shalt thou serve thine enemies which the LORD shall send against thee, in hunger, and in thirst, and in nakedness, and in want of all \emph{things}: and he shall put a yoke of iron upon thy neck, until he have destroyed thee."],
['Deuteronomy','Due','28','049','49',"The LORD shall bring a nation against thee from far, from the end of the earth, \emph{as} \emph{swift} as the eagle flieth; a nation whose tongue thou shalt not understand;"],
['Deuteronomy','Due','28','050','50',"A nation of fierce countenance, which shall not regard the person of the old, nor shew favour to the young:"],
['Deuteronomy','Due','28','051','51',"And he shall eat the fruit of thy cattle, and the fruit of thy land, until thou be destroyed: which \emph{also} shall not leave thee \emph{either} corn, wine, or oil, \emph{or} the increase of thy kine, or flocks of thy sheep, until he have destroyed thee."],
['Deuteronomy','Due','28','052','52',"And he shall besiege thee in all thy gates, until thy high and fenced walls come down, wherein thou trustedst, throughout all thy land: and he shall besiege thee in all thy gates throughout all thy land, which the LORD thy God hath given thee."],
['Deuteronomy','Due','28','053','53',"And thou shalt eat the fruit of thine own body, the flesh of thy sons and of thy daughters, which the LORD thy God hath given thee, in the siege, and in the straitness, wherewith thine enemies shall distress thee:"],
['Deuteronomy','Due','28','054','54',"\emph{So} \emph{that} the man \emph{that} \emph{is} tender among you, and very delicate, his eye shall be evil toward his brother, and toward the wife of his bosom, and toward the remnant of his children which he shall leave:"],
['Deuteronomy','Due','28','055','55',"So that he will not give to any of them of the flesh of his children whom he shall eat: because he hath nothing left him in the siege, and in the straitness, wherewith thine enemies shall distress thee in all thy gates."],
['Deuteronomy','Due','28','056','56',"The tender and delicate woman among you, which would not adventure to set the sole of her foot upon the ground for delicateness and tenderness, her eye shall be evil toward the husband of her bosom, and toward her son, and toward her daughter,"],
['Deuteronomy','Due','28','057','57',"And toward her young one that cometh out from between her feet, and toward her children which she shall bear: for she shall eat them for want of all \emph{things} secretly in the siege and straitness, wherewith thine enemy shall distress thee in thy gates."],
['Deuteronomy','Due','28','058','58',"If thou wilt not observe to do all the words of this law that are written in this book, that thou mayest fear this glorious and fearful name, THE LORD THY GOD;"],
['Deuteronomy','Due','28','059','59',"Then the LORD will make thy plagues wonderful, and the plagues of thy seed, \emph{even} great plagues, and of long continuance, and sore sicknesses, and of long continuance."],
['Deuteronomy','Due','28','060','60',"Moreover he will bring upon thee all the diseases of Egypt, which thou wast afraid of; and they shall cleave unto thee."],
['Deuteronomy','Due','28','061','61',"Also every sickness, and every plague, which \emph{is} not written in the book of this law, them will the LORD bring upon thee, until thou be destroyed."],
['Deuteronomy','Due','28','062','62',"And ye shall be left few in number, whereas ye were as the stars of heaven for multitude; because thou wouldest not obey the voice of the LORD thy God."],
['Deuteronomy','Due','28','063','63',"And it shall come to pass, \emph{that} as the LORD rejoiced over you to do you good, and to multiply you; so the LORD will rejoice over you to destroy you, and to bring you to nought; and ye shall be plucked from off the land whither thou goest to possess it."],
['Deuteronomy','Due','28','064','64',"And the LORD shall scatter thee among all people, from the one end of the earth even unto the other; and there thou shalt serve other gods, which neither thou nor thy fathers have known, \emph{even} wood and stone."],
['Deuteronomy','Due','28','065','65',"And among these nations shalt thou find no ease, neither shall the sole of thy foot have rest: but the LORD shall give thee there a trembling heart, and failing of eyes, and sorrow of mind:"],
['Deuteronomy','Due','28','066','66',"And thy life shall hang in doubt before thee; and thou shalt fear day and night, and shalt have none assurance of thy life:"],
['Deuteronomy','Due','28','067','67',"In the morning thou shalt say, Would God it were even! and at even thou shalt say, Would God it were morning! for the fear of thine heart wherewith thou shalt fear, and for the sight of thine eyes which thou shalt see."],
['Deuteronomy','Due','28','068','68',"And the LORD shall bring thee into Egypt again with ships, by the way whereof I spake unto thee, Thou shalt see it no more again: and there ye shall be sold unto your enemies for bondmen and bondwomen, and no man shall buy \emph{you}."]]

Deuteronomy29 = [
        ['Deuteronomy','Due','29','001','1',"These \emph{are} the words of the covenant, which the LORD commanded Moses to make with the children of Israel in the land of Moab, beside the covenant which he made with them in Horeb."],
['Deuteronomy','Due','29','002','2',"And Moses called unto all Israel, and said unto them, Ye have seen all that the LORD did before your eyes in the land of Egypt unto Pharaoh, and unto all his servants, and unto all his land;"],
['Deuteronomy','Due','29','003','3',"The great temptations which thine eyes have seen, the signs, and those great miracles:"],
['Deuteronomy','Due','29','004','4',"Yet the LORD hath not given you an heart to perceive, and eyes to see, and ears to hear, unto this day."],
['Deuteronomy','Due','29','005','5',"And I have led you forty years in the wilderness: your clothes are not waxen old upon you, and thy shoe is not waxen old upon thy foot."],
['Deuteronomy','Due','29','006','6',"Ye have not eaten bread, neither have ye drunk wine or strong drink: that ye might know that I \emph{am} the LORD your God."],
['Deuteronomy','Due','29','007','7',"And when ye came unto this place, Sihon the king of Heshbon, and Og the king of Bashan, came out against us unto battle, and we smote them:"],
['Deuteronomy','Due','29','008','8',"And we took their land, and gave it for an inheritance unto the Reubenites, and to the Gadites, and to the half tribe of Manasseh."],
['Deuteronomy','Due','29','009','9',"Keep therefore the words of this covenant, and do them, that ye may prosper in all that ye do."],
['Deuteronomy','Due','29','010','10',"Ye stand this day all of you before the LORD your God; your captains of your tribes, your elders, and your officers, \emph{with} all the men of Israel,"],
['Deuteronomy','Due','29','011','11',"Your little ones, your wives, and thy stranger that \emph{is} in thy camp, from the hewer of thy wood unto the drawer of thy water:"],
['Deuteronomy','Due','29','012','12',"That thou shouldest enter into covenant with the LORD thy God, and into his oath, which the LORD thy God maketh with thee this day:"],
['Deuteronomy','Due','29','013','13',"That he may establish thee to day for a people unto himself, and \emph{that} he may be unto thee a God, as he hath said unto thee, and as he hath sworn unto thy fathers, to Abraham, to Isaac, and to Jacob."],
['Deuteronomy','Due','29','014','14',"Neither with you only do I make this covenant and this oath;"],
['Deuteronomy','Due','29','015','15',"But with \emph{him} that standeth here with us this day before the LORD our God, and also with \emph{him} that \emph{is} not here with us this day:"],
['Deuteronomy','Due','29','016','16',"(For ye know how we have dwelt in the land of Egypt; and how we came through the nations which ye passed by;"],
['Deuteronomy','Due','29','017','17',"And ye have seen their abominations, and their idols, wood and stone, silver and gold, which \emph{were} among them:)"],
['Deuteronomy','Due','29','018','18',"Lest there should be among you man, or woman, or family, or tribe, whose heart turneth away this day from the LORD our God, to go \emph{and} serve the gods of these nations; lest there should be among you a root that beareth gall and wormwood;"],
['Deuteronomy','Due','29','019','19',"And it come to pass, when he heareth the words of this curse, that he bless himself in his heart, saying, I shall have peace, though I walk in the imagination of mine heart, to add drunkenness to thirst:"],
['Deuteronomy','Due','29','020','20',"The LORD will not spare him, but then the anger of the LORD and his jealousy shall smoke against that man, and all the curses that are written in this book shall lie upon him, and the LORD shall blot out his name from under heaven."],
['Deuteronomy','Due','29','021','21',"And the LORD shall separate him unto evil out of all the tribes of Israel, according to all the curses of the covenant that are written in this book of the law:"],
['Deuteronomy','Due','29','022','22',"So that the generation to come of your children that shall rise up after you, and the stranger that shall come from a far land, shall say, when they see the plagues of that land, and the sicknesses which the LORD hath laid upon it;"],
['Deuteronomy','Due','29','023','23',"\emph{And} \emph{that} the whole land thereof \emph{is} brimstone, and salt, \emph{and} burning, \emph{that} it is not sown, nor beareth, nor any grass groweth therein, like the overthrow of Sodom, and Gomorrah, Admah, and Zeboim, which the LORD overthrew in his anger, and in his wrath:"],
['Deuteronomy','Due','29','024','24',"Even all nations shall say, Wherefore hath the LORD done thus unto this land? what \emph{meaneth} the heat of this great anger?"],
['Deuteronomy','Due','29','025','25',"Then men shall say, Because they have forsaken the covenant of the LORD God of their fathers, which he made with them when he brought them forth out of the land of Egypt:"],
['Deuteronomy','Due','29','026','26',"For they went and served other gods, and worshipped them, gods whom they knew not, and \emph{whom} he had not given unto them:"],
['Deuteronomy','Due','29','027','27',"And the anger of the LORD was kindled against this land, to bring upon it all the curses that are written in this book:"],
['Deuteronomy','Due','29','028','28',"And the LORD rooted them out of their land in anger, and in wrath, and in great indignation, and cast them into another land, as \emph{it} \emph{is} this day."],
['Deuteronomy','Due','29','029','29',"The secret \emph{things} \emph{belong} unto the LORD our God: but those \emph{things} \emph{which} \emph{are} revealed \emph{belong} unto us and to our children for ever, that \emph{we} may do all the words of this law."]
]


Deuteronomy30 = [
    ['Deuteronomy','Due','30','001','1',"And it shall come to pass, when all these things are come upon thee, the blessing and the curse, which I have set before thee, and thou shalt call \emph{them} to mind among all the nations, whither the LORD thy God hath driven thee,"],
['Deuteronomy','Due','30','002','2',"And shalt return unto the LORD thy God, and shalt obey his voice according to all that I command thee this day, thou and thy children, with all thine heart, and with all thy soul;"],
['Deuteronomy','Due','30','003','3',"That then the LORD thy God will turn thy captivity, and have compassion upon thee, and will return and gather thee from all the nations, whither the LORD thy God hath scattered thee."],
['Deuteronomy','Due','30','004','4',"If \emph{any} of thine be driven out unto the outmost \emph{parts} of heaven, from thence will the LORD thy God gather thee, and from thence will he fetch thee:"],
['Deuteronomy','Due','30','005','5',"And the LORD thy God will bring thee into the land which thy fathers possessed, and thou shalt possess it; and he will do thee good, and multiply thee above thy fathers."],
['Deuteronomy','Due','30','006','6',"And the LORD thy God will circumcise thine heart, and the heart of thy seed, to love the LORD thy God with all thine heart, and with all thy soul, that thou mayest live."],
['Deuteronomy','Due','30','007','7',"And the LORD thy God will put all these curses upon thine enemies, and on them that hate thee, which persecuted thee."],
['Deuteronomy','Due','30','008','8',"And thou shalt return and obey the voice of the LORD, and do all his commandments which I command thee this day."],
['Deuteronomy','Due','30','009','9',"And the LORD thy God will make thee plenteous in every work of thine hand, in the fruit of thy body, and in the fruit of thy cattle, and in the fruit of thy land, for good: for the LORD will again rejoice over thee for good, as he rejoiced over thy fathers:"],
['Deuteronomy','Due','30','010','10',"If thou shalt hearken unto the voice of the LORD thy God, to keep his commandments and his statutes which are written in this book of the law, \emph{and} if thou turn unto the LORD thy God with all thine heart, and with all thy soul."],
['Deuteronomy','Due','30','011','11',"For this commandment which I command thee this day, it \emph{is} not hidden from thee, neither \emph{is} it far off."],
['Deuteronomy','Due','30','012','12',"It \emph{is} not in heaven, that thou shouldest say, Who shall go up for us to heaven, and bring it unto us, that we may hear it, and do it?"],
['Deuteronomy','Due','30','013','13',"Neither \emph{is} it beyond the sea, that thou shouldest say, Who shall go over the sea for us, and bring it unto us, that we may hear it, and do it?"],
['Deuteronomy','Due','30','014','14',"But the word \emph{is} very nigh unto thee, in thy mouth, and in thy heart, that thou mayest do it."],
['Deuteronomy','Due','30','015','15',"See, I have set before thee this day life and good, and death and evil;"],
['Deuteronomy','Due','30','016','16',"In that I command thee this day to love the LORD thy God, to walk in his ways, and to keep his commandments and his statutes and his judgments, that thou mayest live and multiply: and the LORD thy God shall bless thee in the land whither thou goest to possess it."],
['Deuteronomy','Due','30','017','17',"But if thine heart turn away, so that thou wilt not hear, but shalt be drawn away, and worship other gods, and serve them;"],
['Deuteronomy','Due','30','018','18',"I denounce unto you this day, that ye shall surely perish, \emph{and} \emph{that} ye shall not prolong \emph{your} days upon the land, whither thou passest over Jordan to go to possess it."],
['Deuteronomy','Due','30','019','19',"I call heaven and earth to record this day against you, \emph{that} I have set before you life and death, blessing and cursing: therefore choose life, that both thou and thy seed may live:"],
['Deuteronomy','Due','30','020','20',"That thou mayest love the LORD thy God, \emph{and} that thou mayest obey his voice, and that thou mayest cleave unto him: for he \emph{is} thy life, and the length of thy days: that thou mayest dwell in the land which the LORD sware unto thy fathers, to Abraham, to Isaac, and to Jacob, to give them."]]


Deuteronomy31 = [
    ['Deuteronomy','Due','31','001','1',"And Moses went and spake these words unto all Israel."],
['Deuteronomy','Due','31','002','2',"And he said unto them, I \emph{am} an hundred and twenty years old this day; I can no more go out and come in: also the LORD hath said unto me, Thou shalt not go over this Jordan."],
['Deuteronomy','Due','31','003','3',"The LORD thy God, he will go over before thee, \emph{and} he will destroy these nations from before thee, and thou shalt possess them: \emph{and} Joshua, he shall go over before thee, as the LORD hath said."],
['Deuteronomy','Due','31','004','4',"And the LORD shall do unto them as he did to Sihon and to Og, kings of the Amorites, and unto the land of them, whom he destroyed."],
['Deuteronomy','Due','31','005','5',"And the LORD shall give them up before your face, that ye may do unto them according unto all the commandments which I have commanded you."],
['Deuteronomy','Due','31','006','6',"Be strong and of a good courage, fear not, nor be afraid of them: for the LORD thy God, he \emph{it} \emph{is} that doth go with thee; he will not fail thee, nor forsake thee."],
['Deuteronomy','Due','31','007','7',"And Moses called unto Joshua, and said unto him in the sight of all Israel, Be strong and of a good courage: for thou must go with this people unto the land which the LORD hath sworn unto their fathers to give them; and thou shalt cause them to inherit it."],
['Deuteronomy','Due','31','008','8',"And the LORD, he \emph{it} \emph{is} that doth go before thee; he will be with thee, he will not fail thee, neither forsake thee: fear not, neither be dismayed."],
['Deuteronomy','Due','31','009','9',"And Moses wrote this law, and delivered it unto the priests the sons of Levi, which bare the ark of the covenant of the LORD, and unto all the elders of Israel."],
['Deuteronomy','Due','31','010','10',"And Moses commanded them, saying, At the end of \emph{every} seven years, in the solemnity of the year of release, in the feast of tabernacles,"],
['Deuteronomy','Due','31','011','11',"When all Israel is come to appear before the LORD thy God in the place which he shall choose, thou shalt read this law before all Israel in their hearing."],
['Deuteronomy','Due','31','012','12',"Gather the people together, men, and women, and children, and thy stranger that \emph{is} within thy gates, that they may hear, and that they may learn, and fear the LORD your God, and observe to do all the words of this law:"],
['Deuteronomy','Due','31','013','13',"And \emph{that} their children, which have not known \emph{any} \emph{thing}, may hear, and learn to fear the LORD your God, as long as ye live in the land whither ye go over Jordan to possess it."],
['Deuteronomy','Due','31','014','14',"And the LORD said unto Moses, Behold, thy days approach that thou must die: call Joshua, and present yourselves in the tabernacle of the congregation, that I may give him a charge. And Moses and Joshua went, and presented themselves in the tabernacle of the congregation."],
['Deuteronomy','Due','31','015','15',"And the LORD appeared in the tabernacle in a pillar of a cloud: and the pillar of the cloud stood over the door of the tabernacle."],
['Deuteronomy','Due','31','016','16',"And the LORD said unto Moses, Behold, thou shalt sleep with thy fathers; and this people will rise up, and go a whoring after the gods of the strangers of the land, whither they go \emph{to} \emph{be} among them, and will forsake me, and break my covenant which I have made with them."],
['Deuteronomy','Due','31','017','17',"Then my anger shall be kindled against them in that day, and I will forsake them, and I will hide my face from them, and they shall be devoured, and many evils and troubles shall befall them; so that they will say in that day, Are not these evils come upon us, because our God \emph{is} not among us?"],
['Deuteronomy','Due','31','018','18',"And I will surely hide my face in that day for all the evils which they shall have wrought, in that they are turned unto other gods."],
['Deuteronomy','Due','31','019','19',"Now therefore write ye this song for you, and teach it the children of Israel: put it in their mouths, that this song may be a witness for me against the children of Israel."],
['Deuteronomy','Due','31','020','20',"For when I shall have brought them into the land which I sware unto their fathers, that floweth with milk and honey; and they shall have eaten and filled themselves, and waxen fat; then will they turn unto other gods, and serve them, and provoke me, and break my covenant."],
['Deuteronomy','Due','31','021','21',"And it shall come to pass, when many evils and troubles are befallen them, that this song shall testify against them as a witness; for it shall not be forgotten out of the mouths of their seed: for I know their imagination which they go about, even now, before I have brought them into the land which I sware."],
['Deuteronomy','Due','31','022','22',"Moses therefore wrote this song the same day, and taught it the children of Israel."],
['Deuteronomy','Due','31','023','23',"And he gave Joshua the son of Nun a charge, and said, Be strong and of a good courage: for thou shalt bring the children of Israel into the land which I sware unto them: and I will be with thee."],
['Deuteronomy','Due','31','024','24',"And it came to pass, when Moses had made an end of writing the words of this law in a book, until they were finished,"],
['Deuteronomy','Due','31','025','25',"That Moses commanded the Levites, which bare the ark of the covenant of the LORD, saying,"],
['Deuteronomy','Due','31','026','26',"Take this book of the law, and put it in the side of the ark of the covenant of the LORD your God, that it may be there for a witness against thee."],
['Deuteronomy','Due','31','027','27',"For I know thy rebellion, and thy stiff neck: behold, while I am yet alive with you this day, ye have been rebellious against the LORD; and how much more after my death?"],
['Deuteronomy','Due','31','028','28',"Gather unto me all the elders of your tribes, and your officers, that I may speak these words in their ears, and call heaven and earth to record against them."],
['Deuteronomy','Due','31','029','29',"For I know that after my death ye will utterly corrupt \emph{yourselves}, and turn aside from the way which I have commanded you; and evil will befall you in the latter days; because ye will do evil in the sight of the LORD, to provoke him to anger through the work of your hands."],
['Deuteronomy','Due','31','030','30',"And Moses spake in the ears of all the congregation of Israel the words of this song, until they were ended."]
]

Deuteronomy32 = [
    ['Deuteronomy','Due','32','001','1',"Give ear, O ye heavens, and I will speak; and hear, O earth, the words of my mouth."],
['Deuteronomy','Due','32','002','2',"My doctrine shall drop as the rain, my speech shall distil as the dew, as the small rain upon the tender herb, and as the showers upon the grass:"],
['Deuteronomy','Due','32','003','3',"Because I will publish the name of the LORD: ascribe ye greatness unto our God."],
['Deuteronomy','Due','32','004','4',"\emph{He} \emph{is} the Rock, his work \emph{is} perfect: for all his ways \emph{are} judgment: a God of truth and without iniquity, just and right \emph{is} he."],
['Deuteronomy','Due','32','005','5',"They have corrupted themselves, their spot \emph{is} not \emph{the} \emph{spot} of his children: \emph{they} \emph{are} a perverse and crooked generation."],
['Deuteronomy','Due','32','006','6',"Do ye thus requite the LORD, O foolish people and unwise? \emph{is} not he thy father \emph{that} hath bought thee? hath he not made thee, and established thee?"],
['Deuteronomy','Due','32','007','7',"Remember the days of old, consider the years of many generations: ask thy father, and he will shew thee; thy elders, and they will tell thee."],
['Deuteronomy','Due','32','008','8',"When the most High divided to the nations their inheritance, when he separated the sons of Adam, he set the bounds of the people according to the number of the children of Israel."],
['Deuteronomy','Due','32','009','9',"For the LORD'S portion \emph{is} his people; Jacob \emph{is} the lot of his inheritance."],
['Deuteronomy','Due','32','010','10',"He found him in a desert land, and in the waste howling wilderness; he led him about, he instructed him, he kept him as the apple of his eye."],
['Deuteronomy','Due','32','011','11',"As an eagle stirreth up her nest, fluttereth over her young, spreadeth abroad her wings, taketh them, beareth them on her wings:"],
['Deuteronomy','Due','32','012','12',"\emph{So} the LORD alone did lead him, and \emph{there} \emph{was} no strange god with him."],
['Deuteronomy','Due','32','013','13',"He made him ride on the high places of the earth, that he might eat the increase of the fields; and he made him to suck honey out of the rock, and oil out of the flinty rock;"],
['Deuteronomy','Due','32','014','14',"Butter of kine, and milk of sheep, with fat of lambs, and rams of the breed of Bashan, and goats, with the fat of kidneys of wheat; and thou didst drink the pure blood of the grape."],
['Deuteronomy','Due','32','015','15',"But Jeshurun waxed fat, and kicked: thou art waxen fat, thou art grown thick, thou art covered \emph{with} \emph{fatness}; then he forsook God \emph{which} made him, and lightly esteemed the Rock of his salvation."],
['Deuteronomy','Due','32','016','16',"They provoked him to jealousy with strange \emph{gods}, with abominations provoked they him to anger."],
['Deuteronomy','Due','32','017','17',"They sacrificed unto devils, not to God; to gods whom they knew not, to new \emph{gods} \emph{that} came newly up, whom your fathers feared not."],
['Deuteronomy','Due','32','018','18',"Of the Rock \emph{that} begat thee thou art unmindful, and hast forgotten God that formed thee."],
['Deuteronomy','Due','32','019','19',"And when the LORD saw \emph{it}, he abhorred \emph{them}, because of the provoking of his sons, and of his daughters."],
['Deuteronomy','Due','32','020','20',"And he said, I will hide my face from them, I will see what their end \emph{shall} \emph{be}: for they \emph{are} a very froward generation, children in whom \emph{is} no faith."],
['Deuteronomy','Due','32','021','21',"They have moved me to jealousy with \emph{that} \emph{which} \emph{is} not God; they have provoked me to anger with their vanities: and I will move them to jealousy with \emph{those} \emph{which} \emph{are} not a people; I will provoke them to anger with a foolish nation."],
['Deuteronomy','Due','32','022','22',"For a fire is kindled in mine anger, and shall burn unto the lowest hell, and shall consume the earth with her increase, and set on fire the foundations of the mountains."],
['Deuteronomy','Due','32','023','23',"I will heap mischiefs upon them; I will spend mine arrows upon them."],
['Deuteronomy','Due','32','024','24',"\emph{They} \emph{shall} \emph{be} burnt with hunger, and devoured with burning heat, and with bitter destruction: I will also send the teeth of beasts upon them, with the poison of serpents of the dust."],
['Deuteronomy','Due','32','025','25',"The sword without, and terror within, shall destroy both the young man and the virgin, the suckling \emph{also} with the man of gray hairs."],
['Deuteronomy','Due','32','026','26',"I said, I would scatter them into corners, I would make the remembrance of them to cease from among men:"],
['Deuteronomy','Due','32','027','27',"Were it not that I feared the wrath of the enemy, lest their adversaries should behave themselves strangely, \emph{and} lest they should say, Our hand \emph{is} high, and the LORD hath not done all this."],
['Deuteronomy','Due','32','028','28',"For they \emph{are} a nation void of counsel, neither \emph{is} \emph{there} \emph{any} understanding in them."],
['Deuteronomy','Due','32','029','29',"O that they were wise, \emph{that} they understood this, \emph{that} they would consider their latter end!"],
['Deuteronomy','Due','32','030','30',"How should one chase a thousand, and two put ten thousand to flight, except their Rock had sold them, and the LORD had shut them up?"],
['Deuteronomy','Due','32','031','31',"For their rock \emph{is} not as our Rock, even our enemies themselves \emph{being} judges."],
['Deuteronomy','Due','32','032','32',"For their vine \emph{is} of the vine of Sodom, and of the fields of Gomorrah: their grapes \emph{are} grapes of gall, their clusters \emph{are} bitter:"],
['Deuteronomy','Due','32','033','33',"Their wine \emph{is} the poison of dragons, and the cruel venom of asps."],
['Deuteronomy','Due','32','034','34',"\emph{Is} not this laid up in store with me, \emph{and} sealed up among my treasures?"],
['Deuteronomy','Due','32','035','35',"To me \emph{belongeth} vengeance, and recompence; their foot shall slide in \emph{due} time: for the day of their calamity \emph{is} at hand, and the things that shall come upon them make haste."],
['Deuteronomy','Due','32','036','36',"For the LORD shall judge his people, and repent himself for his servants, when he seeth that \emph{their} power is gone, and \emph{there} \emph{is} none shut up, or left."],
['Deuteronomy','Due','32','037','37',"And he shall say, Where \emph{are} their gods, \emph{their} rock in whom they trusted,"],
['Deuteronomy','Due','32','038','38',"Which did eat the fat of their sacrifices, \emph{and} drank the wine of their drink offerings? let them rise up and help you, \emph{and} be your protection."],
['Deuteronomy','Due','32','039','39',"See now that I, \emph{even} I, \emph{am} he, and \emph{there} \emph{is} no god with me: I kill, and I make alive; I wound, and I heal: neither \emph{is} \emph{there} \emph{any} that can deliver out of my hand."],
['Deuteronomy','Due','32','040','40',"For I lift up my hand to heaven, and say, I live for ever."],
['Deuteronomy','Due','32','041','41',"If I whet my glittering sword, and mine hand take hold on judgment; I will render vengeance to mine enemies, and will reward them that hate me."],
['Deuteronomy','Due','32','042','42',"I will make mine arrows drunk with blood, and my sword shall devour flesh; \emph{and} \emph{that} with the blood of the slain and of the captives, from the beginning of revenges upon the enemy."],
['Deuteronomy','Due','32','043','43',"Rejoice, O ye nations, \emph{with} his people: for he will avenge the blood of his servants, and will render vengeance to his adversaries, and will be merciful unto his land, \emph{and} to his people."],
['Deuteronomy','Due','32','044','44',"And Moses came and spake all the words of this song in the ears of the people, he, and Hoshea the son of Nun."],
['Deuteronomy','Due','32','045','45',"And Moses made an end of speaking all these words to all Israel:"],
['Deuteronomy','Due','32','046','46',"And he said unto them, Set your hearts unto all the words which I testify among you this day, which ye shall command your children to observe to do, all the words of this law."],
['Deuteronomy','Due','32','047','47',"For it \emph{is} not a vain thing for you; because it \emph{is} your life: and through this thing ye shall prolong \emph{your} days in the land, whither ye go over Jordan to possess it."],
['Deuteronomy','Due','32','048','48',"And the LORD spake unto Moses that selfsame day, saying,"],
['Deuteronomy','Due','32','049','49',"Get thee up into this mountain Abarim, \emph{unto} mount Nebo, which \emph{is} in the land of Moab, that \emph{is} over against Jericho; and behold the land of Canaan, which I give unto the children of Israel for a possession:"],
['Deuteronomy','Due','32','050','50',"And die in the mount whither thou goest up, and be gathered unto thy people; as Aaron thy brother died in mount Hor, and was gathered unto his people:"],
['Deuteronomy','Due','32','051','51',"Because ye trespassed against me among the children of Israel at the waters of Meribah-Kadesh, in the wilderness of Zin; because ye sanctified me not in the midst of the children of Israel."],
['Deuteronomy','Due','32','052','52',"Yet thou shalt see the land before \emph{thee}; but thou shalt not go thither unto the land which I give the children of Israel."]
]


Deuteronomy33 = [
    ['Deuteronomu','Dt','33','001','1',"And this \emph{is} the blessing, wherewith Moses the man of God blessed the children of Israel before his death."],
['Deuteronomu','Dt','33','002','2',"And he said, The LORD came from Sinai, and rose up from Seir unto them; he shined forth from mount Paran, and he came with ten thousands of saints: from his right hand \emph{went} a fiery law for them."],
['Deuteronomu','Dt','33','003','3',"Yea, he loved the people; all his saints \emph{are} in thy hand: and they sat down at thy feet; \emph{every} \emph{one} shall receive of thy words."],
['Deuteronomu','Dt','33','004','4',"Moses commanded us a law, \emph{even} the inheritance of the congregation of Jacob."],
['Deuteronomu','Dt','33','005','5',"And he was king in Jeshurun, when the heads of the people \emph{and} the tribes of Israel were gathered together."],
['Deuteronomu','Dt','33','006','6',"Let Reuben live, and not die; and let \emph{not} his men be few."],
['Deuteronomu','Dt','33','007','7',"And this \emph{is} \emph{the} \emph{blessing} of Judah: and he said, Hear, LORD, the voice of Judah, and bring him unto his people: let his hands be sufficient for him; and be thou an help \emph{to} \emph{him} from his enemies."],
['Deuteronomu','Dt','33','008','8',"And of Levi he said, \emph{Let} thy Thummim and thy Urim \emph{be} with thy holy one, whom thou didst prove at Massah, \emph{and} \emph{with} whom thou didst strive at the waters of Meribah;"],
['Deuteronomu','Dt','33','009','9',"Who said unto his father and to his mother, I have not seen him; neither did he acknowledge his brethren, nor knew his own children: for they have observed thy word, and kept thy covenant."],
['Deuteronomu','Dt','33','010','10',"They shall teach Jacob thy judgments, and Israel thy law: they shall put incense before thee, and whole burnt sacrifice upon thine altar."],
['Deuteronomu','Dt','33','011','11',"Bless, LORD, his substance, and accept the work of his hands: smite through the loins of them that rise against him, and of them that hate him, that they rise not again."],
['Deuteronomu','Dt','33','012','12',"\emph{And} of Benjamin he said, The beloved of the LORD shall dwell in safety by him; \emph{and} \emph{the} \emph{LORD} shall cover him all the day long, and he shall dwell between his shoulders."],
['Deuteronomu','Dt','33','013','13',"And of Joseph he said, Blessed of the LORD \emph{be} his land, for the precious things of heaven, for the dew, and for the deep that coucheth beneath,"],
['Deuteronomu','Dt','33','014','14',"And for the precious fruits \emph{brought} \emph{forth} by the sun, and for the precious things put forth by the moon,"],
['Deuteronomu','Dt','33','015','15',"And for the chief things of the ancient mountains, and for the precious things of the lasting hills,"],
['Deuteronomu','Dt','33','016','16',"And for the precious things of the earth and fulness thereof, and \emph{for} the good will of him that dwelt in the bush: let \emph{the} \emph{blessing} come upon the head of Joseph, and upon the top of the head of him \emph{that} \emph{was} separated from his brethren."],
['Deuteronomu','Dt','33','017','17',"His glory \emph{is} \emph{like} the firstling of his bullock, and his horns \emph{are} \emph{like} the horns of unicorns: with them he shall push the people together to the ends of the earth: and they \emph{are} the ten thousands of Ephraim, and they \emph{are} the thousands of Manasseh."],
['Deuteronomu','Dt','33','018','18',"And of Zebulun he said, Rejoice, Zebulun, in thy going out; and, Issachar, in thy tents."],
['Deuteronomu','Dt','33','019','19',"They shall call the people unto the mountain; there they shall offer sacrifices of righteousness: for they shall suck \emph{of} the abundance of the seas, and \emph{of} treasures hid in the sand."],
['Deuteronomu','Dt','33','020','20',"And of Gad he said, Blessed \emph{be} he that enlargeth Gad: he dwelleth as a lion, and teareth the arm with the crown of the head."],
['Deuteronomu','Dt','33','021','21',"And he provided the first part for himself, because there, \emph{in} a portion of the lawgiver, \emph{was} \emph{he} seated; and he came with the heads of the people, he executed the justice of the LORD, and his judgments with Israel."],
['Deuteronomu','Dt','33','022','22',"And of Dan he said, Dan \emph{is} a lion's whelp: he shall leap from Bashan."],
['Deuteronomu','Dt','33','023','23',"And of Naphtali he said, O Naphtali, satisfied with favour, and full with the blessing of the LORD: possess thou the west and the south."],
['Deuteronomu','Dt','33','024','24',"And of Asher he said, \emph{Let} Asher \emph{be} blessed with children; let him be acceptable to his brethren, and let him dip his foot in oil."],
['Deuteronomu','Dt','33','025','25',"Thy shoes \emph{shall} \emph{be} iron and brass; and as thy days, \emph{so} \emph{shall} thy strength \emph{be}."],
['Deuteronomu','Dt','33','026','26',"\emph{There} \emph{is} none like unto the God of Jeshurun, \emph{who} rideth upon the heaven in thy help, and in his excellency on the sky."],
['Deuteronomu','Dt','33','027','27',"The eternal God \emph{is} \emph{thy} refuge, and underneath \emph{are} the everlasting arms: and he shall thrust out the enemy from before thee; and shall say, Destroy \emph{them}."],
['Deuteronomu','Dt','33','028','28',"Israel then shall dwell in safety alone: the fountain of Jacob \emph{shall} \emph{be} upon a land of corn and wine; also his heavens shall drop down dew."],
['Deuteronomu','Dt','33','029','29',"Happy \emph{art} thou, O Israel: who \emph{is} like unto thee, O people saved by the LORD, the shield of thy help, and who \emph{is} the sword of thy excellency! and thine enemies shall be found liars unto thee; and thou shalt tread upon their high places."]
]

Deuteronomy34 = [
['Deuteronomy','Dt','34','001','1',"And Moses went up from the plains of Moab unto the mountain of Nebo, to the top of Pisgah, that \emph{is} over against Jericho. And the LORD shewed him all the land of Gilead, unto Dan,"],
['Deuteronomy','Dt','34','002','2',"And all Naphtali, and the land of Ephraim, and Manasseh, and all the land of Judah, unto the utmost sea,"],
['Deuteronomy','Dt','34','003','3',"And the south, and the plain of the valley of Jericho, the city of palm trees, unto Zoar."],
['Deuteronomy','Dt','34','004','4',"And the LORD said unto him, This \emph{is} the land which I sware unto Abraham, unto Isaac, and unto Jacob, saying, I will give it unto thy seed: I have caused thee to see \emph{it} with thine eyes, but thou shalt not go over thither."],
['Deuteronomy','Dt','34','005','5',"So Moses the servant of the LORD died there in the land of Moab, according to the word of the LORD."],
['Deuteronomy','Dt','34','006','6',"And he buried him in a valley in the land of Moab, over against Beth-peor: but no man knoweth of his sepulchre unto this day."],
['Deuteronomy','Dt','34','007','7',"And Moses \emph{was} an hundred and twenty years old when he died: his eye was not dim, nor his natural force abated."],
['Deuteronomy','Dt','34','008','8',"And the children of Israel wept for Moses in the plains of Moab thirty days: so the days of weeping \emph{and} mourning for Moses were ended."],
['Deuteronomy','Dt','34','009','9',"And Joshua the son of Nun was full of the spirit of wisdom; for Moses had laid his hands upon him: and the children of Israel hearkened unto him, and did as the LORD commanded Moses."],
['Deuteronomy','Dt','34','010','10',"And there arose not a prophet since in Israel like unto Moses, whom the LORD knew face to face,"],
['Deuteronomy','Dt','34','011','11',"In all the signs and the wonders, which the LORD sent him to do in the land of Egypt to Pharaoh, and to all his servants, and to all his land,"],
['Deuteronomy','Dt','34','012','12',"And in all that mighty hand, and in all the great terror which Moses shewed in the sight of all Israel."]
]

def main():
    ProcessVerses('Deuteronomy34',Deuteronomy34)

if __name__ == "__main__":
    main()



%%%%%%%%%%%%%%%%%%%%%%%%%%%
Chapter with Latex format
%%%%%%%%%%%%%%%%%%%%%%%%%%%


\footnote{\textcolor[cmyk]{0.99998,1,0,0}{\hyperlink{TOC}{Return to end of Table of Contents.}}}\textcolor[cmyk]{0.99998,1,0,0}{And Moses went up from the plains of Moab unto the mountain of Nebo, to the top of Pisgah, that \emph{is} over against Jericho. And the LORD shewed him all the land of Gilead, unto Dan,}
[2] \textcolor[cmyk]{0.99998,1,0,0}{And all Naphtali, and the land of Ephraim, and Manasseh, and all the land of Judah, unto the utmost sea,}
[3] \textcolor[cmyk]{0.99998,1,0,0}{And the south, and the plain of the valley of Jericho, the city of palm trees, unto Zoar.}
[4] \textcolor[cmyk]{0.99998,1,0,0}{And the LORD said unto him, This \emph{is} the land which I sware unto Abraham, unto Isaac, and unto Jacob, saying, I will give it unto thy seed: I have caused thee to see \emph{it} with thine eyes, but thou shalt not go over thither.}
[5] \textcolor[cmyk]{0.99998,1,0,0}{So Moses the servant o